* lix and rix stylo metrics
* rollover for sentiment and readibility
* KDE distribution plot
* summary statistics before ascore norm


# **DIRECTIONS:** Please Read First

Browser:
* Must use **Chrome** browser (esp not Safari)

Colab:
* Use **GPU Runtime (e.g. T4 high memory)** for Ollama LLMs and Transformer Models

Input Raw Text File:
* Filename Format **(TitleInCamelCase)_(FnameLnameInCamelCase).txt**
* Use only **plain text** files (no *.rtf, *.doc, etc)
* ***Headers/Footers deleted***, only first line to last line of novel text
* ***Paragraphs*** separarted by at least **two blank lines**
* ***Chapters/Sections*** separated by line starting with **'CHAPTER...'** and preceeded/suceeded by at least two blank lines
* Encode in **'utf-8'**

Novels (Get plain text if possible):
* https://gutenberg.net.au/ (AUS)
* https://gutenberg.org/ (US)

Notebook Notation:
* **OPTION (n)** means execute only **ONE** of the OPTIONS provided
* **STEP (n)** means execute **ALL** of the STEPS that follow

# SentimentArcs Simplified Notebook

Created:

* 1 June 2024
* Jon Chun

A simplified version of SentimentArcs Notebooks for use with diachronic sentiment and stylometric analysis and time series plot.

* https://github.com/jon-chun/sentimentarcs_notebooks

* https://arxiv.org/pdf/2110.09454.pdfol

# Install Libraries

## SpaCy

In [ ]:
!pip install -U spacy

## SpaCy English Models (RESTART REQUIRED)

In [ ]:
# Download SpaCy English Model

!python -m spacy download en_core_web_lg

## SpaCy French Models (RESTART REQUIRED)

In [ ]:
# Download SpaCy French Model

!python -m spacy download fr_core_news_lg

## SpaCy German Models (RESTART REQUIRED)

In [ ]:
# Download SpaCy German Model

!python -m spacy download de_core_news_lg

## **[RESTART RUNTIME]**

## Ollama LLM Server

In [ ]:
#Install package and load the extension
!pip install colab-xterm
%load_ext colabxterm

### mistral7bsenti.modelfile

```
PARAMETER temperature 0.0
PARAMETER top_p 0.5
PARAMETER seed 42
PARAMETER num_predict 5
SYSTEM """You are a text sentiment analysis engine that responds with only one float number for the sentiment polarity of the input text. You only reply with one float number between -1.0 and 1.0 which represent the most negative to most positive sentiment polarity. Use 0.0 for perfectly neutral sentiment. Do not respond with any other text. Do not give an greeting, explaination, definition, introduction, overview or conclusion. Only reply with the float number representing the sentiment polarity of the input text."""

NOTE: .modelfile is very sensitive to cut-and-paste hidden characters. If all else fails, manually retype the above into vi editor
```

**NOTE:** .modelfile is very sensitive to cut-and-paste hidden characters. If all else fails, manually retype the above into vi editor

In [ ]:
%xterm

# curl -fsSL https://ollama.com/install.sh | sh

# ollama serve & ollama pull mistral

# ollama pull mistral

# lsof -i :11434

# ollama show mistral --modelfile > mistral7bsenti.modelfile

# vi mistral7bsenti.modelfile (insert new PARAMETERS and SYSTEM lines above)

# ollama create mistral7bsenti --file mistral7bsenti.modelfile


In [ ]:
!lsof -i :11434

## Ollama, LangChain and Transformers

In [ ]:
!pip install ollama

In [ ]:
!pip install Transformers

## NLP and Spacy


In [ ]:
!pip install pysbd

In [ ]:
!pip install langdetect

In [ ]:
!pip install ftfy

In [ ]:
!pip install chardet

## Stylometry

In [ ]:
# https://hlasse.github.io/TextDescriptives/

!pip install textdescriptives

In [ ]:
# https://github.com/LSYS/lexicalrichness

!pip install lexicalrichness

## Numeric and Graphing

In [ ]:
!pip install seaborn

In [ ]:
!pip install scikit-learn

# Import Libraries

## Common Libraries

In [ ]:
from google.colab import files

In [ ]:
import os
import shutil
import re
import json
import pprint
import logging
import time
import copy
import pandas as pd

from itertools import cycle

import numpy as np
import pandas as pd

import string
import datetime
from typing import Dict, List, Tuple
import glob


import unicodedata

import pickle
import gc

import getpass

from pprint import pprint

from tqdm import tqdm
# from tqdm.auto import tqdm
# from tqdm.notebook import tqdm

from itertools import combinations
import random

# 20240525 from cleantext import clean
# 20240525 import contractions


## Ollama, LangChain and Transformers

#### ollama-python JSON

In [ ]:
import ollama

In [ ]:
%%time

# TEST:

response = ollama.chat(
    model='mistral7bsenti',
    # model='mistral',
    # messages=[{'role': 'user', 'content': 'Why is the sky blue?'}],
    messages=[{'role': 'user', 'content': 'I was unimpressed with the spectacle of the event?'}],
    stream=False,
)

print(response['message']['content'])

In [ ]:
# from langchain_community.llms import Ollama

## NLP and SpaCy

In [ ]:
import chardet

In [ ]:
from langdetect import detect, DetectorFactory

In [ ]:
import pysbd

In [ ]:
import spacy

In [ ]:
# Preinstalled on Google Colab, not on runpod.io VMs

import ftfy

## Stylometry


In [ ]:
import lexicalrichness

In [ ]:
from lexicalrichness import LexicalRichness

In [ ]:
import textdescriptives as td

## Numeric and Graphing

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

import plotly.express as px

In [ ]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler

from scipy.signal import find_peaks
from scipy.signal import savgol_filter

from scipy.stats import gaussian_kde
from scipy.stats import zscore


# Configuration

In [ ]:
# Jupyter Notebook Configurations

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
# Increase the data rate limit
%config NotebookApp.iopub_data_rate_limit=10000000.0  # 10 MB/sec

In [ ]:
# Python Interpreter Warnings
# DEBUG: Comment out these lines

import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Matplotlib Plot Configurations

# %matplotlib inline

plt.rcParams["figure.figsize"] = (20,10)

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import warnings
warnings.filterwarnings('ignore')

# Benchmark Prompts

### Sentiment Rubrics

In [ ]:
# NOTE: Unnecessary if using custom Ollama models with sentiment SYSTEM message

SENTIMENT_RUBRIC = """
Evaluate this sentence for sentiment polarity
as perceived in the language it was written in then
return a floating point value anywhere betweeen -1.0 (most negative) to 0.0 (neutral) to 1.0 (most positive).
Only return a floating point number between -1.0 and 1.0 and nothing else
Do not respond with an introduction, description, definition, summary, or anything but a single floating point number
""";

## Translation Text (pick one)

### (a) Opening Paragraph

In [ ]:
# SAMPLE TRANSLATIONS:

french_original_str = """
Longtemps, je me suis couché de bonne heure. Parfois, à peine ma
bougie éteinte, mes yeux se fermaient si vite que je n’avais pas le
temps de me dire: «Je m’endors.» Et, une demi-heure après, la pensée
qu’il était temps de chercher le sommeil m’éveillait; je voulais poser
le volume que je croyais avoir encore dans les mains et souffler ma
lumière; je n’avais pas cessé en dormant de faire des réflexions sur
ce que je venais de lire, mais ces réflexions avaient pris un tour un
peu particulier; il me semblait que j’étais moi-même ce dont parlait
l’ouvrage: une église, un quatuor, la rivalité de François Ier et de
Charles Quint. Cette croyance survivait pendant quelques secondes à
mon réveil; elle ne choquait pas ma raison mais pesait comme des
écailles sur mes yeux et les empêchait de se rendre compte que le
bougeoir n’était plus allumé. Puis elle commençait à me devenir
inintelligible, comme après la métempsycose les pensées d’une
existence antérieure; le sujet du livre se détachait de moi, j’étais
libre de m’y appliquer ou non; aussitôt je recouvrais la vue et
j’étais bien étonné de trouver autour de moi une obscurité, douce et
reposante pour mes yeux, mais peut-être plus encore pour mon esprit, à
qui elle apparaissait comme une chose sans cause, incompréhensible,
comme une chose vraiment obscure. Je me demandais quelle heure il
pouvait être; j’entendais le sifflement des trains qui, plus ou moins
éloigné, comme le chant d’un oiseau dans une forêt, relevant les
distances, me décrivait l’étendue de la campagne déserte où le
voyageur se hâte vers la station prochaine; et le petit chemin qu’il
suit va être gravé dans son souvenir par l’excitation qu’il doit à des
lieux nouveaux, à des actes inaccoutumés, à la causerie récente et aux
adieux sous la lampe étrangère qui le suivent encore dans le silence
de la nuit, à la douceur prochaine du retour.
""";

english_translation_davis_str = """
For a long time, I went to bed early.
Sometimes, my candle scarcely out, my eyes would close so quickly that I did not have time to say to myself: ‘I’m falling asleep.’ And, half an hour later, the thought that it was time to try to sleep would wake me; I wanted to put down the book I thought I still had in my hands and blow out my light; I had not ceased while sleeping to form reflections on what I had just read, but these reflections had taken a rather peculiar turn; it seemed to me that I myself was what the book was talking about: a church, a quartet, the rivalry between François I and Charles V.
This belief lived on for a few seconds after my waking; it did not shock my reason but lay heavy like scales on my eyes and kept them from realizing that the candlestick was no longer lit.
Then it began to grow unintelligible to me, as after metempsychosis do the thoughts of an earlier existence; the subject of the book detached itself from me, I was free to apply myself to it or not; immediately I recovered my sight and I was amazed to find a darkness around me soft and restful for my eyes, but perhaps even more so for my mind, to which it appeared a thing without cause, incomprehensible, a thing truly dark.
I would ask myself what time it might be; I could hear the whistling of the trains which, remote or near by, like the singing of a bird in a forest, plotting the distances, described to me the extent of the deserted countryside where the traveller hastens towards the nearest station; and the little road he is following will be engraved on his memory by the excitement he owes to new places, to unaccustomed activities, to the recent conversation and the farewells under the unfamiliar lamp that follow him still through the silence of the night, to the imminent sweetness of his return.
""";

english_translation_enright_str = """
For a long time I would go to bed early.
Sometimes, the candle barely out, my eyes closed so quickly that I did not have time to tell myself: “I’m falling asleep.”
And half an hour later the thought that it was time to look for sleep would awaken me; I would make as if to put away the book which I imagined was still in my hands, and to blow out the light; I had gone on thinking, while I was asleep, about what I had just been reading, but these thoughts had taken a rather peculiar turn; it seemed to me that I myself was the immediate subject of my book: a church, a quartet, the rivalry between François I and Charles V.
This impression would persist for some moments after I awoke; it did not offend my reason, but lay like scales upon my eyes and prevented them from registering the fact that the candle was no longer burning.
Then it would begin to seem unintelligible, as the thoughts of a previous existence must be after reincarnation; the subject of my book would separate itself from me, leaving me free to apply myself to it or not; and at the same time my sight would return and I would be astonished to find myself in a state of darkness, pleasant and restful enough for my eyes, but even more, perhaps, for my mind, to which it appeared incomprehensible, without a cause, something dark indeed.
I would ask myself what time it could be; I could hear the whistling of trains, which, now nearer and now further 1 off, punctuating the distance like the note of a bird in a forest, showed me in perspective the deserted countryside through which a traveller is hurrying towards the nearby station; and the path he is taking will be engraved in his memory by the excitement induced by strange surroundings, by unaccustomed activities, by the conversation he has had and the farewells exchanged beneath an unfamiliar lamp that still echo in his ears amid the silence of the night, and by the happy prospect of being home again.
""";

english_translation_moncrieff_str = """
For a long time I used to go to bed early.
Sometimes, when I had put out my candle, my eyes would close so quickly that I had not even time to say "I'm going to sleep." And half an hour later the thought that it was time to go to sleep would awaken me; I would try to put away the book which, I imagined, was still in my hands, and to blow out the light; I had been thinking all the time, while I was asleep, of what I had just been reading, but my thoughts had run into a channel of their own, until I myself seemed actually to have become the subject of my book: a church, a quartet, the rivalry between François I and Charles V.
This impression would persist for some moments after I was awake; it did not disturb my mind, but it lay like scales upon my eyes and prevented them from registering the fact that the candle was no longer burning.
Then it would begin to seem unintelligible, as the thoughts of a former existence must be to a reincarnate spirit; the subject of my book would separate itself from me, leaving me free to choose whether I would form part of it or no; and at the same time my sight would return and I would be astonished to find myself in a state of darkness, pleasant and restful enough for the eyes, and even more, perhaps, for my mind, to which it appeared incomprehensible, without a cause, a matter dark I would ask myself what o'clock it could be; I could hear the whistling of trains, which, now nearer and now farther off, punctuating the distance like the note of a bird in a forest, shewed me in perspective the deserted countryside through which a traveller would be hurrying towards the nearest station: the path that he followed being fixed for ever in his memory by the general excitement due to being in a strange place, to doing unusual things, to the last words of conversation, to farewells exchanged beneath an unfamiliar lamp which echoed still in his ears amid the silence of the night; and to the delightful prospect of I would lay my cheeks gently against the comfortable cheeks of my pillow, as plump and blooming as the cheeks of babyhood.
""";


### (b) Ending Paragraph

In [ ]:
# SAMPLE TRANSLATIONS:

french_original_str = """
Quelle horreur! Ma consolation c’est de penser aux femmes que j’ai connues, aujourd’hui qu’il n’y a plus d’élégance.
Mais comment des gens qui contemplent ces horribles créatures sous leurs chapeaux couverts d’une volière ou d’un potager, pourraient-ils même sentir ce qu’il y avait de charmant à voir Mme Swann coiffée d’une simple capote mauve ou d’un petit chapeau que dépassait une seule fleur d’iris toute droite.
Aurais-je même pu leur faire comprendre l’émotion que j’éprouvais par les matins d’hiver à rencontrer Mme Swann à pied, en paletot de loutre, coiffée d’un simple béret que dépassaient deux couteaux de plumes de perdrix, mais autour de laquelle la tiédeur factice de son appartement était évoquée, rien que par le bouquet de violettes qui s’écrasait à son corsage et dont le fleurissement vivant et bleu en face du ciel gris, de l’air glacé, des arbres aux branches nues, avait le même charme de ne prendre la saison et le temps que comme un cadre, et de vivre dans une atmosphère humaine, dans l’atmosphère de cette femme, qu’avaient dans les vases et les jardinières de son salon, près du feu allumé, devant le canapé de soie, les fleurs qui regardaient par la fenêtre close la neige tomber?
D’ailleurs il ne m’eût pas suffi que les toilettes fussent les mêmes qu’en ces années-là.
A cause de la solidarité qu’ont entre elles les différentes parties d’un souvenir et que notre mémoire maintient équilibrées dans un assemblage où il ne nous est pas permis de rien distraire, ni refuser, j’aurais voulu pouvoir aller finir la journée chez une de ces femmes, devant une tasse de thé, dans un appartement aux murs peints de couleurs sombres, comme était encore celui de Mme Swann (l’année d’après celle où se termine la première partie de ce récit) et où luiraient les feux orangés, la rouge combustion, la flamme rose et blanche des chrysanthèmes dans le crépuscule de novembre pendant des instants pareils à ceux où (comme on le verra plus tard) je n’avais pas su découvrir les plaisirs que je désirais.
Mais maintenant, même ne me conduisant à rien, ces instants me semblaient avoir eu eux-mêmes assez de charme.
Je voudrais les retrouver tels que je me les rappelais.
Hélas! il n’y avait plus que des appartements Louis XVI tout blancs, émaillés d’hortensias bleus.
D’ailleurs, on ne revenait plus à Paris que très tard.
Mme Swann m’eût répondu d’un château qu’elle ne rentrerait qu’en février, bien après le temps des chrysanthèmes, si je lui avais demandé de reconstituer pour moi les éléments de ce souvenir que je sentais attaché à une année lointaine, à un millésime vers lequel il ne m’était pas permis de remonter, les éléments de ce désir devenu lui-même inaccessible comme le plaisir qu’il avait jadis vainement poursuivi.
Et il m’eût fallu aussi que ce fussent les mêmes femmes, celles dont la toilette m’intéressait parce que, au temps où je croyais encore, mon imagination les avait individualisées et les avait pourvues d’une légende.
Hélas! dans l’avenue des Acacias--l’allée de Myrtes--j’en revis quelques-unes, vieilles, et qui n’étaient plus que les ombres terribles de ce qu’elles avaient été, errant, cherchant désespérément on ne sait quoi dans les bosquets virgiliens.
Elles avaient fui depuis longtemps que j’étais encore à interroger vainement les chemins désertés.
Le soleil s’était caché.
La nature recommençait à régner sur le Bois d’où s’était envolée l’idée qu’il était le Jardin élyséen de la Femme; au-dessus du moulin factice le vrai ciel était gris; le vent ridait le Grand Lac de petites vaguelettes, comme un lac; de gros oiseaux parcouraient rapidement le Bois, comme un bois, et poussant des cris aigus se posaient l’un après l’autre sur les grands chênes qui sous leur couronne druidique et avec une majesté dodonéenne semblaient proclamer le vide inhumain de la forêt désaffectée, et m’aidaient à mieux comprendre la contradiction que c’est de chercher dans la réalité les tableaux de la mémoire, auxquels manquerait toujours le charme qui leur vient de la mémoire même et de n’être pas perçus par les sens.
La réalité que j’avais connue n’existait plus.
Il suffisait que Mme Swann n’arrivât pas toute pareille au même moment, pour que l’Avenue fût autre.
Les lieux que nous avons connus n’appartiennent pas qu’au monde de l’espace où nous les situons pour plus de facilité.
Ils n’étaient qu’une mince tranche au milieu d’impressions contiguës qui formaient notre vie d’alors; le souvenir d’une certaine image n’est que le regret d’un certain instant; et les maisons, les routes, les avenues, sont fugitives, hélas, comme les années.
""";

english_translation_davis_str = """
How awful! I said to myself: can anyone think these automobiles are as elegant as the old carriages and pairs? I’m probably too old already – but I’m not meant for a world in which women hobble themselves in dresses that aren’t even made of cloth. What’s the use of walking among these trees, if nothing is left of what used to gather under the delicate reddening leaves, if vulgarity and idiocy have taken the place of the exquisite thing they once framed? How awful! My consolation is to think about the women I have known, now that there is no more elegance. But how could anyone contemplating these horrible creatures under their hats topped with a birdcage or a vegetable patch even sense what was so charming about the sight of Mme Swann in a simple mauve hood or a little hat with a single stiff, straight iris poking up from it? Could I even have made them understand the emotion I felt on winter mornings when I met Mme Swann on foot, in a sealskin coat, wearing a simple beret with two blades of partridge feathers sticking up from it, but enveloped also by the artificial warmth of her apartment, which was conjured by nothing more than the bouquet of violets crushed at her breast whose live blue flowering against the grey sky, the icy air, the bare-branched trees, had the same charming manner of accepting the season and the weather merely as a setting, and of living in a human atmosphere, in the atmosphere of this woman, as had, in the vases and flower-stands of her drawing-room, close to the lit fire, before the silk sofa, the flowers that looked out through the closed window at the falling snow? But it would not have been enough for me anyway for the clothes to be the same as in those earlier times. Because of the dependence which the different parts of a recollection have on one another, parts which our memory keeps balanced in an aggregate from which we are not permitted to abstract anything, or reject anything, I would have wanted to be able to go and spend the last part of the day in the home of one of these women, over a cup of tea, in an apartment with walls painted in dark colours, as Mme Swann’s still was (in the year after the one in which the first part of this story ends) and in which the orange flares, the red combustion, the pink and white flame of the chrysanthemums would gleam in the November twilight, during moments like those in which (as we will see later) I was not able to discover the pleasures I desired. But now, even though they had led to nothing, those moments seemed to me to have had enough charm in themselves. I wanted to find them again as I remembered them. Alas, there was no longer anything but Louis XVI apartments all white and dotted with blue hydrangeas. Moreover, people no longer returned to Paris until very late. Mme Swann would have answered me from a country house that she would not be back until February, well after the time of the chrysanthemums, had I asked her to reconstruct for me the elements of that memory which I felt belonged to a distant year, to a vintage to which I was not allowed to go back, the elements of that desire which had itself become as inaccessible as the pleasure it had once vainly pursued. And I would also have needed them to be the same women, those whose clothing interested me because, at the time when I still believed, my imagination had individualized them and given them each a legend. Alas, in the avenue des Acacias – the allée de Myrtes – I did see a few of them again, old, now no more than terrible shadows of what they had been, wandering, desperately searching for who knows what in the Virgilian groves. They had fled long since as I still vainly questioned the deserted paths. The sun had hidden itself. Nature was resuming its rule over the Bois, from which the idea that it was the Elysian Garden of Woman had vanished; above the artificial mill the real sky was grey; the wind wrinkled the Grand Lac with little wavelets, like a real lake; large birds swiftly crossed the Bois, like a real wood, and uttering sharp cries alighted one after another in the tall oaks which under their druidical crowns and with a Dodonean39 majesty seemed to proclaim the inhuman emptiness of the disused forest, and helped me better understand what a contradiction it is to search in reality for memory’s pictures, which would never have the charm that comes to them from memory itself and from not being perceived by the senses. The reality I had known no longer existed. That Mme Swann did not arrive exactly the same at the same moment was enough to make the avenue different. The places we have known do not belong solely to the world of space in which we situate them for our greater convenience. They were only a thin slice among contiguous impressions that formed our life at that time; the memory of a certain image is only regret for a certain moment; and houses, roads, avenues are as fleeting, alas, as the years.
""";

english_translation_enright_str = """
How horrible! I exclaimed to myself. Can anyone find these motor-cars as elegant as the old carriage-and-pair? I dare say I am too old now—but I was not intended for a world in which women shackle themselves in garments that are not even made of cloth. To what purpose shall I walk among these trees if there is nothing left now of the assembly that used to gather beneath this delicate tracery of reddening leaves, if vulgarity and folly have supplanted the exquisite thing that their branches once framed. How horrible! My consolation is to think of the women whom I knew in the past, now that there is no elegance left. But how could the people who watch these dreadful creatures hobble by beneath hats on which have been heaped the spoils of aviary or kitchen-garden, how could they even imagine the charm that there was in the sight of Mme Swann in a simple mauve bonnet or a little hat with a single iris sticking up out of it?
Could I even have made them understand the emotion that I used to feel on winter mornings, when I met Mme Swann on foot, in an otter-skin coat, with a woolen cap from which stuck out two blade-like partridge-feathers, but enveloped also in the artificial warmth of her own house, which was suggested by nothing more than the bunch of violets crushed into her bosom, whose flowering, vivid and blue against the
grey sky, the freezing air, the naked boughs, had the same charming effect of using the season and the weather merely as a setting, and of living actually in a human atmosphere, in the atmosphere of this woman, as had, in the vases and jardinières of her drawing-room, beside the blazing fire, in front of the silk-covered settee, the flowers that looked out through closed windows at the falling snow? But it would not have sufficed me that the costumes alone should still have been the same as those in distant years. Because of the solidarity that binds together the different parts of a general impression that our memory keeps in a balanced whole of which we are not permitted to subtract or to decline any fraction, I should have liked to be able to pass the rest of the day with one of those women, over a cup of tea, in an apartment with dark-painted walls (as Mme Swann’s were still in the year after that in which the first part of this story ends) against which would glow the orange flame, the red combustion, the pink and white flickering of her chrysanthemums in the twilight of a November evening, in moments similar to those in which (as we shall see) I had not managed to discover the pleasures for which I longed. But now, even though they had led to nothing, those moments struck me as having been charming enough in themselves. I wanted to find them again as I remembered them. Alas! there was nothing now but flats decorated in the Louis XVI style, all white, with a sprinkling of blue hydrangeas. Moreover, people did not return to Paris, now, until much later. Mme Swann would have written to me from a country house to say that she would not be in town before February, long after the chrysanthemum season, had I asked her to reconstruct for me the elements of that memory which I felt to belong to a particular distant year, a particular vintage towards which it was forbidden me to ascend again the fatal slope, the
elements of that longing which had itself become as inaccessible as the pleasure that it had once vainly pursued.
And I should have required also that they should be the same women, those whose costume interested me because, at the time when I still had faith, my imagination had individualised them and had provided each of them with a legend. Alas! in the acacia-avenue—the myrtle-alley—I did see some of them again, grown old, no more now than grim spectres of what they had once been, wandering, desperately searching for heaven knew what, through the Virgilian groves. They had long since fled, and still I stood vainly questioning the deserted paths. The sun had gone. Nature was resuming its reign over the Bois, from which had vanished all trace of the idea that it was the Elysian Garden of Woman; above the gimcrack windmill the real sky was grey; the wind wrinkled the surface of the Grand Lac in little wavelets, like a real lake; large birds flew swiftly over the Bois, as over a real wood, and with shrill cries perched, one after another, on the great oaks which, beneath their Druidical crown, and with Dodonian majesty, seemed to proclaim the inhuman emptiness of this deconsecrated forest, and helped me to understand how paradoxical it is to seek in reality for the pictures that are stored in one’s memory, which must inevitably lose the charm that comes to them from memory itself and from their not being apprehended by the senses. The reality that I had known no longer existed. It sufficed that Mme Swann did not appear, in the same attire and at the same moment, for the whole avenue to be altered.
The places we have known do not belong only to the world of space on which we map them for our own convenience. They were only a thin slice, held between the contiguous impressions that composed our life at that time; the memory
of a particular image is but regret for a particular moment; and houses, roads, avenues are as fugitive, alas, as the years.
""";

english_translation_moncrieff_str = """
"Oh, horrible!" I exclaimed to myself: "Does anyone really imagine that
these motor-cars are as smart as the old carriage-and-pair? I dare say.
I am too old now--but I was not intended for a world in which women
shackle themselves in garments that are not even made of cloth. To what
purpose shall I walk among these trees if there is nothing left now of
the assembly that used to meet beneath the delicate tracery of reddening
leaves, if vulgarity and fatuity have supplanted the exquisite thing
that once their branches framed? Oh, horrible! My consolation is to
think of the women whom I have known, in the past, now that there is
no standard left of elegance. But how can the people who watch these
dreadful creatures hobble by, beneath hats on which have been heaped
the spoils of aviary or garden-bed,--how can they imagine the charm that
there was in the sight of Mme. Swann, crowned with a close-fitting lilac
bonnet, or with a tiny hat from which rose stiffly above her head a
single iris?" Could I ever have made them understand the emotion that
I used to feel on winter mornings, when I met Mme. Swann on foot, in an
otter-skin coat, with a woollen cap from which stuck out two blade-like
partridge-feathers, but enveloped also in the deliberate, artificial
warmth of her own house, which was suggested by nothing more than the
bunch of violets crushed into her bosom, whose flowering, vivid and blue
against the grey sky, the freezing air, the naked boughs, had the same
charming effect of using the season and the weather merely as a setting,
and of living actually in a human atmosphere, in the atmosphere of this
woman, as had in the vases and beaupots of her drawing-room, beside the
blazing fire, in front of the silk-covered sofa, the flowers that looked
out through closed windows at the falling snow? But it would not have
sufficed me that the costumes alone should still have been the same as
in those distant years. Because of the solidarity that binds together
the different parts of a general impression, parts that our memory keeps
in a balanced whole, of which we are not permitted to subtract or to
decline any fraction, I should have liked to be able to pass the rest
of the day with one of those women, over a cup of tea, in a little house
with dark-painted walls (as Mme. Swann's were still in the year after
that in which the first part of this story ends) against which would
glow the orange flame, the red combustion, the pink and white flickering
of her chrysanthemums in the twilight of a November evening, in moments
similar to those in which (as we shall see) I had not managed to
discover the pleasures for which I longed. But now, albeit they had led
to nothing, those moments struck me as having been charming enough in
themselves. I sought to find them again as I remembered them. Alas!
there was nothing now but flats decorated in the Louis XVI style, all
white paint, with hortensias in blue enamel. Moreover, people did not
return to Paris, now, until much later. Mme. Swann would have written to
me, from a country house, that she would not be in town before February,
had I asked her to reconstruct for me the elements of that memory which
I felt to belong to a distant era, to a date in time towards which it
was forbidden me to ascend again the fatal slope, the elements of that
longing which had become, itself, as inaccessible as the pleasure that
it had once vainly pursued. And I should have required also that they
be the same women, those whose costume interested me because, at a time
when I still had faith, my imagination had individualised them and had
provided each of them with a legend. Alas! in the acacia-avenue--the
myrtle-alley--I did see some of them again, grown old, no more now
than grim spectres of what once they had been, wandering to and fro, in
desperate search of heaven knew what, through the Virgilian groves. They
had long fled, and still I stood vainly questioning the deserted paths.
The sun's face was hidden. Nature began again to reign over the Bois,
from which had vanished all trace of the idea that it was the Elysian
Garden of Woman; above the gimcrack windmill the real sky was grey; the
wind wrinkled the surface of the Grand Lac in little wavelets, like
a real lake; large birds passed swiftly over the Bois, as over a real
wood, and with shrill cries perched, one after another, on the great
oaks which, beneath their Druidical crown, and with Dodonaic majesty,
seemed to proclaim the unpeopled vacancy of this estranged forest, and
helped me to understand how paradoxical it is to seek in reality for the
pictures that are stored in one's memory, which must inevitably lose
the charm that comes to them from memory itself and from their not
being apprehended by the senses. The reality that I had known no longer
existed. It sufficed that Mme. Swann did not appear, in the same attire
and at the same moment, for the whole avenue to be altered. The places
that we have known belong now only to the little world of space on which
we map them for our own convenience. None of them was ever more than a
thin slice, held between the contiguous impressions that composed our
life at that time; remembrance of a particular form is but regret for a
particular moment; and houses, roads, avenues are as fugitive, alas, as
the years.
""";


### (c) Any Paragraph

In [ ]:
# SAMPLE TRANSLATIONS:

french_original_str = """
Quelle horreur! Ma consolation c’est de penser aux femmes que j’ai connues, aujourd’hui qu’il n’y a plus d’élégance.
Mais comment des gens qui contemplent ces horribles créatures sous leurs chapeaux couverts d’une volière ou d’un potager, pourraient-ils même sentir ce qu’il y avait de charmant à voir Mme Swann coiffée d’une simple capote mauve ou d’un petit chapeau que dépassait une seule fleur d’iris toute droite.
Aurais-je même pu leur faire comprendre l’émotion que j’éprouvais par les matins d’hiver à rencontrer Mme Swann à pied, en paletot de loutre, coiffée d’un simple béret que dépassaient deux couteaux de plumes de perdrix, mais autour de laquelle la tiédeur factice de son appartement était évoquée, rien que par le bouquet de violettes qui s’écrasait à son corsage et dont le fleurissement vivant et bleu en face du ciel gris, de l’air glacé, des arbres aux branches nues, avait le même charme de ne prendre la saison et le temps que comme un cadre, et de vivre dans une atmosphère humaine, dans l’atmosphère de cette femme, qu’avaient dans les vases et les jardinières de son salon, près du feu allumé, devant le canapé de soie, les fleurs qui regardaient par la fenêtre close la neige tomber?
D’ailleurs il ne m’eût pas suffi que les toilettes fussent les mêmes qu’en ces années-là.
A cause de la solidarité qu’ont entre elles les différentes parties d’un souvenir et que notre mémoire maintient équilibrées dans un assemblage où il ne nous est pas permis de rien distraire, ni refuser, j’aurais voulu pouvoir aller finir la journée chez une de ces femmes, devant une tasse de thé, dans un appartement aux murs peints de couleurs sombres, comme était encore celui de Mme Swann (l’année d’après celle où se termine la première partie de ce récit) et où luiraient les feux orangés, la rouge combustion, la flamme rose et blanche des chrysanthèmes dans le crépuscule de novembre pendant des instants pareils à ceux où (comme on le verra plus tard) je n’avais pas su découvrir les plaisirs que je désirais.
Mais maintenant, même ne me conduisant à rien, ces instants me semblaient avoir eu eux-mêmes assez de charme.
Je voudrais les retrouver tels que je me les rappelais.
Hélas! il n’y avait plus que des appartements Louis XVI tout blancs, émaillés d’hortensias bleus.
D’ailleurs, on ne revenait plus à Paris que très tard.
Mme Swann m’eût répondu d’un château qu’elle ne rentrerait qu’en février, bien après le temps des chrysanthèmes, si je lui avais demandé de reconstituer pour moi les éléments de ce souvenir que je sentais attaché à une année lointaine, à un millésime vers lequel il ne m’était pas permis de remonter, les éléments de ce désir devenu lui-même inaccessible comme le plaisir qu’il avait jadis vainement poursuivi.
Et il m’eût fallu aussi que ce fussent les mêmes femmes, celles dont la toilette m’intéressait parce que, au temps où je croyais encore, mon imagination les avait individualisées et les avait pourvues d’une légende.
Hélas! dans l’avenue des Acacias--l’allée de Myrtes--j’en revis quelques-unes, vieilles, et qui n’étaient plus que les ombres terribles de ce qu’elles avaient été, errant, cherchant désespérément on ne sait quoi dans les bosquets virgiliens.
Elles avaient fui depuis longtemps que j’étais encore à interroger vainement les chemins désertés.
Le soleil s’était caché.
La nature recommençait à régner sur le Bois d’où s’était envolée l’idée qu’il était le Jardin élyséen de la Femme; au-dessus du moulin factice le vrai ciel était gris; le vent ridait le Grand Lac de petites vaguelettes, comme un lac; de gros oiseaux parcouraient rapidement le Bois, comme un bois, et poussant des cris aigus se posaient l’un après l’autre sur les grands chênes qui sous leur couronne druidique et avec une majesté dodonéenne semblaient proclamer le vide inhumain de la forêt désaffectée, et m’aidaient à mieux comprendre la contradiction que c’est de chercher dans la réalité les tableaux de la mémoire, auxquels manquerait toujours le charme qui leur vient de la mémoire même et de n’être pas perçus par les sens.
La réalité que j’avais connue n’existait plus.
Il suffisait que Mme Swann n’arrivât pas toute pareille au même moment, pour que l’Avenue fût autre.
Les lieux que nous avons connus n’appartiennent pas qu’au monde de l’espace où nous les situons pour plus de facilité.
Ils n’étaient qu’une mince tranche au milieu d’impressions contiguës qui formaient notre vie d’alors; le souvenir d’une certaine image n’est que le regret d’un certain instant; et les maisons, les routes, les avenues, sont fugitives, hélas, comme les années.
""";

english_translation_davis_str = """
How awful! I said to myself: can anyone think these automobiles are as elegant as the old carriages and pairs? I’m probably too old already – but I’m not meant for a world in which women hobble themselves in dresses that aren’t even made of cloth. What’s the use of walking among these trees, if nothing is left of what used to gather under the delicate reddening leaves, if vulgarity and idiocy have taken the place of the exquisite thing they once framed? How awful! My consolation is to think about the women I have known, now that there is no more elegance. But how could anyone contemplating these horrible creatures under their hats topped with a birdcage or a vegetable patch even sense what was so charming about the sight of Mme Swann in a simple mauve hood or a little hat with a single stiff, straight iris poking up from it? Could I even have made them understand the emotion I felt on winter mornings when I met Mme Swann on foot, in a sealskin coat, wearing a simple beret with two blades of partridge feathers sticking up from it, but enveloped also by the artificial warmth of her apartment, which was conjured by nothing more than the bouquet of violets crushed at her breast whose live blue flowering against the grey sky, the icy air, the bare-branched trees, had the same charming manner of accepting the season and the weather merely as a setting, and of living in a human atmosphere, in the atmosphere of this woman, as had, in the vases and flower-stands of her drawing-room, close to the lit fire, before the silk sofa, the flowers that looked out through the closed window at the falling snow? But it would not have been enough for me anyway for the clothes to be the same as in those earlier times. Because of the dependence which the different parts of a recollection have on one another, parts which our memory keeps balanced in an aggregate from which we are not permitted to abstract anything, or reject anything, I would have wanted to be able to go and spend the last part of the day in the home of one of these women, over a cup of tea, in an apartment with walls painted in dark colours, as Mme Swann’s still was (in the year after the one in which the first part of this story ends) and in which the orange flares, the red combustion, the pink and white flame of the chrysanthemums would gleam in the November twilight, during moments like those in which (as we will see later) I was not able to discover the pleasures I desired. But now, even though they had led to nothing, those moments seemed to me to have had enough charm in themselves. I wanted to find them again as I remembered them. Alas, there was no longer anything but Louis XVI apartments all white and dotted with blue hydrangeas. Moreover, people no longer returned to Paris until very late. Mme Swann would have answered me from a country house that she would not be back until February, well after the time of the chrysanthemums, had I asked her to reconstruct for me the elements of that memory which I felt belonged to a distant year, to a vintage to which I was not allowed to go back, the elements of that desire which had itself become as inaccessible as the pleasure it had once vainly pursued. And I would also have needed them to be the same women, those whose clothing interested me because, at the time when I still believed, my imagination had individualized them and given them each a legend. Alas, in the avenue des Acacias – the allée de Myrtes – I did see a few of them again, old, now no more than terrible shadows of what they had been, wandering, desperately searching for who knows what in the Virgilian groves. They had fled long since as I still vainly questioned the deserted paths. The sun had hidden itself. Nature was resuming its rule over the Bois, from which the idea that it was the Elysian Garden of Woman had vanished; above the artificial mill the real sky was grey; the wind wrinkled the Grand Lac with little wavelets, like a real lake; large birds swiftly crossed the Bois, like a real wood, and uttering sharp cries alighted one after another in the tall oaks which under their druidical crowns and with a Dodonean39 majesty seemed to proclaim the inhuman emptiness of the disused forest, and helped me better understand what a contradiction it is to search in reality for memory’s pictures, which would never have the charm that comes to them from memory itself and from not being perceived by the senses. The reality I had known no longer existed. That Mme Swann did not arrive exactly the same at the same moment was enough to make the avenue different. The places we have known do not belong solely to the world of space in which we situate them for our greater convenience. They were only a thin slice among contiguous impressions that formed our life at that time; the memory of a certain image is only regret for a certain moment; and houses, roads, avenues are as fleeting, alas, as the years.
""";

english_translation_enright_str = """
How horrible! I exclaimed to myself. Can anyone find these motor-cars as elegant as the old carriage-and-pair? I dare say I am too old now—but I was not intended for a world in which women shackle themselves in garments that are not even made of cloth. To what purpose shall I walk among these trees if there is nothing left now of the assembly that used to gather beneath this delicate tracery of reddening leaves, if vulgarity and folly have supplanted the exquisite thing that their branches once framed. How horrible! My consolation is to think of the women whom I knew in the past, now that there is no elegance left. But how could the people who watch these dreadful creatures hobble by beneath hats on which have been heaped the spoils of aviary or kitchen-garden, how could they even imagine the charm that there was in the sight of Mme Swann in a simple mauve bonnet or a little hat with a single iris sticking up out of it?
Could I even have made them understand the emotion that I used to feel on winter mornings, when I met Mme Swann on foot, in an otter-skin coat, with a woolen cap from which stuck out two blade-like partridge-feathers, but enveloped also in the artificial warmth of her own house, which was suggested by nothing more than the bunch of violets crushed into her bosom, whose flowering, vivid and blue against the
grey sky, the freezing air, the naked boughs, had the same charming effect of using the season and the weather merely as a setting, and of living actually in a human atmosphere, in the atmosphere of this woman, as had, in the vases and jardinières of her drawing-room, beside the blazing fire, in front of the silk-covered settee, the flowers that looked out through closed windows at the falling snow? But it would not have sufficed me that the costumes alone should still have been the same as those in distant years. Because of the solidarity that binds together the different parts of a general impression that our memory keeps in a balanced whole of which we are not permitted to subtract or to decline any fraction, I should have liked to be able to pass the rest of the day with one of those women, over a cup of tea, in an apartment with dark-painted walls (as Mme Swann’s were still in the year after that in which the first part of this story ends) against which would glow the orange flame, the red combustion, the pink and white flickering of her chrysanthemums in the twilight of a November evening, in moments similar to those in which (as we shall see) I had not managed to discover the pleasures for which I longed. But now, even though they had led to nothing, those moments struck me as having been charming enough in themselves. I wanted to find them again as I remembered them. Alas! there was nothing now but flats decorated in the Louis XVI style, all white, with a sprinkling of blue hydrangeas. Moreover, people did not return to Paris, now, until much later. Mme Swann would have written to me from a country house to say that she would not be in town before February, long after the chrysanthemum season, had I asked her to reconstruct for me the elements of that memory which I felt to belong to a particular distant year, a particular vintage towards which it was forbidden me to ascend again the fatal slope, the
elements of that longing which had itself become as inaccessible as the pleasure that it had once vainly pursued.
And I should have required also that they should be the same women, those whose costume interested me because, at the time when I still had faith, my imagination had individualised them and had provided each of them with a legend. Alas! in the acacia-avenue—the myrtle-alley—I did see some of them again, grown old, no more now than grim spectres of what they had once been, wandering, desperately searching for heaven knew what, through the Virgilian groves. They had long since fled, and still I stood vainly questioning the deserted paths. The sun had gone. Nature was resuming its reign over the Bois, from which had vanished all trace of the idea that it was the Elysian Garden of Woman; above the gimcrack windmill the real sky was grey; the wind wrinkled the surface of the Grand Lac in little wavelets, like a real lake; large birds flew swiftly over the Bois, as over a real wood, and with shrill cries perched, one after another, on the great oaks which, beneath their Druidical crown, and with Dodonian majesty, seemed to proclaim the inhuman emptiness of this deconsecrated forest, and helped me to understand how paradoxical it is to seek in reality for the pictures that are stored in one’s memory, which must inevitably lose the charm that comes to them from memory itself and from their not being apprehended by the senses. The reality that I had known no longer existed. It sufficed that Mme Swann did not appear, in the same attire and at the same moment, for the whole avenue to be altered.
The places we have known do not belong only to the world of space on which we map them for our own convenience. They were only a thin slice, held between the contiguous impressions that composed our life at that time; the memory
of a particular image is but regret for a particular moment; and houses, roads, avenues are as fugitive, alas, as the years.
""";

english_translation_moncrieff_str = """
"Oh, horrible!" I exclaimed to myself: "Does anyone really imagine that
these motor-cars are as smart as the old carriage-and-pair? I dare say.
I am too old now--but I was not intended for a world in which women
shackle themselves in garments that are not even made of cloth. To what
purpose shall I walk among these trees if there is nothing left now of
the assembly that used to meet beneath the delicate tracery of reddening
leaves, if vulgarity and fatuity have supplanted the exquisite thing
that once their branches framed? Oh, horrible! My consolation is to
think of the women whom I have known, in the past, now that there is
no standard left of elegance. But how can the people who watch these
dreadful creatures hobble by, beneath hats on which have been heaped
the spoils of aviary or garden-bed,--how can they imagine the charm that
there was in the sight of Mme. Swann, crowned with a close-fitting lilac
bonnet, or with a tiny hat from which rose stiffly above her head a
single iris?" Could I ever have made them understand the emotion that
I used to feel on winter mornings, when I met Mme. Swann on foot, in an
otter-skin coat, with a woollen cap from which stuck out two blade-like
partridge-feathers, but enveloped also in the deliberate, artificial
warmth of her own house, which was suggested by nothing more than the
bunch of violets crushed into her bosom, whose flowering, vivid and blue
against the grey sky, the freezing air, the naked boughs, had the same
charming effect of using the season and the weather merely as a setting,
and of living actually in a human atmosphere, in the atmosphere of this
woman, as had in the vases and beaupots of her drawing-room, beside the
blazing fire, in front of the silk-covered sofa, the flowers that looked
out through closed windows at the falling snow? But it would not have
sufficed me that the costumes alone should still have been the same as
in those distant years. Because of the solidarity that binds together
the different parts of a general impression, parts that our memory keeps
in a balanced whole, of which we are not permitted to subtract or to
decline any fraction, I should have liked to be able to pass the rest
of the day with one of those women, over a cup of tea, in a little house
with dark-painted walls (as Mme. Swann's were still in the year after
that in which the first part of this story ends) against which would
glow the orange flame, the red combustion, the pink and white flickering
of her chrysanthemums in the twilight of a November evening, in moments
similar to those in which (as we shall see) I had not managed to
discover the pleasures for which I longed. But now, albeit they had led
to nothing, those moments struck me as having been charming enough in
themselves. I sought to find them again as I remembered them. Alas!
there was nothing now but flats decorated in the Louis XVI style, all
white paint, with hortensias in blue enamel. Moreover, people did not
return to Paris, now, until much later. Mme. Swann would have written to
me, from a country house, that she would not be in town before February,
had I asked her to reconstruct for me the elements of that memory which
I felt to belong to a distant era, to a date in time towards which it
was forbidden me to ascend again the fatal slope, the elements of that
longing which had become, itself, as inaccessible as the pleasure that
it had once vainly pursued. And I should have required also that they
be the same women, those whose costume interested me because, at a time
when I still had faith, my imagination had individualised them and had
provided each of them with a legend. Alas! in the acacia-avenue--the
myrtle-alley--I did see some of them again, grown old, no more now
than grim spectres of what once they had been, wandering to and fro, in
desperate search of heaven knew what, through the Virgilian groves. They
had long fled, and still I stood vainly questioning the deserted paths.
The sun's face was hidden. Nature began again to reign over the Bois,
from which had vanished all trace of the idea that it was the Elysian
Garden of Woman; above the gimcrack windmill the real sky was grey; the
wind wrinkled the surface of the Grand Lac in little wavelets, like
a real lake; large birds passed swiftly over the Bois, as over a real
wood, and with shrill cries perched, one after another, on the great
oaks which, beneath their Druidical crown, and with Dodonaic majesty,
seemed to proclaim the unpeopled vacancy of this estranged forest, and
helped me to understand how paradoxical it is to seek in reality for the
pictures that are stored in one's memory, which must inevitably lose
the charm that comes to them from memory itself and from their not
being apprehended by the senses. The reality that I had known no longer
existed. It sufficed that Mme. Swann did not appear, in the same attire
and at the same moment, for the whole avenue to be altered. The places
that we have known belong now only to the little world of space on which
we map them for our own convenience. None of them was ever more than a
thin slice, held between the contiguous impressions that composed our
life at that time; remembrance of a particular form is but regret for a
particular moment; and houses, roads, avenues are as fugitive, alas, as
the years.
""";


## Translation Rubrics (pick one)

### (a) Count 4: Merged

In [ ]:
SCORING_RUBRIC_FOUR = """
Evaluate the English translation of the French text using the following criteria, each scored from 0(terrible) to 5 (perfect):
A. Accuracy-Adequacy (40%): Does the translation fully convey the meaning, intent, and information of the source text with minimal or no errors?
B. Fluency-Readability (30%): Is the translation  fluent, with natural phrasing, correct grammar, and excellent readability, closely resembling native language usage?
C. Terminology-Consistency-Style (20%): Consistently uses accurate and appropriate terminology/domain-specific terms, maintaining a consistent style, tone, and register throughout?
D. Cultural-Linguistic-Appropriateness (10%): Does the translation handle cultural element, nuances, and idiomatic expressions effectively, reflecting the target language and culture?
""";

### (b) Count 8: Jon

In [ ]:
SCORING_RUBRIC_EIGHT = """
Evaluate the quality of the ##ENGLISH_TRANSLATION of the ###FRENCH_ORIGINAL text using the following criteria, each scored from 0(terrible) to 5 (perfect):

1. Accuracy-Adequacy: Preserves source language meaning, information, and fidelity to the source text.

2. Fluency: Preserves naturalness, readability, and grammatical correctness of the target language.

3. Terminology: Accurate and consistent use of domain-specific terms.

4. Style-Tone: Adheres to the appropriate tone, register, formality, and alignment with the source text and target audience.

5. Cultural-Appropriateness: Conveys cultural nuances, context-specific meanings, and idiomatic expressions.

6. Consistency: Has coherence and cohesion within the translated text, including pronoun agreement and discourse markers.

7. Punctuation-Format: Correctness of punctuation and adherence to formatting guidelines.

8. Idiomatic: Correct use of idiomatic expressions in the target language, ensuring they are contextually appropriate and sound natural.

"""


### (c) Count 8: Kate

In [ ]:
# KATE 5/28/2024
# Sentiment (0 very negative to 5 very positive)
# Formality: (0 conversational to 5 formal tone)
# Impersonality: (0 very personal to 5 very impersonal)
# Lexical Density: (0 to 5)
# Lexical Diversity: (0 to 5)


SCORING_RUBRIC_EIGHT = """
Evaluate the quality of the ##ENGLISH_TRANSLATION of the ###FRENCH_ORIGINAL text using the following criteria, each scored from 0(terrible) to 5 (perfect):

1. Accuracy-Adequacy: Preserves source language meaning, information, and fidelity to the source text.

2. Fluency: Preserves naturalness, readability, and grammatical correctness of the target language.

3. Terminology: Accurate and consistent use of domain-specific terms.

4. Style-Tone: Adheres to the appropriate tone, register, formality, and alignment with the source text and target audience.

5. Cultural-Appropriateness: Conveys cultural nuances, context-specific meanings, and idiomatic expressions.

6. Consistency: Has coherence and cohesion within the translated text, including pronoun agreement and discourse markers.

7. Punctuation-Format: Correctness of punctuation and adherence to formatting guidelines.

8. Idiomatic: Correct use of idiomatic expressions in the target language, ensuring they are contextually appropriate and sound natural.

""";



# Get Raw Text

### Upload Hand-Cleaned Files

```
 book_proust_en_swans-way_davis_original_verified.txt
 book_proust_en_swans-way_enright_original_verified.txt
 book_proust_en_swans-way_moncrieff_original_verified.txt
 book_proust_fr_swans-way_proust_original_verified.txt
 ```

In [ ]:
# 20240525 Get clean segmented text for individual book translations
# e.g. data/step1_segments/book_proust_en_swans-way_davis/book_proust_en_swans-way_davis_sentence_clean.txt

# Upload combo files:

uploaded_raw_text = files.upload()

In [ ]:
# !ls *_clean.txt
!ls *_verified.txt

In [ ]:
filenames_in_list = [f for f in os.listdir() if f.endswith('_verified.txt')]
print(filenames_in_list)
print(f"\n TOTAL: {len(filenames_in_list)} files")

In [ ]:
# print(uploaded_raw_text)

!head -n 10 book_proust_en_swans-way_davis_original_verified.txt

### Read File into Clean String

In [ ]:
def read_file_to_text(filename_in):
    """
    Reads a text file, handles multiple languages and encodings,
    removes non-printable and illegal characters, and returns a clean string in UTF-8.

    Parameters:
    filename_in (str): The input filename.

    Returns:
    str: The cleaned string with only printable characters in UTF-8 encoding.
    """
    # Read the raw bytes from the file
    with open(filename_in, 'rb') as file:
        raw_data = file.read()

    # Detect the encoding of the file
    detected_encoding = chardet.detect(raw_data)['encoding']

    # Decode the raw data to a string
    decoded_text = raw_data.decode(detected_encoding, errors='ignore')

    # Fix text encoding issues
    fixed_text = ftfy.fix_text(decoded_text)

    # Normalize the text to NFKD (Normalization Form KD)
    normalized_text = unicodedata.normalize('NFKD', fixed_text)

    # Create a set of all printable characters
    printable_chars = set(string.printable)

    # Filter out non-printable characters
    cleaned_text = ''.join(c for c in normalized_text if c in printable_chars)

    # Ensure the cleaned text is in UTF-8 encoding
    cleaned_text_utf8 = cleaned_text.encode('utf-8').decode('utf-8')

    return cleaned_text_utf8

# Example usage (commented out for PCI):
# cleaned = read_file_to_text('example.txt')
# print(cleaned)


In [ ]:
# Read File to Text
clean_text_dict = {}
for filename_index, filename_now in enumerate(filenames_in_list):
  print(f"PROCESSSING #{filename_index}: {filename_now}")
  clean_text_dict[filename_now] = read_file_to_text(filename_now)
  print(f"LENGTH: {len(clean_text_dict[filename_now])}")


In [ ]:
for filename_in_list_now in clean_text_dict.keys():
  print(f"FILENAME: {filename_in_list_now}")
  print(clean_text_dict[filename_in_list_now][:500])
  print("\n\n==========")

In [ ]:
def reformat_paragraphs(text_output: str) -> str:
    """
    Reformats the paragraphs in the given text to remove hard returns.

    Parameters:
    text_output (str): The input text to be reformatted.

    Returns:
    str: The reformatted text with hard returns removed.
    """
    # Split the text into paragraphs based on blank lines (one or more newlines)
    paragraphs = re.split(r'\n\s*\n', text_output.strip())

    # Process each paragraph to remove hard returns
    reformatted_paragraphs = []
    for paragraph in paragraphs:
        # Replace hard returns (newlines within a paragraph) with a space
        reformatted_paragraph = paragraph.replace('\n', ' ')
        reformatted_paragraphs.append(reformatted_paragraph)

    # Join paragraphs back with double newlines to separate them
    reformatted_text = '\n\n'.join(reformatted_paragraphs)

    return reformatted_text

In [ ]:
def write_str_to_file(directory_output: str, filename_output: str, text_output: str) -> bool:
    """
    Writes the given text to a file in the specified output directory after checking for hard returns within paragraphs.

    Parameters:
    directory_output (str): The directory where the file will be saved.
    filename_output (str): The name of the output file.
    text_output (str): The text to be written to the file.

    Returns:
    bool: True if the file is written successfully, False otherwise.
    """
    try:
        # Ensure the output directory exists
        if not os.path.exists(directory_output):
            os.makedirs(directory_output)

        # Reformat the text to remove hard returns within paragraphs
        reformatted_text = reformat_paragraphs(text_output)

        # Determine the full path for the output file
        output_file_path = os.path.join(directory_output, filename_output)

        # Write the reformatted text to the output file
        with open(output_file_path, 'w', encoding='utf-8') as file:
            file.write(reformatted_text)

        return True
    except Exception as e:
        print(f"An error occurred: {e}")
        return False

# Example usage (commented out for PCI):
# result = write_str_to_file('output_directory', 'cleaned_text_file.txt', 'This is a sample text with hard returns within paragraphs.\nHere is the next line of the same paragraph.\n\nThis is a new paragraph.')
# print(result)


In [ ]:
# Specify the output directory
directory_clean_output = "./clean"

clean_text_reformat_dict = {}

# Iterate over the dictionary
for filename_input, cleaned_text in clean_text_dict.items():
    # Print the processing message
    print(f"PROCESSING: {filename_input}")

    # Create the output filename by replacing '_verified.txt' with '_cleaned.txt'
    filename_output = filename_input.replace('_verified.txt', '_cleaned.txt')

    # Call the function to write the cleaned text to the file
    result = write_str_to_file(directory_clean_output, filename_output, cleaned_text)

    # Create clean reformatted text dictionary
    clean_text_reformat_dict[filename_input] = reformat_paragraphs(clean_text_dict[filename_input])

    # Print the result of the call
    print(f"Result of writing {filename_output}: {result}")

# Example usage (commented out for PCI):
# clean_text_dict = {
#     'book_proust_en_swans-way_moncrieff_original_verified.txt': 'Cleaned text content for book 1.',
#     'another_book_verified.txt': 'Cleaned text content for another book.',
#     # Add more entries as needed
# }
# directory_clean_output = "./"
# for filename_input, cleaned_text in clean_text_dict.items():
#     print(f"PROCESSING: {filename_input}")
#     filename_output = filename_input.replace('_verified.txt', '_cleaned.txt')
#     result = write_str_to_file(directory_clean_output, filename_output, cleaned_text)
#     print(f"Result of writing {filename_output}: {result}")


In [ ]:
for text_name, text_clean in clean_text_reformat_dict.items():
  print(f"FILENAME: {text_name}")
  print(text_clean[:3000])
  print("\n\n")

# A. Segment Text: Create

In [ ]:
def detect_language_from_filename(filename: str) -> str:
    """
    Detects the language from the given filename based on substrings '_en_', '_fr_', '_de_'.

    Parameters:
    filename (str): The filename to extract the language code from.

    Returns:
    str: The language code ('en', 'fr', 'de').
    """

    print(f"FILENAME: {filename}")
    if '_en_' in filename:
        return 'en'
    elif '_fr_' in filename:
        return 'fr'
    elif '_de_' in filename:
        return 'de'
    else:
        # Default to English if language detection fails
        return 'en'

In [ ]:
for filename_clean_now in os.listdir(directory_clean_output):
  print(f"FILENAME: {filename_clean_now}")
  print(f"          {detect_language_from_filename(filename_clean_now)}")
  print("\n\n")

In [ ]:
# Ensure consistent language detection
DetectorFactory.seed = 0

def detect_language(text_str: str) -> str:
    """
    Detects the language of the given text using langdetect.

    Parameters:
    text_str (str): The input text whose language needs to be detected.

    Returns:
    str: The language code ('en', 'fr', 'de').
    """
    lang = detect(text_str)
    if '_en_' in lang:
        return 'en'
    elif '_fr_' in lang:
        return 'fr'
    elif '_de_' in lang:
        return 'de'
    else:
        # Default to English if language detection fails
        return 'en'


In [ ]:
def segment_text(text_str: str, segment_type: str, language_code: str = 'en') -> List[str]:
    """
    Segments the input text into a list of strings based on the segment_type method.

    Parameters:
    text_str (str): The input text to be segmented.
    segment_type (str): The method of segmentation ('sentence', 'paragraph', 'windowDDDD').
    language_code (str): The language code for sentence segmentation ('en', 'fr', 'de'). If None, detect language automatically.

    Returns:
    List[str]: A list of segmented strings.
    """
    # Detect the language of the text if not provided
    if language_code is None:
        language_code = detect_language(text_str)

    if segment_type == 'sentence':
        if language_code == 'en':
            # Use PySBD for English
            print(f"USING PySBD FOR LANGUAGE CODE: {language_code}")
            segmenter = pysbd.Segmenter(language=language_code, clean=False)
            segments = segmenter.segment(text_str)
            segments = [x.strip() for x in segments]
            return segments
        elif language_code == 'fr':
            # Use improved RegEx for French
            # print(f"USING RegEx FOR LANGUAGE CODE: {language_code}")
            # sentence_endings = re.compile(r'(?<=[.!?])\s+|(?<=;\s)')
            # segments = sentence_endings.split(text_str)
            print(f"USING PySBD FOR LANGUAGE CODE: {language_code}")
            segmenter = pysbd.Segmenter(language=language_code, clean=False)
            segments = segmenter.segment(text_str)
            segments = [x.strip() for x in segments]
            return segments
        elif language_code == 'de':
            # Use SpaCy for German
            # print(f"USING SpaCy FOR LANGUAGE CODE: {language_code}")
            # nlp = spacy.load('de_core_news_lg')
            # doc = nlp(text_str)
            # segments = [sent.text for sent in doc.sents]
            print(f"USING PySBD FOR LANGUAGE CODE: {language_code}")
            segmenter = pysbd.Segmenter(language=language_code, clean=False)
            segments = segmenter.segment(text_str)
            segments = [x.strip() for x in segments]
            return segments
        else:
            raise ValueError(f"Invalid language code: {language_code}")

    elif segment_type == 'paragraph':
        # Split paragraphs based on one or more blank lines
        return re.split(r'\n\s*\n', text_str.strip())

    elif segment_type.startswith('window'):
        # Extract the window size from segment_type (e.g., 'window500')
        match = re.match(r'window(\d+)', segment_type)
        if match:
            window_size = int(match.group(1))
            words = text_str.split()
            # Ensure the window size chunking is done by word tokens
            segments = [' '.join(words[i:i + window_size]) for i in range(0, len(words), window_size)]
            return segments
        else:
            raise ValueError("Invalid segment_type format for 'window'. It should be like 'window500'.")

    else:
        raise ValueError("Invalid segment_type. It should be 'sentence', 'paragraph', or 'windowDDDD'.")

# Example usage (commented out for PCI):
# text = "This is a sample text. It contains multiple sentences. And also paragraphs.\n\nThis is a new paragraph."
# segments = segment_text(text, 'sentence', 'en')
# print(segments)
# segments = segment_text(text, 'paragraph')
# print(segments)
# segments = segment_text(text, 'window500')
# print(segments)

# Example usage with French text (commented out for PCI):
# clean_text_dict = {
#     'book_proust_fr_swans-way_proust_original_verified.txt': "Longtemps, je me suis couché de bonne heure. Parfois, à peine ma bougie éteinte, mes yeux se fermaient si vite que je n'avais pas le temps de me dire: 'Je m'endors.' Et, une demi-heure après, la pensée qu'il était temps de chercher le sommeil m'éveillait; je voulais poser le volume que je croyais avoir encore dans les mains et souffler ma lumière."
# }
# segments = segment_text(clean_text_dict['book_proust_fr_swans-way_proust_original_verified.txt'], 'sentence', 'fr')
# print(segments[:5])


In [ ]:
# segments_list = segment_text(clean_text_reformat_dict['book_proust_fr_swans-way_proust_original_verified.txt'], 'sentence', 'fr')
# print(segments_list[:5])

In [ ]:
%%time

# segment_text
clean_text_reformat_seg_dict = {}

# Loop over each key in clean_text_reformat_dict
for key in clean_text_reformat_dict.keys():
    print(f"PROCESSING: {key}")
    # Extract the language code from the filename key
    language_code = detect_language_from_filename(key)
    # Call segment_text() for each value with the extracted language code
    segments_list = segment_text(clean_text_reformat_dict[key], 'sentence', language_code)
    # Save the resulting list of strings into the new dictionary
    clean_text_reformat_seg_dict[key] = segments_list

# Example: Print the first 5 sentences for each key to verify
for key, segments in clean_text_reformat_seg_dict.items():
    print(f"Key: {key}")
    print("First 5 sentences:")
    for segment in segments[:5]:
        print(segment)
    print("\n")

## Filter out Subtitles

In [ ]:
clean_text_reformat_seg_dict.keys()

In [ ]:
clean_text_reformat_seg_dict['book_proust_en_swans-way_moncrieff_original_verified.txt'][:20]

In [ ]:
%whos dict

In [ ]:
for akey in clean_text_reformat_seg_dict.keys():
  print(f"{akey}: {len(clean_text_reformat_seg_dict[akey])}")

In [ ]:
def filter_lines(segments_list, segment_char_min=5):
    """
    Filters the input list of strings based on specified criteria.

    Parameters:
    segments_list (List[str]): The input list of strings to be filtered.
    segment_char_min (int): Minimum number of characters a line must have to not be filtered out.

    Returns:
    List[str]: The filtered list of strings.
    """
    def is_blank_or_non_printing(line):
        return not line.strip()

    def is_all_caps(line):
        return line.isupper() and not any(c.islower() for c in line)

    def is_only_numbers(line):
        arabic_numbers = r'^\d+[\s]*[.!?,;:]*$'
        roman_numerals = r'\b(M{0,4}(CM|CD|D?C{0,3})(XC|XL|L?X{0,3})(IX|IV|V?I{0,3}))\b[\s]*[.!?,;:]*$'
        number_words = r'^(zero|one|two|three|four|five|six|seven|eight|nine|ten|eleven|twelve|thirteen|fourteen|fifteen|sixteen|seventeen|eighteen|nineteen|twenty|thirty|forty|fifty|sixty|seventy|eighty|ninety|hundred|thousand|million|billion)[\s]*[.!?,;:]*$'
        return bool(re.match(arabic_numbers, line, re.IGNORECASE) or
                    re.match(roman_numerals, line, re.IGNORECASE) or
                    re.match(number_words, line, re.IGNORECASE))

    def starts_with_chapter_section_part_episode_book(line):
        pattern = r'^(Chapter|Section|Part|Episode|Book)(\s+[\dIVXLCDM]+[\s]*[.!?,;:]*|[.!?,;:]*\s*[\dIVXLCDM]*)?$'
        return bool(re.match(pattern, line, re.IGNORECASE))

    filtered_segments = []
    for line in segments_list:
        trimmed_line = line.strip()
        if (len(trimmed_line) < segment_char_min or
            is_blank_or_non_printing(trimmed_line) or
            is_all_caps(trimmed_line) or
            is_only_numbers(trimmed_line) or
            starts_with_chapter_section_part_episode_book(trimmed_line)):
            continue
        filtered_segments.append(line)

    return filtered_segments
"""
# Example usage (commented out for PCI):
# Process each file and segment text

directory_clean_output = './clean'
for filename_clean_now in os.listdir(directory_clean_output):
    print(f"PROCESSING: {filename_clean_now}")
    filepath = os.path.join(directory_clean_output, filename_clean_now)
    with open(filepath, 'r', encoding='utf-8') as file:
        content = file.read()

    language_code = detect_language(content)
    # segments = segment_text(content, 'sentence', language_code)
    filtered_segments = filter_lines(segments)

    print(f"FILENAME: {filename_clean_now}")
    print("First 5 sentences:")
    for segment in filtered_segments[:5]:
        print(segment)
""";

In [ ]:
clean_text_reformat_seg_dict.keys()

In [ ]:
clean_text_reformat_seg_dict["book_proust_en_swans-way_davis_original_verified.txt"][:20]

In [ ]:
# CREATE: clean_text_reformat_seg_filter_dict
clean_text_reformat_seg_filter_dict = {}

# Loop over each key in clean_text_reformat_dict
for key in clean_text_reformat_seg_dict.keys():
    print(f"PROCESSING: {key}")
    # Extract the language code from the filename key
    language_code = detect_language_from_filename(key)
    # Call segment_text() for each value with the extracted language code
    segments_list = filter_lines(clean_text_reformat_seg_dict[key])
    # Save the resulting list of strings into the new dictionary
    clean_text_reformat_seg_filter_dict[key] = segments_list

# Example: Print the first 5 sentences for each key to verify
for key, segments in clean_text_reformat_seg_filter_dict.items():
    print(f"Key: {key}")
    print("First 5 sentences:")
    for segment_index, segment_str in enumerate(segments[:10]):
        print(f"  Line #{segment_index}: {segment_str}")
    print("\n")

In [ ]:
clean_text_reformat_seg_filter_dict.keys()

## Save Clean Filtered Segments

In [ ]:
def write_dict_of_lists_to_files(directory_out, dict_of_lists, segment_type='segments'):
    """
    Saves each list of strings from the dictionary to separate files in the specified directory.

    Parameters:
    directory_out (str): The directory where the files will be saved.
    dict_of_lists (dict): Dictionary where keys are filenames and values are lists of strings.
    segment_type (str): The suffix to be added to the output filenames.

    Returns:
    dict: A dictionary with filenames as keys and their paths as values.
    """
    # Ensure the output directory exists
    if not os.path.exists(directory_out):
        os.makedirs(directory_out)

    file_paths = {}

    for key in dict_of_lists.keys():
        # Create the output filename
        filename_out = key.replace('_verified.txt', f'_{segment_type}.txt')
        output_file_path = os.path.join(directory_out, filename_out)

        # Debug: Print the current filename and output path
        print(f"Saving file: {output_file_path}")

        # Write the list of strings to the file, one string per line
        with open(output_file_path, 'w', encoding='utf-8') as file:
            for i, line in enumerate(dict_of_lists[key]):
                file.write(line + '\n')
                # Print only the first few lines for debugging
                if i < 5:
                    print(f"Writing line {i+1}: {line}")
                elif i == 5:
                    print("... (more lines not shown)")

        # Store the filename and its path in the dictionary
        file_paths[filename_out] = output_file_path

    return file_paths




In [ ]:
segmented_file_paths = write_dict_of_lists_to_files('./segmented', clean_text_reformat_seg_filter_dict, 'segments')
print(segmented_file_paths)

# Verify the content of the files by reading them back
for filename, filepath in segmented_file_paths.items():
    print(f"Verifying content of {filename}:")
    with open(filepath, 'r', encoding='utf-8') as file:
        for i, line in enumerate(file):
            if i < 5:
                print(line.strip())
            elif i == 5:
                print("... (more lines not shown)")
                break


In [ ]:
# Download a zip archive of these files
subdir = 'segmented'

# Zip the subdirectory
shutil.make_archive(subdir, 'zip', subdir)

# Download the zip file
files.download(subdir + '.zip')

# B. Segment Text: Upload

In [ ]:
%whos dict

In [ ]:
clean_text_reformat_seg_filter_dict.keys()

In [ ]:
type(clean_text_reformat_seg_filter_dict[list(clean_text_reformat_seg_filter_dict.keys())[0]])

In [ ]:
def upload_segments(segments_subdir="segmented", overwrite_flag=False):
    """
    Uploads multiple files to the specified directory in the Colab VM.

    Parameters:
    segments_subdir (str): The subdirectory where the files will be uploaded.
    overwrite_flag (bool): If True, existing files will be overwritten.
    """
    # Step 1: Create the directory if it does not exist
    if not os.path.exists(segments_subdir):
        os.makedirs(segments_subdir)
        print(f"Created directory: {segments_subdir}")
    else:
        print(f"Directory already exists: {segments_subdir}")

    # Step 2: Upload multiple files
    uploaded_files = files.upload()

    # Step 3: Move uploaded files to the specified directory, considering the overwrite_flag
    for filename in uploaded_files.keys():
        src_path = os.path.join("/content", filename)
        dest_path = os.path.join(segments_subdir, filename)

        if os.path.exists(dest_path):
            if overwrite_flag:
                shutil.move(src_path, dest_path)
                print(f"Overwritten existing file: {filename}")
            else:
                os.remove(src_path)
                print(f"File already exists and overwrite_flag is False: {filename}")
        else:
            shutil.move(src_path, dest_path)
            print(f"Saved new file: {filename}")

# Example usage:
# upload_segments("segmented", True)


In [ ]:
upload_segments("segmented", False)

In [ ]:


def read_txt_files_into_dict_of_lists(directory_in):
    """
    Reads each .txt file from the specified directory and stores the content
    into a dictionary with filename roots as keys and lists of strings as values.

    Parameters:
    directory_in (str): The directory from where the files will be read.

    Returns:
    dict: A dictionary with filename roots as keys and lists of strings as values.
    """
    dict_of_lists = {}

    # Iterate over the files in the directory
    for filename in os.listdir(directory_in):
        if filename.endswith('.txt'):
            file_path = os.path.join(directory_in, filename)

            # Read the file and store its lines in a list
            with open(file_path, 'r', encoding='utf-8') as file:
                lines = file.readlines()
                lines = [line.strip() for line in lines]  # Strip newline characters

            # Remove the suffix to get filename_root
            filename_root = filename.replace('_original_segments.txt', '')

            # Store the list of strings in the dictionary with the filename_root as the key
            dict_of_lists[filename_root] = lines

    return dict_of_lists


In [ ]:


def process_files_in_directory(directory_in='./segmented'):
    clean_text_reformat_seg_filter_dict = {}

    for afile in os.listdir(directory_in):
        file_path = os.path.join(directory_in, afile)
        if os.path.isfile(file_path):
            print(f"PROCESS file: {afile}:")
            with open(file_path, 'r', encoding='utf-8') as file:
                lines_in = file.readlines()
                lines_clean_in = [line.strip() for line in lines_in]  # Strip newline characters
                filename_key = afile.replace('_original_segments.txt', '')
                clean_text_reformat_seg_filter_dict[filename_key] = lines_clean_in
        else:
            print(f"Skipping directory: {afile}")

    print(clean_text_reformat_seg_filter_dict.keys())
    return clean_text_reformat_seg_filter_dict




In [ ]:
# Example usage
clean_text_reformat_seg_filter_dict = process_files_in_directory('./segmented')

In [ ]:
for key,list in clean_text_reformat_seg_filter_dict.items():
  print(f"{key}: {len(list)}")

### [END]

In [ ]:
# Read in cleaned, filtered segments from ./segments/<files>_segements.txt

def read_seg_files_to_dict(directory):
    """
    Reads all files in the given directory matching the *_segments.txt pattern and returns a dictionary
    with filenames as keys and lists of strings as values.

    Parameters:
    directory (str): The directory path containing the segment files.

    Returns:
    dict: A dictionary with keys as filenames and values as lists of strings (one per line).
    """
    segments_dictionary = {}

    # Ensure the directory path ends with a separator
    directory = os.path.join(directory, '')

    # Get the list of all files matching *_segments.txt in the directory
    file_pattern = os.path.join(directory, '*_segments.txt')
    segment_files = glob.glob(file_pattern)

    for file_path in segment_files:
        # Extract the filename without the directory path
        filename = os.path.basename(file_path)

        # Read the file contents into a list of strings
        with open(file_path, 'r', encoding='utf-8') as file:
            lines = file.readlines()
            segments = [line.strip() for line in lines]

        # Store the list of strings in the dictionary
        segments_dictionary[filename] = segments

    return segments_dictionary

# Example usage
# directory = './segments_directory'
# segments_dict = read_seg_files_to_dict(directory)
# print(segments_dict)


In [ ]:
clean_text_reformat_seg_filter_dict = read_seg_files_to_dict("./segments")

In [ ]:
clean_text_reformat_seg_filter_dict.keys()

In [ ]:
clean_text_reformat_seg_filter_dict['book_proust_en_swans-way_moncrieff_original_segments.txt'][:50]

In [ ]:
# [END]

In [ ]:
segments_list = segments
SAMPLE_LEN = 50
print(f"len(segments_list): {len(segments_list)}")
print(f"segments_list[:SAMPLE_LEN]:\n")
segments_list_first10 = [segment_now.strip() for segment_now in segments_list[:SAMPLE_LEN]]
# Printing the first 10 strings in the list, one per line, after stripping whitespace
for segment_now in segments_list_first10[:SAMPLE_LEN]:
    print(segment_now.strip())
print(f"segments_list[-SAMPLE_LEN:]:\n") #  {segments_list[-SAMPLE_LEN:]}\n\n")
segments_list_lastN = [segment_now.strip() for segment_now in segments_list[-SAMPLE_LEN:]]
for segment_now in segments_list_lastN: # [-SAMPLE_LEN:]:
    print(segment_now.strip())

In [ ]:

# Directory containing cleaned files
directory_clean_output = './clean'

# Iterate over files in the directory
dir_file_list_sorted = reversed(sorted(os.listdir(directory_clean_output)))
for filename_clean_now in dir_file_list_sorted:
    print(f"FILENAME: {filename_clean_now}")

    # Read the content of the file into a string
    filepath = os.path.join(directory_clean_output, filename_clean_now)
    with open(filepath, 'r', encoding='utf-8') as file:
        content = file.read()

    # Extract the language code from the filename
    try:
        language_code = get_language_from_filename(filename_clean_now)
    except ValueError as e:
        print(f"Error: {e}")
        continue

    # Segment the text using the language code
    print(f"  calling segment_text with language_code = {language_code}")
    segments = segment_text(content, 'sentence', language_code)

    # Print the total number of lines
    print(f"Total number of lines: {len(segments)}")

    # Print the first 50 lines
    for line in segments[:50]:
        print(line)

    print("\n\n")

In [ ]:
%whos

In [ ]:
for filename_clean_now in os.listdir(directory_clean_output):
  print(f"FILENAME: {filename_clean_now}")
  print(f"          {segment_text(filename_clean_now)}")
  print("\n\n")

In [ ]:
def segment_text(text_str: str, segment_type: str) -> List[str]:
    """
    Segments the input text into a list of strings based on the segment_type method.

    Parameters:
    text_str (str): The input text to be segmented.
    segment_type (str): The method of segmentation ('sentence', 'paragraph', 'windowDDDD').

    Returns:
    List[str]: A list of segmented strings.
    """
    if segment_type == 'sentence':
        # Detect the language and use the appropriate SpaCy model
        if re.search(r'[a-zA-Z]', text_str):
            doc = nlp_en(text_str)
        elif re.search(r'[a-zA-Zéèêëàâçùûô]', text_str):
            doc = nlp_fr(text_str)
        else:
            doc = nlp_de(text_str)
        return [sent.text for sent in doc.sents]

    elif segment_type == 'paragraph':
        # Split paragraphs based on one or more blank lines
        return re.split(r'\n\s*\n', text_str.strip())

    elif segment_type.startswith('window'):
        # Extract the window size from segment_type (e.g., 'window500')
        match = re.match(r'window(\d+)', segment_type)
        if match:
            window_size = int(match.group(1))
            words = text_str.split()
            segments = [' '.join(words[i:i + window_size]) for i in range(0, len(words), window_size)]
            return segments
        else:
            raise ValueError("Invalid segment_type format for 'window'. It should be like 'window500'.")

    else:
        raise ValueError("Invalid segment_type. It should be 'sentence', 'paragraph', or 'windowDDDD'.")

# Example usage (commented out for PCI):
# text = "This is a sample text. It contains multiple sentences. And also paragraphs.\n\nThis is a new paragraph."
# segments = segment_text(text, 'sentence')
# print(segments)
# segments = segment_text(text, 'paragraph')
# print(segments)
# segments = segment_text(text, 'window500')
# print(segments)


In [ ]:
clean_text_dict = {}
for filename_index, filename_now in enumerate(filenames_in_list):
  print(f"PROCESSSING #{filename_index}: {filename_now}")
  clean_text_dict[filename_now] = read_file_to_text(filename_now)
  print(f"LENGTH: {len(clean_text_dict[filename_now])}")

In [ ]:
def remove_unprintable_characters(text):
    """
    Removes or converts unprintable characters from a given text.

    Parameters:
    text (str): The input string.

    Returns:
    str: The cleaned string with printable characters.
    """
    # Normalize the text to NFKD (Normalization Form KD)
    text = unicodedata.normalize('NFKD', text)
    # Encode to ASCII bytes, ignoring non-ASCII characters
    text = text.encode('ascii', 'ignore')
    # Decode back to string
    text = text.decode('ascii')
    return text

def dictionary_of_list_sentences_from_file_list(filenames):
    """
    Reads a list of files and returns a dictionary with translators as keys and lists of sentences as values.

    Parameters:
    filenames (list of str): A list of filenames to read.

    Returns:
    dict: A dictionary with translator names as keys and lists of sentences as values.
    """
    translator_dict = {}
    translators = ['davis', 'enright', 'moncrieff', 'proust']

    for filename in filenames:
        # Identify the translator's name from the filename
        translator = None
        for t in translators:
            if re.search(fr'_{t}_', filename):
                translator = t
                break

        if translator is None:
            raise ValueError(f"Translator not found in filename: {filename}")

        with open(filename, 'r', encoding='utf-8') as file:
            sentences = file.readlines()
            # Remove trailing newline characters and unprintable characters
            sentences = [remove_unprintable_characters(sentence.strip()) for sentence in sentences]
            translator_dict[translator] = sentences

    return translator_dict

# Example usage (ensure you have appropriate files to test this):
# filenames = [
#     'book_proust_en_swans-way_davis_sentence_clean.txt',
#     'book_proust_en_swans-way_enright_sentence_clean.txt',
#     'book_proust_en_swans-way_moncrieff_sentence_clean.txt',
#     'book_proust_fr_swans-way_proust_sentence_clean.txt'
# ]
translator_sentences_dt = dictionary_of_list_sentences_from_file_list(list_clean_filenames)
# Displaying a small sample to avoid large outputs
for translator, sentences in translator_sentences_dt.items():
    print(f"Sentence Count: {len(sentences)}")  # Print only the first 5 sentences for each translator
    print(f"{translator}: {sentences[:5]}")  # Print only the first 5 sentences for each translator
    print("\n")

In [ ]:
print(translator_sentences_dt.keys())

In [ ]:
print(json.dumps(translator_sentences_dt['davis'][:50], indent=4))

In [ ]:
print(json.dumps(translator_sentences_dt['davis'][:50], indent=4))

In [ ]:
translator_sentences_dt.keys()

### [SHORTCUT] LangChain Ollama Sentiment Call

In [ ]:
%%time

response = ollama.chat(model=model_name, messages=[
  {
    'role': 'user',
    'content': 'Explain why love is blind, Ray Charles is blind, yet Ray Charles is not love?',
  },
])

print(response['message']['content'])

In [ ]:
%%time

response = ollama.chat(
    model="mistral",
    messages='Explain why love is blind, Ray Charles is blind, yet Ray Charles is not love?',
    format="json",
    options=Options(
        temperature=0.0,
        num_ctx=100000,
        num_predict=-1,
    )
)

print(response['message']['content'])

In [ ]:
%%time

response = ollama.generate(
    model=model_name,
    prompt='Explain why love is blind, Ray Charles is blind, yet Ray Charles is not love?'
)

print(response['message']['content'])

In [ ]:
print(SENTIMENT_RUBRIC)

In [ ]:
response = ollama.chat(model=model_name, messages=[
  {
    'role': 'user',
    'content': 'Explain why love is blind, Ray Charles is blind, yet Ray Charles is not love?',
  },
])

print(response['message']['content'])

def get_ollama_sentiment(text_str):
    """
    Mock function to simulate sentiment analysis.

    Parameters:
    text (str): The input string.

    Returns:
    float: A mock sentiment score.
    """
    # construct Prompt
    sentiment_prompt = f"""

    ###SENTENCE:
    {text_str}

    ###INSTRUCTIONS:
    {SENTIMENT_RUBRIC}
    """

    sentiment_polarity_float_str = llm.invoke(sentiment_prompt)
    print(f"sentiment_polarity_float_str: \n\n{sentiment_polarity_float_str}\n\n")

    return sentiment_polarity_float_str  # Replace with actual sentiment analysis call

resp_polarity_float_str = get_ollama_sentiment("I don't care about lint")
print(f"resp_polarity_float_str: {resp_polarity_float_str}")

In [ ]:
            completion = ollama.chat(
                model="mistral",
                messages=messages,
                format="json",
                options=Options(
                    temperature=0.0,
                    num_ctx=100000,
                    num_predict=-1,
                ),

In [ ]:
print(f"USING LLM model_name: {model_name}")

llm = Ollama(model = model_name, temperature=0.0, format="JSON")

In [ ]:
def get_ollama_sentiment(text_str):
    """
    Mock function to simulate sentiment analysis.

    Parameters:
    text (str): The input string.

    Returns:
    float: A mock sentiment score.
    """
    # construct Prompt
    sentiment_prompt = f"""

    ###SENTENCE:
    {text_str}

    ###INSTRUCTIONS:
    {SENTIMENT_RUBRIC}
    """

    sentiment_polarity_float_str = llm.invoke(sentiment_prompt)
    print(f"sentiment_polarity_float_str: \n\n{sentiment_polarity_float_str}\n\n")

    return sentiment_polarity_float_str  # Replace with actual sentiment analysis call

resp_polarity_float_str = get_ollama_sentiment("I don't care about lint")
print(f"resp_polarity_float_str: {resp_polarity_float_str}")

#### Version #1

In [ ]:
def remove_unprintable_characters(text):
    """
    Removes or converts unprintable characters from a given text.

    Parameters:
    text (str): The input string.

    Returns:
    str: The cleaned string with printable characters.
    """
    # Normalize the text to NFKD (Normalization Form KD)
    text = unicodedata.normalize('NFKD', text)
    # Encode to ASCII bytes, ignoring non-ASCII characters
    text = text.encode('ascii', 'ignore')
    # Decode back to string
    text = text.decode('ascii')
    return text

def dictionary_of_list_sentences_from_file_list(filenames):
    """
    Reads a list of files and returns a dictionary with translators as keys and lists of sentences as values,
    and their corresponding sentiment scores with modified keys.

    Parameters:
    filenames (list of str): A list of filenames to read.

    Returns:
    dict: A dictionary with translator names as keys and lists of sentences as values,
          and sentiment scores with modified keys.
    """
    translator_dict = {}
    translators = ['davis', 'enright', 'moncrieff', 'proust']

    for filename in filenames:
        # Identify the translator's name from the filename
        translator = None
        for t in translators:
            if re.search(fr'_{t}_', filename):
                translator = t
                break

        if translator is None:
            raise ValueError(f"Translator not found in filename: {filename}")

        with open(filename, 'r', encoding='utf-8') as file:
            sentences = file.readlines()
            # Remove trailing newline characters and unprintable characters
            sentences = [remove_unprintable_characters(sentence.strip()) for sentence in sentences]
            translator_dict[translator] = sentences

            # Calculate sentiment scores for each sentence
            sentiment_scores = [get_ollama_sentiment(sentence) for sentence in sentences]
            # sentiment_scores = [get_ollama_sentiment(sentence) for sentence in tqdm(sentences, desc=f"Processing {translator}")]
            sentiment_key = f"{translator}_sentiment"
            translator_dict[sentiment_key] = sentiment_scores

    return translator_dict

# Example usage (ensure you have appropriate files to test this):
# filenames = [
#     'book_proust_en_swans-way_davis_sentence_clean.txt',
#     'book_proust_en_swans-way_enright_sentence_clean.txt',
#     'book_proust_en_swans-way_moncrieff_sentence_clean.txt',
#     'book_proust_fr_swans-way_proust_sentence_clean.txt'
# ]
translator_sentences_dict = dictionary_of_list_sentences_from_file_list(list_clean_filenames)
# Displaying a small sample to avoid large outputs
for translator, sentences in translator_sentences_dict.items():
  print(f"{translator}: {sentences[:5]}")  # Print only the first 5 sentences for each translator


#### Version #2

In [ ]:
import re
import unicodedata
from tqdm import tqdm
import requests

def remove_unprintable_characters(text):
    """
    Removes or converts unprintable characters from a given text.

    Parameters:
    text (str): The input string.

    Returns:
    str: The cleaned string with printable characters.
    """
    # Normalize the text to NFKD (Normalization Form KD)
    text = unicodedata.normalize('NFKD', text)
    # Encode to ASCII bytes, ignoring non-ASCII characters
    text = text.encode('ascii', 'ignore')
    # Decode back to string
    text = text.decode('ascii')
    return text

def get_ollama_sentiment(text_str):
    """
    Function to simulate sentiment analysis.

    Parameters:
    text_str (str): The input string.

    Returns:
    float: A sentiment score.
    """
    # Construct Prompt
    sentiment_prompt = f"""

    ###SENTENCE:
    {text_str}

    ###INSTRUCTIONS:
    {SENTIMENT_RUBRIC}
    """

    try:
        # Assuming llm.invoke is the method to call the sentiment API
        # Mock implementation here for demo purposes
        sentiment_polarity_float_str = llm.invoke(sentiment_prompt)
        print(f"sentiment_polarity_float_str: \n\n{sentiment_polarity_float_str}\n\n")
        return sentiment_polarity_float_str  # Replace with actual sentiment analysis call
    except requests.ConnectionError as e:
        print(f"Connection error: {e}")
        return None

def dictionary_of_list_sentences_from_file_list(filenames):
    """
    Reads a list of files and returns a dictionary with translators as keys and lists of sentences as values,
    and their corresponding sentiment scores with modified keys.

    Parameters:
    filenames (list of str): A list of filenames to read.

    Returns:
    dict: A dictionary with translator names as keys and lists of sentences as values,
          and sentiment scores with modified keys.
    """
    translator_dict = {}
    translators = ['davis', 'enright', 'moncrieff', 'proust']

    for filename in filenames:
        # Identify the translator's name from the filename
        translator = None
        for t in translators:
            if re.search(fr'_{t}_', filename):
                translator = t
                break

        if translator is None:
            raise ValueError(f"Translator not found in filename: {filename}")

        with open(filename, 'r', encoding='utf-8') as file:
            sentences = file.readlines()
            # Remove trailing newline characters and unprintable characters
            sentences = [remove_unprintable_characters(sentence.strip()) for sentence in sentences]
            translator_dict[translator] = sentences

            # Calculate sentiment scores for each sentence with progress bar
            sentiment_scores = [get_ollama_sentiment(sentence) for sentence in tqdm(sentences, desc=f"Processing {translator}")]
            sentiment_key = f"{translator}_sentiment"
            translator_dict[sentiment_key] = sentiment_scores

    return translator_dict

# Example usage (ensure you have appropriate files to test this):
# filenames = [
#     'book_proust_en_swans-way_davis_sentence_clean.txt',
#     'book_proust_en_swans-way_enright_sentence_clean.txt',
#     'book_proust_en_swans-way_moncrieff_sentence_clean.txt',
#     'book_proust_fr_swans-way_proust_sentence_clean.txt'
# ]
# translator_sentences = dictionary_of_list_sentences_from_file_list(filenames)
# Displaying a small sample to avoid large outputs
# for translator, sentences in translator_sentences.items():
#     print(f"{translator}: {sentences[:5]}")  # Print only the first 5 sentences for each translator

translator_sentences_dict = dictionary_of_list_sentences_from_file_list(list_clean_filenames)
# Displaying a small sample to avoid large outputs
for translator, sentences in translator_sentences_dict.items():
  print(f"{translator}: {sentences[:5]}")  # Print only the first 5 sentences for each translator


In [ ]:
translator_sentences_dict = dictionary_of_list_sentences_from_file_list(list_clean_filenames)
print(json.dumps(translator_sentences_dict))

#### Version #3

In [ ]:
import re
import unicodedata
from tqdm import tqdm

def remove_unprintable_characters(text):
    """
    Removes or converts unprintable characters from a given text.

    Parameters:
    text (str): The input string.

    Returns:
    str: The cleaned string with printable characters.
    """
    # Normalize the text to NFKD (Normalization Form KD)
    text = unicodedata.normalize('NFKD', text)
    # Encode to ASCII bytes, ignoring non-ASCII characters
    text = text.encode('ascii', 'ignore')
    # Decode back to string
    text = text.decode('ascii')
    return text

def batch_sentences(sentences, batch_size=10):
    """
    Splits the sentences into batches of specified size.

    Parameters:
    sentences (list of str): The list of sentences.
    batch_size (int): The size of each batch.

    Returns:
    generator: A generator that yields batches of sentences.
    """
    for i in range(0, len(sentences), batch_size):
        yield sentences[i:i + batch_size]

def get_ollama_sentiment_batch(sentences_batch):
    """
    Function to simulate sentiment analysis for a batch of sentences.

    Parameters:
    sentences_batch (list of str): The batch of input sentences.

    Returns:
    list of float: A list of sentiment scores.
    """
    sentiment_scores = []
    for sentence in sentences_batch:
        # Construct Prompt
        sentiment_prompt = f"""
        ###SENTENCE:
        {sentence}

        ###INSTRUCTIONS:
        {SENTIMENT_RUBRIC}
        """

        try:
            # Mock implementation here for demo purposes
            sentiment_polarity_float_str = llm.invoke(sentiment_prompt)
            print(f"sentiment_polarity_float_str: \n\n{sentiment_polarity_float_str}\n\n")
            sentiment_scores.append(sentiment_polarity_float_str)  # Replace with actual sentiment analysis call
        except requests.ConnectionError as e:
            print(f"Connection error: {e}")
            sentiment_scores.append(None)

    return sentiment_scores

def dictionary_of_list_sentences_from_file_list(filenames, batch_size=10):
    """
    Reads a list of files and returns a dictionary with translators as keys and lists of sentences as values,
    and their corresponding sentiment scores with modified keys.

    Parameters:
    filenames (list of str): A list of filenames to read.
    batch_size (int): The number of sentences to process in each batch.

    Returns:
    dict: A dictionary with translator names as keys and lists of sentences as values,
          and sentiment scores with modified keys.
    """
    translator_dict = {}
    translators = ['davis', 'enright', 'moncrieff', 'proust']

    for filename in filenames:
        # Identify the translator's name from the filename
        translator = None
        for t in translators:
            if re.search(fr'_{t}_', filename):
                translator = t
                break

        if translator is None:
            raise ValueError(f"Translator not found in filename: {filename}")

        with open(filename, 'r', encoding='utf-8') as file:
            sentences = file.readlines()
            # Remove trailing newline characters and unprintable characters
            sentences = [remove_unprintable_characters(sentence.strip()) for sentence in sentences]
            translator_dict[translator] = sentences

            # Process sentiment scores in batches
            sentiment_scores = []
            for sentences_batch in tqdm(batch_sentences(sentences, batch_size), desc=f"Processing {translator}"):
                batch_scores = get_ollama_sentiment_batch(sentences_batch)
                sentiment_scores.extend(batch_scores)

            sentiment_key = f"{translator}_sentiment"
            translator_dict[sentiment_key] = sentiment_scores

    return translator_dict

# Example usage (ensure you have appropriate files to test this):
# filenames = [
#     'book_proust_en_swans-way_davis_sentence_clean.txt',
#     'book_proust_en_swans-way_enright_sentence_clean.txt',
#     'book_proust_en_swans-way_moncrieff_sentence_clean.txt',
#     'book_proust_fr_swans-way_proust_sentence_clean.txt'
# ]
# translator_sentences = dictionary_of_list_sentences_from_file_list(filenames)
# Displaying a small sample to avoid large outputs
# for translator, sentences in translator_sentences.items():
#     print(f"{translator}: {sentences[:5]}")  # Print only the first 5 sentences for each translator


In [ ]:
print(f"USING LLM model_name: {model_name}")

llm = Ollama(model = model_name)

In [ ]:
%%time

# Test JSON metrics only call

score_json_only = llm.invoke(prompt_score_translation_json_only)
print(f"score_json_only: \n\n{score_json_only}\n\n")

In [ ]:
# Get the filename from the uploaded files
upload_filename = list(uploaded_raw_text.keys())[0]

# Extract the book title from the filename
book_title = "_".join(upload_filename.split(".")[0].split("_")[1:5])
print(book_title)

In [ ]:
lines_list = []
for fn in uploaded_raw_text.keys():
  with open(fn, 'r') as fp:
    lines_list = fp.readlines()

print(f" len(lines_list): {len(lines_list)}")
print(lines_list[:5])

# Sentence Length Histogram

In [ ]:
# Calculate the length of each line
def plot_histogram_line_lengths(book_title, lines_list):
    line_lengths = [len(line) for line in lines_list]

    # Plot the histogram of line lengths
    plt.figure(figsize=(10, 6))
    plt.hist(line_lengths, bins=100, edgecolor='black', alpha=0.7)
    plt.title(f'Histogram of Line Lengths\n{book_title}', fontsize=14)
    plt.xlabel('Line Length', fontsize=12)
    plt.ylabel('Frequency', fontsize=12)
    plt.grid(True)
    plt.show()

In [ ]:
for book_title, lines_list, in clean_text_reformat_seg_filter_dict.items():
  print(f"{book_title}: {len(lines_list)}")
  plot_histogram_line_lengths(book_title, lines_list)

In [ ]:
def plot_kde_line_lengths(book_title, lines_list):
    # Convert lines_list to an array of line lengths
    line_lengths = np.array([len(line) for line in lines_list], dtype=np.float64)

    # Compute the KDE of the line lengths
    kde = gaussian_kde(line_lengths)
    x_values = np.linspace(min(line_lengths), max(line_lengths), 1000)
    kde_values = kde(x_values)

    # Plot the KDE
    plt.figure(figsize=(10, 6))
    plt.plot(x_values, kde_values, color='blue')
    plt.title(f'KDE of Line Lengths\n{book_title}', fontsize=14)
    plt.xlabel('Line Length', fontsize=12)
    plt.ylabel('Density', fontsize=12)
    plt.grid(True)
    plt.show()

In [ ]:
for book_title, lines_list, in clean_text_reformat_seg_filter_dict.items():
  print(f"{book_title}: {len(lines_list)}")
  plot_kde_line_lengths(book_title, lines_list)

# Split into n Chunks

In [ ]:
%whos dict

In [ ]:
for book_title, lines_list, in clean_text_reformat_seg_filter_dict.items():
  print(f"{book_title}: {len(lines_list)}")

# len(clean_text_reformat_seg_filter_dict[list(clean_text_reformat_seg_filter_dict.keys())[0]])

In [ ]:
def chunk_dict_of_lists(dict_of_list, chunk_count):
    def chunk_list(lst, chunk_count):
        chunks = [[] for _ in range(chunk_count)]
        for i, item in enumerate(lst):
            chunks[i % chunk_count].append(item)
        return [" ".join(chunk) for chunk in chunks]

    chunked_dict = {key: chunk_list(value, chunk_count) for key, value in dict_of_list.items()}
    return chunked_dict




In [ ]:
# Split into 100 Chunks
# Assuming clean_text_reformat_seg_filter_dict is defined as the input dictionary
clean_text_chunked100_dict = chunk_dict_of_lists(clean_text_reformat_seg_filter_dict, 100)

for book_title, lines_list in clean_text_chunked100_dict.items():
  print(f"{book_title}: {len(lines_list)}")

In [ ]:
# Split into 500 Chunks
# Assuming clean_text_reformat_seg_filter_dict is defined as the input dictionary
clean_text_chunked500_dict = chunk_dict_of_lists(clean_text_reformat_seg_filter_dict, 500)

for book_title, lines_list in clean_text_chunked500_dict.items():
  print(f"{book_title}: {len(lines_list)}")

# Lexical Richness

## Extract Test Lines

In [ ]:
def extract_text(lines_list, start_per, length_per):
    # Step 1: Concatenate the entire lines_list into big_string
    big_string = ''.join(lines_list)

    # Step 2: Calculate the start and end character indexes
    total_chars = len(big_string)
    char_index_start = int(start_per / 100 * total_chars)
    char_index_end = char_index_start + int(length_per / 100 * total_chars)

    # Ensure char_index_end does not exceed the total length
    if char_index_end > total_chars:
        char_index_end = total_chars

    # Step 3: Translate char indexes into line indexes
    cumulative_length = 0
    line_index_start = line_index_end = None

    for i, line in enumerate(lines_list):
        cumulative_length += len(line)
        if line_index_start is None and cumulative_length > char_index_start:
            line_index_start = i
        if line_index_end is None and cumulative_length >= char_index_end:
            line_index_end = i
            break

    # If line_index_end was not set, it means char_index_end was the last character
    if line_index_end is None:
        line_index_end = len(lines_list) - 1

    # Step 4: Extract the sublines list
    sublines_list = lines_list[line_index_start:line_index_end+1]

    return sublines_list

# Example usage
lines_test_list = [
    "This is the first line.\n",
    "This is the second line.\n",
    "This is the third line.\n",
    "This is the fourth line.\n",
    "This is the fifth line.\n"
]
start_per = 20  # Start extraction from 20% into the text
length_per = 30  # Extract 30% of the text length

extracted_lines = extract_text(lines_test_list, start_per, length_per)
for line in extracted_lines:
    print(line, end='')


In [ ]:
len(lines_list)

In [ ]:
start_per = 0  # Start extraction from 20% into the text
length_per = 10  # Extract 30% of the text length

extracted_lines = extract_text(lines_list, start_per, length_per)
print(f"len(extracted_lines): {len(extracted_lines)}")
# for line in extracted_lines:
#     print(line, end='')

extracted_string = ' '.join(extracted_lines)
print(f"len(extracted_string): {len(extracted_string)}")
print(extracted_string[:100])

## Metrics on Test Line

In [ ]:
novel_title = "Swan's Way"

In [ ]:
lex = LexicalRichness(extracted_string)

print(f"Lexical Richness")
print(f"       novel_title: {novel_title}")
print(f"   start char len%: {start_per}")
print(f"  length char len%: {length_per}")
print("==================================================")

print("\nwords")
print(f"lex.words: {lex.words}")

print("\nterms")
print(f"lex.terms: {lex.terms}")

print("\ntype-token ratio (TTR) of text")
print(f"lex.ttr: {lex.ttr}")

print("\nroot type-token ratio (RTTR) of text")
print(f"lex.rttr: {lex.rttr}")

print("\ncorrected type-token ratio (CTTR) of text")
print(f"lex.cttr: {lex.cttr}")

print("\nmean segmental type-token ratio (MSTTR)")
print(f"lex.msttr(segment_window=25): {lex.msttr(segment_window=25)}")

print("\nmoving average type-token ratio (MATTR)")
print(f"lex.mattr(window_size=25): {lex.mattr(window_size=25)}")

print("\nMeasure of Textual Lexical Diversity (MTLD)")
print(f"lex.mtld(threshold=0.72): {lex.mtld(threshold=0.72)}")

print("\nhypergeometric distribution diversity (HD-D) measure")
print(f"lex.hdd(draws=42) : {lex.hdd(draws=42)}")

print("\nvoc-D measure")
print(f"lex.vocd(ntokens=50, within_sample=100, iterations=3): {lex.vocd(ntokens=50, within_sample=100, iterations=3)}")

print("\nHerdan's lexical diversity measure")
print(f"lex.Herdan: {lex.Herdan}")

print("\nSummer's lexical diversity measure")
print(f"lex.Summer: {lex.Summer}")

print("\nDugast's lexical diversity measure")
print(f"lex.Dugast: {lex.Dugast}")

print("\nMaas's lexical diversity measure")
print(f"lex.Maas: {lex.Maas}")

print("\nYule's K")
print(f"lex.yulek: {lex.yulek}")

print("\nYule's I")
print(f"lex.yulei: {lex.yulei}")

print("\nHerdan's Vm")
print(f"lex.herdanvm: {lex.herdanvm}")

print("\nSimpson's D")
print(f"lex.simpsond: {lex.simpsond}")

## Metrics on Translations

In [ ]:
def get_lexical_richness_metrics(lines_list):
    metrics = []

    for line in lines_list:
        lex = LexicalRichness(line)

        metrics.append({
            "words": lex.words,
            "terms": lex.terms,
            "ttr": lex.ttr,
            "rttr": lex.rttr,
            "cttr": lex.cttr,
            "msttr": lex.msttr(segment_window=3),
            "mattr": lex.mattr(window_size=5),
            "mtld": lex.mtld(threshold=0.72),
            "hdd": lex.hdd(draws=42),
            "vocd": lex.vocd(ntokens=50, within_sample=100, iterations=3),
            "Herdan": lex.Herdan,
            "Summer": lex.Summer,
            "Dugast": lex.Dugast,
            "Maas": lex.Maas,
            "yulek": lex.yulek,
            "yulei": lex.yulei,
            "herdanvm": lex.herdanvm,
            "simpsond": lex.simpsond
        })

    df = pd.DataFrame(metrics)

    # Interpolate to handle NaN values
    df = df.interpolate(method='linear', limit_direction='both', axis=0)

    # Check if there are still NaN values after interpolation
    if df.isnull().values.any():
        print("NaN values found in the dataframe after interpolation. Filling remaining NaN values with column means.")
        # Fill any remaining NaN values with the mean of the column
        df = df.fillna(df.mean())

    return df

""";
# SOME METRICS DO NOT WORK ON SHORT TEXT LIKE THIS



# Mock data and dictionary for testing
clean_text_reformat_seg_filter_test_dict = {
    "Book1": ["This is a sample text.", "Another line of text."],
    "Book2": ["Yet another sample text.", "More text for testing."],
    "Book3": ["This is a sample text.", "Another line of text."],
    "Book4": ["Yet another sample text.", "More text for testing."],
    "Book5": ["This is a sample text.", "Another line of text."],
    "Book6": ["Yet another sample text.", "More text for testing."],
    "Book7": ["This is a sample text.", "Another line of text."],
    "Book8": ["Yet another sample text.", "More text for testing."],
    "Book9": ["This is a sample text.", "Another line of text."],
    "Book10": ["Yet another sample text.", "More text for testing."],

}

lexical_richness_test_dict = {}

for book_title, segments_list in clean_text_reformat_seg_filter_test_dict.items():
    print(f"{book_title}: {len(segments_list)}")
    lexical_richness_test_dict[book_title] = get_lexical_richness_metrics(segments_list)

for book_title, metrics_df in lexical_richness_test_dict.items():
    print(f"Metrics for {book_title}:")
    print(metrics_df)
    print("-" * 30)
""";

In [ ]:
# Initialize the dictionary to hold the lexical richness metrics dataframes

lexical_richness_chunks100_dict = {}

In [ ]:
%%time

# Iterate over the chunked dictionary and calculate lexical richness metrics

for book_title, chunks in clean_text_chunked100_dict.items():
  if "_en_" in book_title or True:
    print(f"{book_title}: {len(chunks)}")
    lexical_richness_chunks100_dict[book_title] = get_lexical_richness_metrics(chunks)


In [ ]:
lexical_richness_chunks100_dict["book_proust_en_swans-way_moncrieff"][:5]


In [ ]:
lexical_richness_chunks100_dict["book_proust_en_swans-way_moncrieff"].info()

In [ ]:
lexical_richness_chunks100_dict["book_proust_en_swans-way_moncrieff"].describe()

In [ ]:
# Initialize the dictionary to hold the lexical richness metrics dataframes

lexical_richness_chunks500_dict = {}

In [ ]:
%%time

# Iterate over the chunked dictionary and calculate lexical richness metrics

for book_title, chunks in clean_text_chunked500_dict.items():
  if "_en_" in book_title:
    print(f"{book_title}: {len(chunks)}")
    lexical_richness_chunks500_dict[book_title] = get_lexical_richness_metrics(chunks)
  elif "_fr_" in book_title:
    print(f"{book_title}: {len(chunks)}")
    lexical_richness_chunks500_dict[book_title] = get_lexical_richness_metrics(chunks)
  else:
    print(f"ERROR: Illegal language_type: {book_title}")
    print(f"SKIPPING...\n")
    continue

In [ ]:
lexical_richness_chunks500_dict["book_proust_en_swans-way_moncrieff"][:5]

In [ ]:
lexical_richness_chunks500_dict["book_proust_en_swans-way_moncrieff"].info()

In [ ]:
lexical_richness_chunks500_dict["book_proust_en_swans-way_moncrieff"].describe()

## Save


In [ ]:
def save_dict_of_df(dict_of_df, subdir):
    # Create the subdirectory if it doesn't exist
    if not os.path.exists(subdir):
        os.makedirs(subdir)

    for filename, df in dict_of_df.items():
        # Sanitize filename to remove or replace invalid characters and ensure a single .csv extension
        sanitized_filename = re.sub(r'[^\w\-_ ]', '_', filename)
        sanitized_filename = re.sub(r'\.[^.]*$', '', sanitized_filename) + '.csv'

        # Full path to save the file in the specified subdirectory
        full_path = os.path.join(subdir, sanitized_filename)

        # Save the DataFrame to a CSV file
        df.to_csv(full_path, index=False)
        print(f"Saved {full_path}")

In [ ]:
save_dict_of_df(lexical_richness_chunks100_dict, "lexical_richness100")

In [ ]:
# Download a zip archive of these files
subdir = 'lexical_richness100'

# Zip the subdirectory
shutil.make_archive(subdir, 'zip', subdir)

# Download the zip file
files.download(subdir + '.zip')

In [ ]:
save_dict_of_df(lexical_richness_chunks500_dict, "lexical_richness500")

In [ ]:
# Download a zip archive of these files
subdir = 'lexical_richness500'

# Zip the subdirectory
shutil.make_archive(subdir, 'zip', subdir)

# Download the zip file
files.download(subdir + '.zip')

## Plot

In [ ]:
# COMMENT OUT any metric you don't want to plot

metric_list = [
    'msttr',
    'mattr',
    'mtld',
    'hdd',
    'vocd',
    # 'Herdan',
    # 'Summer',
    # 'Dugast',
    # 'Maas',
    # 'yulek',
    # 'yulei',
    'herdanvm',
    'simpsond'
    ]

In [ ]:
def plot_normalized_smoothed_metrics(dict_of_dataframes, metric_list):
    """
    Plots the normalized and smoothed metrics from the dataframes in the given dictionary.

    Parameters:
    dict_of_dataframes (dict): Dictionary where keys are book titles and values are pandas dataframes with metrics.
    metric_list (list): List of metrics to be plotted.
    """
    for book_title, df in dict_of_dataframes.items():
        # Filter the dataframe to include only the specified metrics
        filtered_df = df[metric_list]

        # Z-score normalization
        normalized_df = filtered_df.apply(zscore)

        # 10% window size for SMA smoothing
        window_size = max(1, len(df) // 10)
        smoothed_df = normalized_df.rolling(window=window_size, min_periods=1).mean()

        # Plotting
        plt.figure(figsize=(12, 6))
        for column in smoothed_df.columns:
            plt.plot(smoothed_df[column], label=column)

        plt.title(f"Metrics for {book_title}")
        plt.xlabel("Chunk Index")
        plt.ylabel("Z-score Normalized Value")
        plt.grid(True)

        # Place legend outside the plot area
        plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
        plt.tight_layout(rect=[0, 0, 0.85, 1])  # Adjust the layout to make space for the legend

        plt.show()

In [ ]:
plot_normalized_smoothed_metrics(lexical_richness_chunks100_dict, metric_list)

In [ ]:
plot_normalized_smoothed_metrics(lexical_richness_chunks500_dict, metric_list)

# Text Descriptives (SpaCy: en/fr)

In [ ]:
# View Available/Default Metrics

td.get_valid_metrics()
# {'quality', 'readability', 'all', 'descriptive_stats', 'dependency_distance', 'pos_proportions', 'information_theory', 'coherence'}

In [ ]:
%%time

test_str = "The world is changed. I feel it in the water. I feel it in the earth. I smell it in the air. Much that once was is lost, for none now live who remember it."

df = td.extract_metrics(text=test_str, spacy_model="en_core_web_lg", metrics=["readability", "coherence"])
df.transpose()

In [ ]:
%%time

df = td.extract_metrics(text=test_str, spacy_model="en_core_web_lg", metrics=None) # ["readability", "coherence"])

df.transpose()

## Use SpaCy Pipeline

In [ ]:
# load your favourite spacy model (remember to install it first using e.g. `python -m spacy download en_core_web_sm`)

nlp = spacy.load("en_core_web_lg")

nlp.add_pipe("textdescriptives/all")
doc = nlp("The world is changed. I feel it in the water. I feel it in the earth. I smell it in the air. Much that once was is lost, for none now live who remember it.")

# access some of the values
doc._.readability
doc._.token_length

In [ ]:
doc._.descriptive_stats

In [ ]:
doc._.entropy
doc._.coherence
doc._.quality
doc._.dependency_distance
doc._.pos_proportions
doc._.information_theory

In [ ]:
td.extract_dict(doc)

In [ ]:
text_metrics_df = td.extract_df(doc)
text_metrics_df.transpose()

In [ ]:
print(type(td.extract_dict(doc)))
print(type(td.extract_df(doc)))

In [ ]:
# Get metrics in dataframe and pivot vertically

# text_metrics_df.head()

text_metrics_vertical_df = text_metrics_df.T
text_metrics_vertical_df.columns = ["value"]

text_metrics_vertical_df.head()


In [ ]:
"""
# Create datetime string in datetime_now str
datetime_now_str = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")


# Create filename with datetime suffix
filename_download = f'text_metrics_{book_title}_{datetime_now_str}.csv'

print(filename_download)
""";

In [ ]:
"""
# Download text_stylo_metrics to file

text_metrics_vertical_df.head()
text_metrics_vertical_df.to_csv(filename_download, index=False)

# Assuming text_metrics_vertical_df is your vertical dataframe
text_metrics_vertical_df = text_metrics_vertical_df.reset_index()
text_metrics_vertical_df.columns = ['Metric', 'Value']

# Save the dataframe to a CSV file
text_metrics_vertical_df.to_csv(filename_download, index=False)

files.download(filename_download)
""";

## Metrics on Text Descriptives

In [ ]:
nlp = spacy.load("en_core_web_lg")

nlp.add_pipe("textdescriptives/all")
doc = nlp("The world is changed. I feel it in the water. I feel it in the earth. I smell it in the air. Much that once was is lost, for none now live who remember it.")

# access some of the values
doc._.readability
doc._.token_length

In [ ]:


# Load spaCy model and add textdescriptives pipeline
# nlp = spacy.load("en_core_web_lg")
# nlp.add_pipe("textdescriptives/all")

def get_text_descriptive_metrics(lines_list, language_type='en'):
    if language_type == 'en':
        print(f"  In get_text_descriptive_metrics(): loading en")
        nlp = spacy.load("en_core_web_lg")
        nlp.add_pipe("textdescriptives/all")
    elif language_type == 'fr':
        print(f"  In get_text_descriptive_metrics(): loading fr")
        nlp = spacy.load("fr_core_news_lg")
        nlp.add_pipe("textdescriptives/all")
    else:
        raise ValueError("Invalid language type. Please choose 'en' or 'fr'.")
        exit()


    metrics = []

    for line in lines_list:
        doc = nlp(line)


        metrics.append({
            "flesch_reading_ease": doc._.readability.get('flesch_reading_ease'),
            "flesch_kincaid_grade": doc._.readability.get('flesch_kincaid_grade'),
            "smog": doc._.readability.get('smog'),
            "gunning_fog": doc._.readability.get('gunning_fog'),
            "automated_readability_index": doc._.readability.get('automated_readability_index'),
            "coleman_liau_index": doc._.readability.get('coleman_liau_index'),
            "lix": doc._.readability.get('lix'),
            "rix": doc._.readability.get('rix'),
            "token_length_mean": doc._.token_length.get('token_length_mean'),
            "token_length_median": doc._.token_length.get('token_length_median'),
            "token_length_std": doc._.token_length.get('token_length_std')
        })


    df = pd.DataFrame(metrics)
    return df

# Mock data and dictionary for testing
clean_text_reformat_seg_filter_test_dict = {
    "Book1": ["This is a sample text.", "Another line of text."],
    "Book2": ["Yet another sample text.", "More text for testing."],
    "Book3": ["This is a sample text.", "Another line of text."],
    "Book4": ["Yet another sample text.", "More text for testing."],
    "Book5": ["This is a sample text.", "Another line of text."],
    "Book6": ["Yet another sample text.", "More text for testing."],
    "Book7": ["This is a sample text.", "Another line of text."],
    "Book8": ["Yet another sample text.", "More text for testing."],
    "Book9": ["This is a sample text.", "Another line of text."],
    "Book10": ["Yet another sample text.", "More text for testing."]
}




In [ ]:
# Initialize the dictionary to hold the lexical richness metrics dataframes

textdescriptives_chunks100_dict = {}

In [ ]:
%%time

# Iterate over the chunked dictionary and calculate lexical richness metrics

for book_title, chunks in clean_text_chunked100_dict.items():
  print(f"PROCESSING: book_title: {book_title}")
  if "_en_" in book_title:
    print(f"{book_title}: {len(chunks)}")
    textdescriptives_chunks100_dict[book_title] = get_text_descriptive_metrics(chunks, "en")
  elif "_fr_" in book_title:
    print(f"{book_title}: {len(chunks)}")
    textdescriptives_chunks100_dict[book_title] = get_text_descriptive_metrics(chunks, "fr")
  else:
    print(f"ERROR: Illegal language_type: {book_title}")
    print(f"SKIPPING...\n")
    continue

In [ ]:
textdescriptives_chunks100_dict.keys()

In [ ]:
textdescriptives_chunks100_dict["book_proust_en_swans-way_moncrieff"].info()

In [ ]:
len(textdescriptives_chunks100_dict)

In [ ]:
textdescriptives_chunks100_dict["book_proust_en_swans-way_moncrieff"].info()

In [ ]:
textdescriptives_chunks100_dict["book_proust_en_swans-way_moncrieff"].describe()

In [ ]:
# Initialize the dictionary to hold the lexical richness metrics dataframes

textdescriptives_chunks500_dict = {}

In [ ]:
%%time

# Iterate over the chunked dictionary and calculate lexical richness metrics

for book_title, chunks in clean_text_chunked500_dict.items():
  if "_en_" in book_title:
    print(f"{book_title}: {len(chunks)}")
    textdescriptives_chunks500_dict[book_title] = get_text_descriptive_metrics(chunks, "en")
  elif "_fr_" in book_title:
    print(f"{book_title}: {len(chunks)}")
    textdescriptives_chunks500_dict[book_title] = get_text_descriptive_metrics(chunks, "fr")
  else:
    print(f"ERROR: Illegal language_type: {book_title}")
    print(f"SKIPPING...\n")
    continue

In [ ]:
len(textdescriptives_chunks500_dict)

In [ ]:
# textdescriptives_chunks500_dict[list(textdescriptives_chunks500_dict.keys())[0]][:5]

In [ ]:
textdescriptives_chunks500_dict["book_proust_en_swans-way_moncrieff"].info()

In [ ]:
textdescriptives_chunks500_dict["book_proust_en_swans-way_moncrieff"].describe()

## Save

In [ ]:
save_dict_of_df(lexical_richness_chunks100_dict, "text_descriptives100")

In [ ]:
# Download a zip archive of these files
subdir = 'text_descriptives100'

# Zip the subdirectory
shutil.make_archive(subdir, 'zip', subdir)

# Download the zip file
files.download(subdir + '.zip')

In [ ]:
save_dict_of_df(lexical_richness_chunks500_dict, "text_descriptives500")

In [ ]:
# Download a zip archive of these files
subdir = 'text_descriptives500'

# Zip the subdirectory
shutil.make_archive(subdir, 'zip', subdir)

# Download the zip file
files.download(subdir + '.zip')

## Plot

In [ ]:
# COMMENT OUT any metric you don't want to plot

metric_list = [
    'flesch_reading_ease',
    'flesch_kincaid_grade',
    'smog',
    'gunning_fog',
    'automated_readability_index',
    'coleman_liau_index',
    'lix',
    'rix',
    'token_length_mean',
    'token_length_median',
    'token_length_std'
    ]

In [ ]:
# Example usage:
# Assuming lexical_richness_dict is the dictionary containing the DataFrames with metrics
plot_normalized_smoothed_metrics(textdescriptives_chunks100_dict, metric_list)

In [ ]:
# Example usage:
# Assuming lexical_richness_dict is the dictionary containing the DataFrames with metrics
plot_normalized_smoothed_metrics(textdescriptives_chunks500_dict, metric_list)

# Sentiments

In [ ]:
# Test sentence lists in en and fr

sentence_test_en_list = [
    "I hate this so much, it's the worst experience I've ever had.",  # Very negative
    "This is absolutely terrible, I can't believe how bad it is.",   # Very negative
    "I'm really disappointed and upset with how things turned out.",  # Negative
    "This is not what I expected at all, quite frustrating.",        # Negative
    "I feel indifferent about this, it neither excites me nor bothers me.",  # Neutral
    "It's okay, not great but not bad either.",                      # Neutral
    "This is pretty good, I'm quite satisfied with it.",             # Positive
    "I really enjoyed this, it made my day better.",                 # Positive
    "Absolutely fantastic! I'm thrilled and very happy with it.",    # Very positive
    "This is the best thing ever, I'm ecstatic and over the moon!"   # Very positive
]

sentence_test_fr_list = [
    "Je déteste ça tellement, c'est la pire expérience que j'ai jamais eue.",  # Very negative
    "C'est absolument terrible, je ne peux pas croire à quel point c'est mauvais.",  # Very negative
    "Je suis vraiment déçu et contrarié par la tournure des événements.",  # Negative
    "Ce n'est pas du tout ce à quoi je m'attendais, c'est assez frustrant.",  # Negative
    "Je me sens indifférent à ce sujet, ça ne me dérange ni ne m'excite.",  # Neutral
    "C'est correct, pas génial mais pas mauvais non plus.",  # Neutral
    "C'est plutôt bien, je suis assez satisfait.",  # Positive
    "J'ai vraiment apprécié ça, ça a égayé ma journée.",  # Positive
    "Absolument fantastique ! Je suis ravi et très heureux.",  # Very positive
    "C'est la meilleure chose qui soit, je suis extatique et sur un nuage !"  # Very positive
]

*italicized text*# II. Compute NEW SENTIMENTS

## VADER

In [ ]:
!pip install vaderSentiment

In [ ]:
def convert_stars_to_int(star_string):
    """
    Converts a star rating string to an integer.

    :param star_string: A string representing the star rating (e.g., "3 stars").
    :return: An integer representing the star rating (1 to 5).
    """
    try:
        # Split the string to extract the numeric part
        parts = star_string.split()

        # Extract the first part and convert to integer
        star_int = int(parts[0])

        # Check if the integer is within the valid range
        if 1 <= star_int <= 5:
            return star_int
        else:
            raise ValueError("Star rating out of valid range (1 to 5).")
    except (ValueError, IndexError) as e:
        print(f"Error converting star string to int: {e}")
        return None  # or raise an exception, or handle it as per your requirements

In [ ]:
# Sentiment: VADER (English Only)
# pip install vaderSentiment
# https://github.com/cjhutto/vaderSentiment
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [ ]:
def get_sentiment_vader_list(sentence_list):
    """
    Given an input list of strings, returns an equal length list of the sentiment polarity values
    for corresponding strings using VADER.

    :param sentence_list: List of sentences (strings) to analyze.
    :return: List of sentiment polarity values.
    """
    # Initialize the VADER sentiment intensity analyzer
    analyzer = SentimentIntensityAnalyzer()

    # Analyze the sentiment for each sentence in the list
    sentiment_scores = []
    for sentence in tqdm(sentence_list, desc="VADER Sentiment Analysis"):
        # Check if the sentence is not empty
        if sentence.strip():
            sentiment = analyzer.polarity_scores(sentence)
            # Append the compound score to the results list
            sentiment_scores.append(sentiment['compound'])
        else:
            # Append a neutral score for empty sentences
            sentiment_scores.append(0.0)

    return sentiment_scores

In [ ]:
# Test English
get_sentiment_vader_list(sentence_test_en_list)


In [ ]:
# Test French
get_sentiment_vader_list(sentence_test_fr_list)


## TextBlob

In [ ]:
# Sentiment: TextBlob (Fr with extension)
!pip install -U textblob
!pip install -U textblob-fr

In [ ]:

from textblob import TextBlob
from textblob import Blobber
from textblob_fr import PatternTagger, PatternAnalyzer

In [ ]:
def get_sentiment_textblob_list(sentence_list, language="en"):

    sentiment_scores = []

    if language == "en":

        # Analyze the sentiment for each sentence in the list
        for sentence in tqdm(sentence_list, desc="TextBlob Sentiment Analysis"):
            # Check if the sentence is not empty
            if sentence.strip():
                textblob_analysis = TextBlob(sentence)
                # Append the compound score to the results list
                sentiment_scores.append(textblob_analysis.sentiment.polarity)
            else:
                # Append a neutral score for empty sentences
                sentiment_scores.append(0.0)

    elif language == "fr":

        tb = Blobber(pos_tagger=PatternTagger(), analyzer=PatternAnalyzer())

        # Analyze the sentiment for each sentence in the list
        for sentence in sentence_list:
            # Check if the sentence is not empty
            if sentence.strip():
                sentiment_blob = tb(sentence)
                # Append the compound score to the results list
                sentiment_scores.append(sentiment_blob.sentiment[0])
            else:
                # Append a neutral score for empty sentences
                sentiment_scores.append(0.0)

    else:
        print(f"  ERROR: Invalid language for TextBlob: {language}")
        exit()

    return sentiment_scores

In [ ]:
# Test English
get_sentiment_textblob_list(sentence_test_en_list, "en")

In [ ]:
# Test French
get_sentiment_textblob_list(sentence_test_fr_list, "fr")


## BERTMulti

In [ ]:
# Sentiment: Huggingface Transformers BERTMulti
# pip install -q transformers
from transformers import pipeline, AutoTokenizer

In [ ]:
def get_sentiment_bertmulti_list(sentence_list, language="en"):
    """
    Given an input list of strings, returns an equal length list of the sentiment polarity values
    for corresponding strings using VADER.

    :param sentence_list: List of sentences (strings) to analyze.
    :return: List of sentiment polarity values.
    """
    # Initialize the HF Transformer sentiment model
    hf_model_name = "nlptown/bert-base-multilingual-uncased-sentiment"
    MAX_BERTMULTI_LEN = 350 # 512 is token limit * 3/4 word/token = 350 + ~50 padding
    tokenizer = AutoTokenizer.from_pretrained(hf_model_name)
    sentiment_bertmulti_pipeline = pipeline("sentiment-analysis", model=hf_model_name, tokenizer=tokenizer)

    # Analyze the sentiment for each sentence in the list
    sentiment_scores = []
    for sentence in tqdm(sentence_list, desc="BERT-Multi Sentiment Analysis"):
        # Trim sentence to MAX_BERTMULTI_LEN = 512
        sentence_trimmed = sentence[:MAX_BERTMULTI_LEN]
        if sentence_trimmed.strip():
            sentiment = sentiment_bertmulti_pipeline(sentence_trimmed)
            # Append the compound score to the results list
            # print(f" type(sentiment): {type(sentiment)}")
            # print(f" sentiment: {sentiment}")
            # print(f" dir(sentiment): {dir(sentiment)}")
            sentiment_star_int = convert_stars_to_int(sentiment[0]['label'])
            sentiment_scores.append(sentiment_star_int)
        else:
            # Append a neutral score for empty sentences
            sentiment_scores.append(0.0)

    return sentiment_scores

In [ ]:
# Test English
get_sentiment_bertmulti_list(sentence_test_en_list, "en")

In [ ]:
# Test French
get_sentiment_bertmulti_list(sentence_test_fr_list, "fr")

## Mistral LLM

In [ ]:
MODEL_OLLAMA = "mistral7bsenti"

In [ ]:
def get_sentiment_ollama_list(sentence_list, language="en", ollama_model=MODEL_OLLAMA):

    failure_count = 0
    sentiment_scores = []
    # for sentence in tqdm(sentence_list, desc="Ollama Sentiment Analysis"):
    # tqdm.tqdm(epochs, position=0, leave=True)
    # from tqdm.auto import tqdm
    for sentence in sentence_list:

        response = ollama.generate(
            model=ollama_model,
            # PROMPT #1: Directional Correctness for human validation of model results
            # prompt=f"###SENTENCE:\n{sentence}\n\n###INSTRUCTIONS:\nGiven the above ###SENTENCE, estimate the sentiment as either 'negative', 'neutral', or 'positive' Return only one word for sentiment and nothing else, no header, explaination, introduction, summary, conclusion. Only return a single float number for the sentiment polarity"
            # PROMPT #2: Precise -1.0 to +1.0 sentiment for calcuation
            prompt=f"###SENTENCE:\n{sentence}\n\n###INSTRUCTIONS:\nGiven the above ###SENTENCE, estimate the sentiment as a float number from -1.0 (most negative) to 0.0 (neutral) to 1.0 (most positive). Return only one float number between -1.0 and 1.0   for sentiment polarity and nothing else, no header, explaination, introduction, summary, conclusion. Only return a single float number for the sentiment polarity"
        )

        sentiment_polarity = response['response'].strip()
        # print(f"sentiment_polarity: {sentiment_polarity}")
        # print(f"type(sentiment_polarity): {type(sentiment_polarity)}")

        try:
            sentiment_polarity = float(sentiment_polarity)
            if sentiment_polarity > 1.0:
                sentiment_scores.append(1.0)
            elif sentiment_polarity < -1.0:
                sentiment_scores.append(-1.0)
            else:
                sentiment_scores.append(sentiment_polarity)
        except (ValueError, TypeError):
            # In case of error, default to 0.0
            failure_count += 1
            sentiment_scores.append(0.0)

    print(f"FAILURE COUNT: {failure_count}")
    print(f"FAILURE RATE: {(failure_count/len(sentence_list)):.2f}")
    return sentiment_scores


In [ ]:
# Test English
get_sentiment_ollama_list(sentence_test_en_list, "en")

In [ ]:
# Test French
get_sentiment_ollama_list(sentence_test_fr_list, "fr")

## Combine into a Dataframe

In [ ]:
clean_text_reformat_seg_filter_dict.keys()

In [ ]:
for asent in clean_text_reformat_seg_filter_dict['book_proust_fr_swans-way_proust'][:10]:
  asentiment = get_sentiment_ollama_list([asent], "fr")
  print(f"  SENTIMENT: {asentiment} for {asent}\n\n")

In [ ]:
# len(clean_text_reformat_seg_filter_dict[list(clean_text_reformat_seg_filter_dict.keys())[0]])

In [ ]:
def create_sentiment_dataframes(clean_text_reformat_seg_filter_dict):
    clean_text_sentiments_dict = {}

    for key, segment_list in clean_text_reformat_seg_filter_dict.items():
        print(f"Processing sentiments for: {key}")

        vader_sentiments = get_sentiment_vader_list(segment_list)
        textblob_sentiments = get_sentiment_textblob_list(segment_list, language="en")
        bertmulti_sentiments = get_sentiment_bertmulti_list(segment_list, language="en")
        ollama_sentiments = get_sentiment_ollama_list(segment_list, language="en") # , ollama_model=MODEL_OLLAMA)

        data = {
            'text': segment_list,
            'vader': vader_sentiments,
            'textblob': textblob_sentiments,
            'bertmulti': bertmulti_sentiments,
            'mistral': ollama_sentiments
        }

        df = pd.DataFrame(data)
        clean_text_sentiments_dict[key] = df

    return clean_text_sentiments_dict

In [ ]:
clean_text_reformat_seg_filter_dict.keys()

In [ ]:
# type(clean_text_reformat_seg_filter_dict["book_proust_en_swans-way_moncrieff_original_segments.txt"])

type(clean_text_reformat_seg_filter_dict["book_proust_en_swans-way_moncrieff"])

len(clean_text_reformat_seg_filter_dict["book_proust_en_swans-way_moncrieff"])

In [ ]:
# clean_text_reformat_seg_filter_dict["book_proust_en_swans-way_moncrieff_original_segments.txt"][:10]

clean_text_reformat_seg_filter_dict["book_proust_en_swans-way_moncrieff"][:5]
len(clean_text_reformat_seg_filter_dict["book_proust_en_swans-way_moncrieff"])

In [ ]:
# len(clean_text_reformat_seg_filter_dict["book_proust_en_swans-way_moncrieff_original_segments.txt"])

len(clean_text_reformat_seg_filter_dict["book_proust_en_swans-way_moncrieff"])

## Truncate Strings for Transformer (max tokens 512)

In [ ]:
MAX_STRING_LEN = 350

In [ ]:
def truncate_strings_in_dict_of_lists(dictionary_in, shorter_length=MAX_STRING_LEN):
    """
    Truncates the individual strings in the input dictionary's lists to the specified shorter length.

    Parameters:
    dictionary_in (dict): The input dictionary with lists of strings as values.
    shorter_length (int): The length to which each string should be truncated.

    Returns:
    dict: A copy of the input dictionary with truncated strings.
    """
    truncated_dict = {}

    for key, value_list in dictionary_in.items():
        # Truncate each string in the list to the specified length
        truncated_list = [s[:shorter_length] for s in value_list]
        print(f"   TRUNCATED Original length for first string in {key}: {len(value_list[0]) if value_list else 0}")
        print(f"             Truncated length for first string in {key}: {len(truncated_list[0]) if truncated_list else 0}")
        # Store the list with truncated strings in the output dictionary
        truncated_dict[key] = truncated_list

    return truncated_dict


In [ ]:
%whos list

In [ ]:
clean_test_reformat_seg_filter_truncate_dict = truncate_strings_in_dict_of_lists(clean_text_reformat_seg_filter_dict, MAX_STRING_LEN)

In [ ]:
clean_test_reformat_seg_filter_truncate_dict.keys()

In [ ]:
# clean_test_reformat_seg_filter_test_dict['book_proust_en_swans-way_moncrieff_original_segments.txt'][:10]

clean_test_reformat_seg_filter_truncate_dict['book_proust_en_swans-way_moncrieff'][:10]
len(clean_test_reformat_seg_filter_truncate_dict['book_proust_en_swans-way_moncrieff'])

In [ ]:
# len(clean_test_reformat_seg_filter_test_dict['book_proust_en_swans-way_moncrieff'])

len(clean_test_reformat_seg_filter_truncate_dict['book_proust_en_swans-way_moncrieff'])

### Get Sentiments One String per

In [ ]:
!ollama list

In [ ]:
MAX_CALL_OLLAMA = 3

In [ ]:
def get_sentiment_ollama(text: str) -> float:
    for attempt in range(1, MAX_CALL_OLLAMA + 1):
        try:
            logging.info(f"Attempt {attempt}: Sending text to Ollama for sentiment analysis")
            res = ollama.chat(
                model="mistral7bsenti",
                messages=[{'role': 'user', 'content': f'Only give the sentiment polarity float value between -1.0 and 1.0 for: {text}'}],
                stream=False,
                options={"temperature": 0.3, "top_p": 0.5}
            )
            if 'message' in res and 'content' in res['message']:
                text_sentiment_float_str = res['message']['content'].strip()
                try:
                    text_sentiment_float = float(text_sentiment_float_str)
                    logging.info(f"Received sentiment analysis response and successfully converted to float")
                    return text_sentiment_float
                except ValueError:
                    logging.warning(f"Attempt {attempt}: Could not convert response to float: {text_sentiment_float_str}")
            else:
                logging.error(f"Attempt {attempt}: Unexpected API response format: {res}")
        except Exception as e:
            logging.error(f"Attempt {attempt}: Error during sentiment analysis for text: {e}")
    logging.error(f"All {MAX_CALL_OLLAMA} attempts failed for text: {text}. Returning 0.0")
    return 0.0

### Get Sentiments by List

In [ ]:
test_str = "I love lint"
test_list = ["i love lint", "I don't care", "I hate you"]

# sentiment_polarity = get_sentiment_ollama(test_str)

# sentiment_polarity = get_sentiment_ollama_list([test_str], "en")
sentiment_polarity = get_sentiment_ollama_list(test_list, "en")
print(sentiment_polarity)

In [ ]:
"""
def get_sentiment_vader_safe(segment_list, language, dir_out):
    try:
        vader_sentiments = get_sentiment_vader_list(segment_list)  # Placeholder for actual sentiment analysis
        df = pd.DataFrame({'text': segment_list, 'vader': vader_sentiments})
        partial_vader_path = os.path.join(dir_out, 'vader_partial.csv')
        df.to_csv(partial_vader_path, index=False)
        return True
    except Exception as e:
        print(f"Error in get_sentiment_vader: {e}")
        return False
""";

In [ ]:
def get_sentiment_vader_safe(segment_list, language, dir_out, book_title):
    try:
        vader_sentiments = get_sentiment_vader_list(segment_list)  # Placeholder for actual sentiment analysis
        df = pd.DataFrame({'text': segment_list, 'vader': vader_sentiments})
        partial_vader_path = os.path.join(dir_out, f"{book_title}_vader_partial.csv")
        df.to_csv(partial_vader_path, index=False)
        return True
    except Exception as e:
        print(f"Error in get_sentiment_vader: {e}")
        return False


In [ ]:
"""
def get_sentiment_textblob_safe(segment_list, language, dir_out):
    try:
        textblob_sentiments = get_sentiment_textblob_list(segment_list)  # Placeholder for actual sentiment analysis
        df = pd.DataFrame({'text': segment_list, 'textblob': textblob_sentiments})
        partial_textblob_path = os.path.join(dir_out, 'textblob_partial.csv')
        df.to_csv(partial_textblob_path, index=False)
        return True
    except Exception as e:
        print(f"Error in get_sentiment_textblob: {e}")
        return False
""";

In [ ]:


def get_sentiment_textblob_safe(segment_list, language, dir_out, book_title):
    try:
        textblob_sentiments = get_sentiment_textblob_list(segment_list, language)  # Placeholder for actual sentiment analysis
        df = pd.DataFrame({'text': segment_list, 'textblob': textblob_sentiments})
        partial_textblob_path = os.path.join(dir_out, f"{book_title}_textblob_partial.csv")
        df.to_csv(partial_textblob_path, index=False)
        return True
    except Exception as e:
        print(f"Error in get_sentiment_textblob: {e}")
        return False



In [ ]:
"""
def get_sentiment_bertmulti_safe(segment_list, language, dir_out):
    try:
        bertmulti_sentiments = get_sentiment_bertmulti_list(segment_list)  # Placeholder for actual sentiment analysis
        df = pd.DataFrame({'text': segment_list, 'bertmulti': bertmulti_sentiments})
        partial_bertmulti_path = os.path.join(dir_out, 'bertmulti_partial.csv')
        df.to_csv(partial_bertmulti_path, index=False)
        return True
    except Exception as e:
        print(f"Error in get_sentiment_bertmulti: {e}")
        return False
""";

In [ ]:


def get_sentiment_bertmulti_safe(segment_list, language, dir_out, book_title):
    try:
        bertmulti_sentiments = get_sentiment_bertmulti_list(segment_list)  # Placeholder for actual sentiment analysis
        df = pd.DataFrame({'text': segment_list, 'bertmulti': bertmulti_sentiments})
        partial_bertmulti_path = os.path.join(dir_out, f"{book_title}_bertmulti_partial.csv")
        df.to_csv(partial_bertmulti_path, index=False)
        return True
    except Exception as e:
        print(f"Error in get_sentiment_bertmulti: {e}")
        return False


In [ ]:
MODEL_OLLAMA = 'mistral7bsenti'

In [ ]:
MODEL_OLLAMA = 'mistral7bsenti'

def get_sentiment_ollama_list(sentence_list, language="en", dir_out="", ollama_model=MODEL_OLLAMA):
    output_file = os.path.join(dir_out, 'ollama_sentiment.csv')

    # Check if the output file already exists
    if os.path.exists(output_file):
        try:
            df = pd.read_csv(output_file)
            # Ensure the existing data matches the input sentence list
            if list(df['text']) == sentence_list:
                print("Output file already exists and matches the input sentence list.")
                return df['ollama_sentiment'].tolist()
            else:
                print("Output file exists but does not match the input sentence list. Reprocessing...")
        except Exception as e:
            print(f"Error reading existing output file: {e}. Reprocessing...")

    failure_count = 0
    sentiment_scores = []

    for sentence in sentence_list:
        response = ollama.generate(
            model=ollama_model,
            prompt=f"###SENTENCE:\n{sentence}\n\n###INSTRUCTIONS:\nGiven the above ###SENTENCE, estimate the sentiment as a float number from -1.0 (most negative) to 0.0 (neutral) to 1.0 (most positive). Return only one float number between -1.0 and 1.0 for sentiment polarity and nothing else, no header, explanation, introduction, summary, conclusion. Only return a single float number for the sentiment polarity"
        )

        sentiment_polarity = response['response'].strip()

        try:
            sentiment_polarity = float(sentiment_polarity)
            if sentiment_polarity > 1.0:
                sentiment_scores.append(1.0)
            elif sentiment_polarity < -1.0:
                sentiment_scores.append(-1.0)
            else:
                sentiment_scores.append(sentiment_polarity)
        except (ValueError, TypeError):
            failure_count += 1
            sentiment_scores.append(0.0)

    df = pd.DataFrame({'text': sentence_list, 'ollama_sentiment': sentiment_scores})
    try:
        df.to_csv(output_file, index=False)
    except Exception as e:
        print(f"Error saving output file: {e}")

    print(f"FAILURE COUNT: {failure_count}")
    print(f"FAILURE RATE: {(failure_count/len(sentence_list)):.2f}")

    return sentiment_scores


In [ ]:
# USE: get_sentiment_ollama_list()

"""
def get_sentiment_ollama_safe(segment_list, language, dir_out):
    try:
      ollama_sentiment_list = []
      for asegment in segment_list:
        # print(asegment)
        ollama_sentiment = get_sentiment_ollama(segment_list)  # Placeholder for actual sentiment analysis
        ollama_sentiment_list.append(ollama_sentiment)
      df = pd.DataFrame({'text': segment_list, 'mistral': ollama_sentiment_list})
      partial_ollama_path = os.path.join(dir_out, 'ollama_partial.csv')
      df.to_csv(partial_ollama_path, index=False)
      return True
    except Exception as e:
        print(f"Error in get_sentiment_ollama: {e}")
        return False
""";

In [ ]:
"""
def get_sentiment_all(dictionary_of_lists, directory_out):
    if not os.path.exists(directory_out):
        os.makedirs(directory_out)

    clean_text_sentiments_dict = {}

    for key, segment_list in dictionary_of_lists.items():
        print(f"Processing: {key} with len(segment_list) = {len(segment_list)}")

        if get_sentiment_vader(segment_list, 'en', directory_out) and \
           get_sentiment_textblob(segment_list, 'en', directory_out) and \
           get_sentiment_bertmulti(segment_list, 'en', directory_out) and \
           get_sentiment_ollama(segment_list, 'en', directory_out):

            vader_path = os.path.join(directory_out, 'vader_partial.csv')
            textblob_path = os.path.join(directory_out, 'textblob_partial.csv')
            bertmulti_path = os.path.join(directory_out, 'bertmulti_partial.csv')
            ollama_path = os.path.join(directory_out, 'ollama_partial.csv')

            vader_df = pd.read_csv(vader_path)
            textblob_df = pd.read_csv(textblob_path)
            bertmulti_df = pd.read_csv(bertmulti_path)
            ollama_df = pd.read_csv(ollama_path)

            final_df = vader_df
            final_df['textblob'] = textblob_df['textblob']
            final_df['bertmulti'] = bertmulti_df['bertmulti']
            final_df['mistral'] = ollama_df['mistral']

            clean_text_sentiments_dict[key] = final_df

            # Optionally, save the final combined dataframe
            final_output_path = os.path.join(directory_out, key.replace('_verified.txt', '_segments.csv'))
            final_df.to_csv(final_output_path, index=False)

    return clean_text_sentiments_dict

# Example usage
clean_test_reformat_seg_filter_dict = {
    'book_proust_fr_swans-way_proust_original_verified.txt': ["Sentence 1", "Sentence 2", "Sentence 3", "Sentence 4"],
    'book_proust_en_swans-way_enright_original_verified.txt': ["Sentence A", "Sentence B", "Sentence C", "Sentence D"],
    'book_proust_en_swans-way_davis_original_verified.txt': ["Sentence X", "Sentence Y", "Sentence Z"],
    'book_proust_en_swans-way_moncrieff_original_verified.txt': ["Sentence Alpha", "Sentence Beta"]
}

# sentiments_test_dict = get_sentiment_all(clean_test_reformat_seg_filter_test_dict, 'sentiments')
""";

In [ ]:
"""
def get_sentiment_all(dictionary_of_lists, directory_out):
    if not os.path.exists(directory_out):
        os.makedirs(directory_out)

    clean_text_sentiments_dict = {}

    for key, segment_list in dictionary_of_lists.items():
        filename_root, filename_ext = os.path.splitext(key)
        print(f"Processing: {key} with len(segment_list) = {len(segment_list)}")

        success = True

        # Process VADER sentiments
        if not get_sentiment_vader_safe(segment_list, 'en', directory_out, filename_root):
            print(f"Failed to process VADER for {key}")
            success = False
        else:
            vader_path = os.path.join(directory_out, 'vader_partial.csv')
            vader_df = pd.read_csv(vader_path)

        # Process TextBlob sentiments
        if not get_sentiment_textblob_safe(segment_list, 'en', directory_out, filename_root):
            print(f"Failed to process TextBlob for {key}")
            success = False
        else:
            textblob_path = os.path.join(directory_out, 'textblob_partial.csv')
            textblob_df = pd.read_csv(textblob_path)

        # Process BERT multi-language sentiments
        if not get_sentiment_bertmulti_safe(segment_list, 'en', directory_out, filename_root):
            print(f"Failed to process BERTMulti for {key}")
            success = False
        else:
            bertmulti_path = os.path.join(directory_out, 'bertmulti_partial.csv')
            bertmulti_df = pd.read_csv(bertmulti_path)

        # Process Ollama sentiments
        if not get_sentiment_ollama_list(segment_list, 'en', directory_out, filename_root, ollama_model="mistral7bsenti"):
            print(f"Failed to process Ollama for {key}")
            success = False
        else:
            ollama_path = os.path.join(directory_out, 'ollama_partial.csv')
            ollama_df = pd.read_csv(ollama_path)

        if success:
            # Combine results
            final_df = vader_df
            final_df['textblob'] = textblob_df['textblob']
            final_df['bertmulti'] = bertmulti_df['bertmulti']
            final_df['mistral'] = ollama_df['mistral']

            clean_text_sentiments_dict[key] = final_df

            # Save the final combined dataframe
            final_output_path = os.path.join(directory_out, key.replace('_verified.txt', '_sentiments.csv'))
            final_df.to_csv(final_output_path, index=False)
        else:
            print(f"Skipping final combination for {key} due to previous failures.")

    return clean_text_sentiments_dict
""";

In [ ]:
import os
import pandas as pd

def get_sentiment_all(dictionary_of_lists, directory_out):
    if not os.path.exists(directory_out):
        os.makedirs(directory_out)

    clean_text_sentiments_dict = {}

    for key, segment_list in dictionary_of_lists.items():
        filename_root, filename_ext = os.path.splitext(key)
        print(f"Processing: {key} with len(segment_list) = {len(segment_list)}")

        success = True

        # Process VADER sentiments
        if not get_sentiment_vader_safe(segment_list, 'en', directory_out, filename_root):
            print(f"Failed to process VADER for {key}")
            success = False
        else:
            vader_path = os.path.join(directory_out, f"{filename_root}_vader_partial.csv")
            vader_df = pd.read_csv(vader_path)

        # Process TextBlob sentiments
        if not get_sentiment_textblob_safe(segment_list, 'en', directory_out, filename_root):
            print(f"Failed to process TextBlob for {key}")
            success = False
        else:
            textblob_path = os.path.join(directory_out, f"{filename_root}_textblob_partial.csv")
            textblob_df = pd.read_csv(textblob_path)

        # Process BERT multi-language sentiments
        if not get_sentiment_bertmulti_safe(segment_list, 'en', directory_out, filename_root):
            print(f"Failed to process BERTMulti for {key}")
            success = False
        else:
            bertmulti_path = os.path.join(directory_out, f"{filename_root}_bertmulti_partial.csv")
            bertmulti_df = pd.read_csv(bertmulti_path)

        # Process Ollama sentiments
        if not get_sentiment_ollama_list(segment_list, 'en', directory_out, filename_root, ollama_model="mistral7bsenti"):
            print(f"Failed to process Ollama for {key}")
            success = False
        else:
            ollama_path = os.path.join(directory_out, f"{filename_root}_ollama_partial.csv")
            ollama_df = pd.read_csv(ollama_path)

        if success:
            # Combine results
            final_df = vader_df
            final_df['textblob'] = textblob_df['textblob']
            final_df['bertmulti'] = bertmulti_df['bertmulti']
            final_df['mistral'] = ollama_df['mistral']

            clean_text_sentiments_dict[key] = final_df

            # Save the final combined dataframe
            final_output_path = os.path.join(directory_out, f"{filename_root}_sentiments.csv")
            final_df.to_csv(final_output_path, index=False)
        else:
            print(f"Skipping final combination for {key} due to previous failures.")

    return clean_text_sentiments_dict



In [ ]:
import os
import pandas as pd
import time
import random
from tqdm import tqdm

def get_sentiment_all(dictionary_of_lists, directory_out):
    if not os.path.exists(directory_out):
        os.makedirs(directory_out)

    clean_text_sentiments_dict = {}

    for key, segment_list in tqdm(dictionary_of_lists.items(), desc="Processing files"):
        filename_root, filename_ext = os.path.splitext(key)
        final_output_path = os.path.join(directory_out, f"{filename_root}_sentiments.csv")

        # Check if the final output file already exists
        if os.path.exists(final_output_path):
            print(f"Output file for {key} already exists. Skipping.")
            continue

        print(f"Processing: {key} with len(segment_list) = {len(segment_list)}")

        success = True

        # Define paths for partial sentiment files
        vader_path = os.path.join(directory_out, f"{filename_root}_vader_partial.csv")
        textblob_path = os.path.join(directory_out, f"{filename_root}_textblob_partial.csv")
        bertmulti_path = os.path.join(directory_out, f"{filename_root}_bertmulti_partial.csv")
        ollama_path = os.path.join(directory_out, f"{filename_root}_ollama_partial.csv")

        # Check for the existence of each partial sentiment file and read them if they exist
        if os.path.exists(vader_path):
            vader_df = pd.read_csv(vader_path)
            print(f"Loaded VADER results from {vader_path}")
        else:
            if not get_sentiment_vader_safe(segment_list, 'en', directory_out, filename_root):
                print(f"Failed to process VADER for {key}")
                success = False
            else:
                vader_df = pd.read_csv(vader_path)

        if os.path.exists(textblob_path):
            textblob_df = pd.read_csv(textblob_path)
            print(f"Loaded TextBlob results from {textblob_path}")
        else:
            if not get_sentiment_textblob_safe(segment_list, 'en', directory_out, filename_root):
                print(f"Failed to process TextBlob for {key}")
                success = False
            else:
                textblob_df = pd.read_csv(textblob_path)

        if os.path.exists(bertmulti_path):
            bertmulti_df = pd.read_csv(bertmulti_path)
            print(f"Loaded BERTMulti results from {bertmulti_path}")
        else:
            if not get_sentiment_bertmulti_safe(segment_list, 'en', directory_out, filename_root):
                print(f"Failed to process BERTMulti for {key}")
                success = False
            else:
                bertmulti_df = pd.read_csv(bertmulti_path)

        # Adding random sleep time between 1 and 3 seconds
        time.sleep(random.uniform(1, 3))

        if os.path.exists(ollama_path):
            ollama_df = pd.read_csv(ollama_path)
            print(f"Loaded Ollama results from {ollama_path}")
        else:
            try:
                if not get_sentiment_ollama_list(segment_list, language='en', dir_out=directory_out, ollama_model="mistral7bsenti"):
                    print(f"Failed to process Ollama for {key}")
                    success = False
                else:
                    ollama_df = pd.read_csv(ollama_path)
            except Exception as e:
                print(f"Error processing Ollama for {key}: {e}")
                # Assign default value 99 to all Ollama sentiment values
                ollama_df = pd.DataFrame({'text': segment_list, 'mistral': [99] * len(segment_list)})

        if success:
            # Combine results
            final_df = vader_df
            final_df['textblob'] = textblob_df['textblob']
            final_df['bertmulti'] = bertmulti_df['bertmulti']
            final_df['mistral'] = ollama_df['mistral']

            clean_text_sentiments_dict[key] = final_df

            # Save the final combined dataframe
            final_df.to_csv(final_output_path, index=False)
        else:
            print(f"Skipping final combination for {key} due to previous failures.")

    return clean_text_sentiments_dict

# Ensure to define or import the ollama.generate function and its dependencies


In [ ]:
sentiments_dict = get_sentiment_all(clean_test_reformat_seg_filter_truncate_dict, 'sentiments')


## Save

In [ ]:
save_dict_of_df(sentiments_dict, "text_sentiments")

In [ ]:
# Download a zip archive of these files
subdir = 'text_sentiments'

# Zip the subdirectory
shutil.make_archive(subdir, 'zip', subdir)

# Download the zip file
files.download(subdir + '.zip')

## END

In [ ]:
sentiments_test_dict.keys()

In [ ]:
sentiments_test_dict['book_proust_fr_swans-way_proust_original_verified.txt']

In [ ]:
clean_text_reformat_seg_filter_dict.keys()

In [ ]:
len(clean_text_reformat_seg_filter_dict['book_proust_fr_swans-way_proust_original_verified.txt'])

In [ ]:
clean_text_reformat_seg_filter_dict['book_proust_fr_swans-way_proust_original_verified.txt'][:10]
len(clean_text_reformat_seg_filter_dict['book_proust_fr_swans-way_proust_original_verified.txt'])

### Save Results

In [ ]:
def write_dict_of_df_to_files(directory_out, dict_of_df, segment_type='segments'):
    """
    Saves each DataFrame from the dictionary to separate files in the specified directory.

    Parameters:
    directory_out (str): The directory where the files will be saved.
    dict_of_df (dict): Dictionary where keys are filenames and values are pandas DataFrames.
    segment_type (str): The suffix to be added to the output filenames.

    Returns:
    dict: A dictionary with filenames as keys and their paths as values.
    """
    # Ensure the output directory exists
    if not os.path.exists(directory_out):
        os.makedirs(directory_out)

    file_paths = {}

    for key in dict_of_df.keys():
        # Create the output filename
        filename_out = key.replace('_verified.txt', f'_{segment_type}.csv')
        output_file_path = os.path.join(directory_out, filename_out)

        # Debug: Print the current filename and output path
        print(f"Saving file: {output_file_path}")

        # Enhanced Debug: Print the shape of the DataFrame
        print(f"DataFrame shape for {key}: {dict_of_df[key].shape}")

        # Write the DataFrame to a CSV file
        dict_of_df[key].to_csv(output_file_path, index=False)

        # Store the filename and its path in the dictionary
        file_paths[filename_out] = output_file_path

    return file_paths

In [ ]:
fileout_paths_dict = write_dict_of_df_to_files('test_sentiments', sentiments_test_dict, 'sentiments')

In [ ]:
fileout_paths_dict

In [ ]:
sentiments_all_dict.keys()

In [ ]:
sentiments_all_dict['book_proust_fr_swans-way_proust_original_verified.txt'][:10]

len(sentiments_all_dict['book_proust_fr_swans-way_proust_original_verified.txt'])

In [ ]:
fileout_paths_dict = save_dict_to_files('sentiments', sentiments_all_dict, 'sentiments')

In [ ]:
def get_sentiment_vader_safe(segment_list, lang, directory_out, filename):
    # Dummy implementation for illustration
    try:
        results = []  # Perform VADER sentiment analysis on segment_list
        df = pd.DataFrame(results)
        df.to_csv(filename, index=False)
        return True
    except Exception as e:
        print(f"Error in VADER sentiment analysis: {e}")
        return False

def get_sentiment_textblob_safe(segment_list, lang, directory_out, filename):
    # Dummy implementation for illustration
    try:
        results = []  # Perform TextBlob sentiment analysis on segment_list
        df = pd.DataFrame(results)
        df.to_csv(filename, index=False)
        return True
    except Exception as e:
        print(f"Error in TextBlob sentiment analysis: {e}")
        return False

def get_sentiment_bertmulti_safe(segment_list, lang, directory_out, filename):
    # Dummy implementation for illustration
    try:
        results = []  # Perform BERT multi-language sentiment analysis on segment_list
        df = pd.DataFrame(results)
        df.to_csv(filename, index=False)
        return True
    except Exception as e:
        print(f"Error in BERTMulti sentiment analysis: {e}")
        return False

def get_sentiment_ollama_safe(segment_list, lang, directory_out, filename):
    # Dummy implementation for illustration
    try:
        results = []  # Perform Ollama sentiment analysis on segment_list
        df = pd.DataFrame(results)
        df.to_csv(filename, index=False)
        return True
    except Exception as e:
        print(f"Error in Ollama sentiment analysis: {e}")
        return False


In [ ]:
import os
import pandas as pd
from google.colab import files

def get_sentiment_all(dictionary_of_lists, directory_out):
    # Ensure the output directory exists
    if not os.path.exists(directory_out):
        os.makedirs(directory_out)

    clean_text_sentiments_dict = {}

    for key, segment_list in dictionary_of_lists.items():
        print(f"Processing: {key} with len(segment_list) = {len(segment_list)}")

        success = True
        vader_df = textblob_df = bertmulti_df = ollama_df = None

        # Extract the base name of the book
        book_name = key.replace('_verified.txt', '')

        # Define intermediate file paths
        vader_path = os.path.join(directory_out, f'{book_name}_vader_partial.csv')
        textblob_path = os.path.join(directory_out, f'{book_name}_textblob_partial.csv')
        bertmulti_path = os.path.join(directory_out, f'{book_name}_bertmulti_partial.csv')
        ollama_path = os.path.join(directory_out, f'{book_name}_ollama_partial.csv')

        # Process VADER sentiments
        if os.path.exists(vader_path):
            vader_df = pd.read_csv(vader_path)
            print(f"Loaded VADER results from {vader_path}")
        else:
            if not get_sentiment_vader_safe(segment_list, 'en', directory_out, vader_path):
                print(f"Failed to process VADER for {key}")
                success = False
            else:
                if os.path.exists(vader_path):
                    vader_df = pd.read_csv(vader_path)
                    print(f"Saved VADER results to {vader_path}")
                    files.download(vader_path)  # Download the VADER results
                else:
                    print(f"VADER results file not found for {key}")
                    success = False

        # Process TextBlob sentiments
        if os.path.exists(textblob_path):
            textblob_df = pd.read_csv(textblob_path)
            print(f"Loaded TextBlob results from {textblob_path}")
        else:
            if not get_sentiment_textblob_safe(segment_list, 'en', directory_out, textblob_path):
                print(f"Failed to process TextBlob for {key}")
                success = False
            else:
                if os.path.exists(textblob_path):
                    textblob_df = pd.read_csv(textblob_path)
                    print(f"Saved TextBlob results to {textblob_path}")
                    files.download(textblob_path)  # Download the TextBlob results
                else:
                    print(f"TextBlob results file not found for {key}")
                    success = False

        # Process BERT multi-language sentiments
        if os.path.exists(bertmulti_path):
            bertmulti_df = pd.read_csv(bertmulti_path)
            print(f"Loaded BERTMulti results from {bertmulti_path}")
        else:
            if not get_sentiment_bertmulti_safe(segment_list, 'en', directory_out, bertmulti_path):
                print(f"Failed to process BERTMulti for {key}")
                success = False
            else:
                if os.path.exists(bertmulti_path):
                    bertmulti_df = pd.read_csv(bertmulti_path)
                    print(f"Saved BERTMulti results to {bertmulti_path}")
                    files.download(bertmulti_path)  # Download the BERTMulti results
                else:
                    print(f"BERTMulti results file not found for {key}")
                    success = False

        # Process Ollama sentiments
        if os.path.exists(ollama_path):
            ollama_df = pd.read_csv(ollama_path)
            print(f"Loaded Ollama results from {ollama_path}")
        else:
            if not get_sentiment_ollama_safe(segment_list, 'en', directory_out, ollama_path):
                print(f"Failed to process Ollama for {key}")
                success = False
            else:
                if os.path.exists(ollama_path):
                    ollama_df = pd.read_csv(ollama_path)
                    print(f"Saved Ollama results to {ollama_path}")
                    files.download(ollama_path)  # Download the Ollama results
                else:
                    print(f"Ollama results file not found for {key}")
                    success = False

        if success:
            # Combine results
            final_df = vader_df
            final_df['textblob'] = textblob_df['textblob']
            final_df['bertmulti'] = bertmulti_df['bertmulti']
            final_df['mistral'] = ollama_df['mistral']

            clean_text_sentiments_dict[key] = final_df

            # Save the final combined dataframe
            final_output_path = os.path.join(directory_out, f'{book_name}_segments.csv')
            final_df.to_csv(final_output_path, index=False)
            print(f"Saved combined results to {final_output_path}")
            files.download(final_output_path)  # Download the final combined results
        else:
            print(f"Skipping final combination for {key} due to previous failures.")

    return clean_text_sentiments_dict


In [ ]:
import os
import pandas as pd
from google.colab import files

def is_file_non_empty(filepath):
    """Check if the file is non-empty."""
    return os.path.exists(filepath) and os.stat(filepath).st_size > 0

def get_sentiment_all(dictionary_of_lists, directory_out):
    # Ensure the output directory exists
    if not os.path.exists(directory_out):
        os.makedirs(directory_out)

    clean_text_sentiments_dict = {}

    for key, segment_list in dictionary_of_lists.items():
        print(f"Processing: {key} with len(segment_list) = {len(segment_list)}")

        success = True
        vader_df = textblob_df = bertmulti_df = ollama_df = None

        # Extract the base name of the book
        book_name = key.replace('_verified.txt', '')

        # Define intermediate file paths
        vader_path = os.path.join(directory_out, f'{book_name}_vader_partial.csv')
        textblob_path = os.path.join(directory_out, f'{book_name}_textblob_partial.csv')
        bertmulti_path = os.path.join(directory_out, f'{book_name}_bertmulti_partial.csv')
        ollama_path = os.path.join(directory_out, f'{book_name}_ollama_partial.csv')

        # Check if VADER results exist and are non-empty
        if is_file_non_empty(vader_path):
            print(f"VADER results already processed for {key}")
        else:
            if not get_sentiment_vader_safe(segment_list, 'en', directory_out, vader_path):
                print(f"Failed to process VADER for {key}")
                success = False
            else:
                print(f"Saved VADER results to {vader_path}")
                files.download(vader_path)  # Download the VADER results

        # Check if TextBlob results exist and are non-empty
        if is_file_non_empty(textblob_path):
            print(f"TextBlob results already processed for {key}")
        else:
            if not get_sentiment_textblob_safe(segment_list, 'en', directory_out, textblob_path):
                print(f"Failed to process TextBlob for {key}")
                success = False
            else:
                print(f"Saved TextBlob results to {textblob_path}")
                files.download(textblob_path)  # Download the TextBlob results

        # Check if BERTMulti results exist and are non-empty
        if is_file_non_empty(bertmulti_path):
            print(f"BERTMulti results already processed for {key}")
        else:
            if not get_sentiment_bertmulti_safe(segment_list, 'en', directory_out, bertmulti_path):
                print(f"Failed to process BERTMulti for {key}")
                success = False
            else:
                print(f"Saved BERTMulti results to {bertmulti_path}")
                files.download(bertmulti_path)  # Download the BERTMulti results

        # Check if Ollama results exist and are non-empty
        if is_file_non_empty(ollama_path):
            print(f"Ollama results already processed for {key}")
        else:
            if not get_sentiment_ollama_safe(segment_list, 'en', directory_out, ollama_path):
                print(f"Failed to process Ollama for {key}")
                success = False
            else:
                print(f"Saved Ollama results to {ollama_path}")
                files.download(ollama_path)  # Download the Ollama results

        if success:
            # Combine results if all models are successfully processed
            if all([is_file_non_empty(vader_path), is_file_non_empty(textblob_path), is_file_non_empty(bertmulti_path), is_file_non_empty(ollama_path)]):
                vader_df = pd.read_csv(vader_path)
                textblob_df = pd.read_csv(textblob_path)
                bertmulti_df = pd.read_csv(bertmulti_path)
                ollama_df = pd.read_csv(ollama_path)

                # Combine results
                final_df = vader_df
                final_df['textblob'] = textblob_df['textblob']
                final_df['bertmulti'] = bertmulti_df['bertmulti']
                final_df['mistral'] = ollama_df['mistral']

                clean_text_sentiments_dict[key] = final_df

                # Save the final combined dataframe
                final_output_path = os.path.join(directory_out, f'{book_name}_segments.csv')
                final_df.to_csv(final_output_path, index=False)
                print(f"Saved combined results to {final_output_path}")
                files.download(final_output_path)  # Download the final combined results
            else:
                print(f"Skipping final combination for {key} due to incomplete sentiment analysis results.")
        else:
            print(f"Skipping final combination for {key} due to previous failures.")

    return clean_text_sentiments_dict


In [ ]:
def test_ollama(prompt_str):
    import ollama

    response = ollama.chat(prompt_str)
    return response.text

res_str = test_ollama("Hello, how are you?")
print(res_str)

In [ ]:
!ls -altr sentiments

In [ ]:
!rm ./sentiments/*_partial.csv

In [ ]:
%%time

# NOTE: 2h05m?

sentiments_all_dict = get_sentiment_all(clean_text_reformat_seg_filter_dict, 'sentiments')

In [ ]:
# sentiments_test_dict = get_sentiment_all(clean_test_reformat_seg_filter_test_dict, 'test_sentiments')


In [ ]:
%cd ..

In [ ]:
!pwd


In [ ]:
!rmdir sentiments

In [ ]:
!ls

In [ ]:
!cat textblob_partial.csv | wc -l

In [ ]:
!rm *

### Save Results

In [ ]:
def write_dict_of_df_to_files(directory_out, dict_of_df, segment_type='segments'):
    """
    Saves each DataFrame from the dictionary to separate files in the specified directory.

    Parameters:
    directory_out (str): The directory where the files will be saved.
    dict_of_df (dict): Dictionary where keys are filenames and values are pandas DataFrames.
    segment_type (str): The suffix to be added to the output filenames.

    Returns:
    dict: A dictionary with filenames as keys and their paths as values.
    """
    # Ensure the output directory exists
    if not os.path.exists(directory_out):
        os.makedirs(directory_out)

    file_paths = {}

    for key in dict_of_df.keys():
        # Create the output filename
        filename_out = key.replace('_verified.txt', f'_{segment_type}.csv')
        output_file_path = os.path.join(directory_out, filename_out)

        # Debug: Print the current filename and output path
        print(f"Saving file: {output_file_path}")

        # Enhanced Debug: Print the shape of the DataFrame
        print(f"DataFrame shape for {key}: {dict_of_df[key].shape}")

        # Write the DataFrame to a CSV file
        dict_of_df[key].to_csv(output_file_path, index=False)

        # Store the filename and its path in the dictionary
        file_paths[filename_out] = output_file_path

    return file_paths

"""
# Example usage
# Ensure the dictionary contains valid data
example_data_1 = pd.DataFrame({
    'Column1': ['A', 'B', 'C', 'D'],
    'Column2': [1, 2, 3, 4]
})
example_data_2 = pd.DataFrame({
    'Column1': ['W', 'X', 'Y', 'Z'],
    'Column2': [10, 20, 30, 40]
})
clean_test_reformat_seg_filter_dict_df = {
    'book_proust_fr_swans-way_proust_original_verified.txt': example_data_1,
    'book_proust_en_swans-way_enright_original_verified.txt': example_data_2
}

file_paths_df = write_dict_of_df_to_files('test_df', clean_test_reformat_seg_filter_dict_df, 'test')
print(file_paths_df)

# Verify the content of the files by reading them back
for filename, filepath in file_paths_df.items():
    print(f"Verifying content of {filename}:")
    df = pd.read_csv(filepath)
    print(df.head())
""";

In [ ]:
sentiments_test_dict['book_proust_fr_swans-way_proust_original_verified.txt']

In [ ]:
!ls test_sentiments/*_sentiments.csv

In [ ]:
# Save test files

for book_sentiments_all_file in os.listdir(os.path.join('.', 'test_sentiments')):
  if book_sentiments_all_file.endswith('_sentiments.csv'):
      print(book_sentiments_all_file)
      files.download(os.path.join('.', 'test_sentiments', book_sentiments_all_file))
    # print(f"{book}: {df.shape}")
    # fileout_paths_dict = write_dict_of_df_to_files('test_sentiments', sentiments_test_dict, 'sentiments')

In [ ]:
fileout_paths_dict = write_dict_of_df_to_files('test_sentiments', sentiments_test_dict, 'sentiments')

In [ ]:
fileout_paths_dict

In [ ]:
sentiments_all_dict.keys()

In [ ]:
sentiments_all_dict['book_proust_fr_swans-way_proust_original_verified.txt'][:10]

len(sentiments_all_dict['book_proust_fr_swans-way_proust_original_verified.txt'])

In [ ]:
fileout_paths_dict = save_dict_to_files('sentiments', sentiments_all_dict, 'sentiments')

In [ ]:
fileout_paths_dict

In [ ]:
save_dict_to_files('./segments', sentiments_all_dict)

In [ ]:
files.download('./segments/book_proust_fr_swans-way_proust_original_segments.txt')

### Get Sentiments All At Once

In [ ]:
%whos


In [ ]:
%%time

# 2h04m

clean_text_sentiments_df = create_sentiment_dataframes(clean_text_reformat_seg_filter_dict)
print(clean_text_sentiments_df.keys())

In [ ]:


def save_flat_dict_csv(dict_of_dataframes):
    filenames = []
    for book_title, dataframe in dict_of_dataframes.items():
        # Create a safe filename
        safe_filename = re.sub(r'[^a-zA-Z0-9_\-]', '_', book_title) + '.csv'
        filenames.append(safe_filename)

        try:
            # Write the dataframe to a CSV file
            dataframe.to_csv(safe_filename, index=False)
            print(f"Successfully saved {safe_filename}")
        except Exception as e:
            print(f"Error saving {safe_filename}: {e}")

    return filenames



In [ ]:
filenames = save_flat_dict_csv(clean_text_sentiments_df)

# For Google Colab
for filename in filenames:
    files.download(filename)

In [ ]:
def load_existing_sentiment_dataframe(filepath):
    if os.path.exists(filepath):
        return pd.read_csv(filepath)
    return None

def save_partial_sentiment_dataframe(filepath, df):
    df.to_csv(filepath, index=False)

def create_sentiment_dataframes(clean_text_reformat_seg_filter_dict_in, output_dir="sentiments"):
    clean_text_sentiments_dict = {}

    if not os.path.exists(output_dir):
        print(f" Directory: {output_dir} DNE, will create it")
        os.makedirs(output_dir)

    for key, segment_list in clean_text_reformat_seg_filter_dict_in.items():
        print(f"Processing sentiments for: {key}")
        print(f"PRocessing sentiment_list has {len(segment_list)} strings")

        # Create the output filenames
        filename_out = key.replace('_verified.txt', '_segments.csv')
        partial_vader_file = key.replace('_verified.txt', '_segments_vader_partial.csv')
        partial_textblob_file = key.replace('_verified.txt', '_segments_textblob_partial.csv')
        partial_bertmulti_file = key.replace('_verified.txt', '_segments_bertmulti_partial.csv')
        partial_ollama_file = key.replace('_verified.txt', '_segments_ollama_partial.csv')

        # Define the paths
        full_output_path = os.path.join(".", output_dir, filename_out)
        partial_vader_path = os.path.join(".", output_dir, partial_vader_file)
        partial_textblob_path = os.path.join(".", output_dir, partial_textblob_file)
        partial_bertmulti_path = os.path.join(".", output_dir, partial_bertmulti_file)
        partial_ollama_path = os.path.join(".", output_dir, partial_ollama_file)

        # Load existing dataframe if available
        # df = load_existing_sentiment_dataframe(full_output_path)
        # print(f"df.info(): {df.info()}")

        # if df is None:
        #     df = pd.DataFrame()

        df = pd.DataFrame()
        print(f'{list(df.columns.values)}')

        # Process VADER sentiments if not already done
        if 'vader' not in df.columns:
            print("PROCESSING Sentiment VADER")
            vader_sentiments = get_sentiment_vader(segment_list)
            df['vader'] = vader_sentiments
            print(f"  Saving to partial_vader_path: {partial_vader_path}")
            save_partial_sentiment_dataframe(partial_vader_path, df)

        # Process TextBlob sentiments if not already done
        if 'textblob' not in df.columns:
            print("PROCESSING Sentiment TextBlob")
            textblob_sentiments = get_sentiment_textblob(segment_list, language="en")
            df['textblob'] = textblob_sentiments
            print(f"  Saving to partial_textblob_path: {partial_textblob_path}")
            save_partial_sentiment_dataframe(partial_textblob_path, df)

        # Process BERT multi-language sentiments if not already done
        if 'bertmulti' not in df.columns:
            print("PROCESSING Sentiment BERTMulti")
            bertmulti_sentiments = get_sentiment_bertmulti(segment_list, language="en")
            print(f" bertmulti_sentiments has {len(bertmulti_sentiments)} strings")
            df['bertmulti'] = bertmulti_sentiments
            print(f"df['bertmulti']: {df['bertmulti']}")
            print(f"  Saving to partial_xxx_path: {partial_bertmulti_path}")
            save_partial_sentiment_dataframe(partial_bertmulti_path, df)

        # Process Ollama sentiments if not already done
        if 'mistral' not in df.columns:
            print("PROCESSING Sentiment Mistral")
            ollama_sentiments = get_sentiment_ollama(segment_list, language="en", ollama_model=MODEL_OLLAMA)
            df['mistral'] = ollama_sentiments
            print(f"  Saving to partial_xxx_path: {partial_ollama_path}")
            save_partial_sentiment_dataframe(partial_ollama_path, df)

        # Save the complete dataframe
        df.to_csv(full_output_path, index=False)
        clean_text_sentiments_dict[key] = df

    return clean_text_sentiments_dict

In [ ]:
clean_text_reformat_seg_filter_dict.keys()

In [ ]:
for book_now, lines_list in clean_text_reformat_seg_filter_dict.items():

    print(f"{book_now} : len={len(clean_text_reformat_seg_filter_dict[book_now])}")

In [ ]:
len(segments_list)

In [ ]:
%%time

# test

clean_test_sentiments_dict = create_sentiment_dataframes(clean_test_reformat_seg_filter_test_dict)
print(clean_test_sentiments_dict.keys())

In [ ]:
clean_test_sentiments_dict.keys()

In [ ]:
clean_test_sentiments_dict['book_proust_fr_swans-way_proust_original_verified.txt']

In [ ]:
clean_test_reformat_seg_filter_test_dict

clean_test_sentiments_df

In [ ]:
%%time

# 2h04m

clean_text_sentiments_df = create_sentiment_dataframes(clean_text_reformat_seg_filter_dict)
print(clean_text_sentiments_df.keys())

In [ ]:
 clean_text_sentiments_dict['book_proust_en_swans-way_moncrieff_original_segments.txt']

In [ ]:
clean_text_sentiments_dict['book_proust_en_swans-way_enright_original_segments.txt']

In [ ]:
clean_text_sentiments_dict['book_proust_fr_swans-way_proust_original_segments.txt']

In [ ]:
clean_text_sentiments_dict.keys()

In [ ]:
clean_text_sentiments_dict

In [ ]:
clean_text_reformat_seg_filter_dict.keys()

In [ ]:
len(clean_text_reformat_seg_filter_dict['book_proust_en_swans-way_moncrieff_original_segments.txt'])

In [ ]:
%%time

clean_text_sentiments_df = create_sentiment_dataframes(clean_text_reformat_seg_filter_dict, segments_list)
print(clean_text_sentiments_df.keys())

In [ ]:
# Example usage
sentence_test_both_list = sentence_test_en_list + sentence_test_fr_list
clean_text_test_dict = {
    'book_proust_en_swans-way_test_original_verified.txt': sentence_test_both_list
    # Add other keys and segment lists as needed
}

clean_test_sentiments_dict = create_sentiment_dataframes(clean_text_test_dict)
print(type(clean_test_sentiments_dict))

In [ ]:
clean_test_sentiments_dict.keys()

In [ ]:

len(clean_text_reformat_seg_filter_dict["book_proust_en_swans-way_moncrieff_original_verified.txt"])

clean_test_sentiments_dict['book_proust_en_swans-way_test_original_verified.txt']

In [ ]:
# GET clean_text_sentiments_dict

clean_text_sentiments_dict = create_sentiment_dataframes(clean_text_reformat_seg_filter_dict)

In [ ]:
save_dict_to_files(clean_text_sentiments_dict)

In [ ]:
save_dict_to_files('./sentiments', clean_text_sentiments_dict)

# [END]

# II. Compute Sentiments

In [ ]:
# 20240525

# Upload combo files:
# Saving book_proust_en_swans-way_davis_sentence_sentiment_combined.csv to book_proust_en_swans-way_davis_sentence_sentiment_combined.csv

uploaded_sentiment = files.upload()

In [ ]:
# Get the filename from the uploaded files
upload_filename = list(uploaded_sentiment.keys())[0]
print(f"upload_filename: {upload_filename}")

# Extract the book title from the filename
book_title = "_".join(upload_filename.split(".")[0].split("_")[1:5])
print(f"book_title: {book_title}")

In [ ]:
sentiment_combined_files_list = [
    "book_proust_en_swans-way_davis_sentence_sentiment_combined.csv",
    "book_proust_en_swans-way_enright_sentence_sentiment_combined.csv",
    "book_proust_en_swans-way_moncrieff_sentence_sentiment_combined.csv",
    "book_proust_fr_swans-way_proust_sentence_sentiment_combined.csv"
]

sentiment_combined_file = sentiment_combined_files_list[0]

sentiment_df = pd.read_csv(upload_filename)

sentiment_df.head()

In [ ]:
sentiment_df.describe()


In [ ]:
model_ls = sentiment_df.columns.tolist()
model_ls

In [ ]:

# "Swan's Way by Proust en_davis"
filename_subwords_list = sentiment_combined_file.split('_')
novel_title = '_'.join([filename_subwords_list[1],filename_subwords_list[3],filename_subwords_list[2],filename_subwords_list[4]])
print(novel_title)

In [ ]:
def read_combine_csv_to_df(list_file_str):
    # Initialize an empty list to hold data
    combined_data = []

    # Iterate over each file in the list
    for file in list_file_str:

        # Read the CSV file into a dataframe
        df = pd.read_csv(file)

        # Add a column for the text title
        text_title = os.path.basename(file).split('_')[3]
        df['title'] = text_title

        # Add a column for the text translator
        text_translator = os.path.basename(file).split('_')[4].split('.')[0]
        df['translator'] = text_translator

        # Add a column for the text language
        text_language = os.path.basename(file).split('_')[2]
        df['language'] = text_language

        # Add a column for the segment number (assuming the CSV files are already in order)
        df['segment_no'] = df.index

        # Append the dataframe to the combined_data list
        combined_data.append(df)

    # Concatenate all dataframes in the list into a single dataframe
    combined_df = pd.concat(combined_data, ignore_index=True)

    # Reorder the columns to have title, translator, language, segment_no first
    cols = ['title', 'translator', 'language', 'segment_no'] + [col for col in combined_df.columns if col not in ['title', 'translator', 'language', 'segment_no']]
    combined_df = combined_df[cols]

    return combined_df

In [ ]:
!ls

In [ ]:
# Example usage:
list_file_str = [
    "book_proust_en_swans-way_davis_sentence_sentiment_combined.csv",
    "book_proust_en_swans-way_enright_sentence_sentiment_combined.csv",
    "book_proust_en_swans-way_moncrieff_sentence_sentiment_combined.csv",
    "book_proust_fr_swans-way_proust_sentence_sentiment_combined.csv"
]

# Call the function to combine the CSV files
combined_df = read_combine_csv_to_df(list_file_str)

# Display the combined dataframe
print(combined_df.head())

In [ ]:
combined_df.to_csv('book_proust_all4translations_all4models_sentiments.csv')

In [ ]:
!ls

In [ ]:
files.download('book_proust_all4translations_all4models_sentiments.csv')

In [ ]:
combined_df[combined_df['translator'] == translator_now]].describe()

In [ ]:
def sentiment_summary_stats_by_author(df):
    # Get the list of unique translators
    translators = df['translator'].unique()

    # Loop over each unique translator
    for translator in translators:
        # Create a temporary dataframe for the current translator
        temp_by_translator_df = df[df['translator'] == translator]

        # Generate descriptive statistics for the temporary dataframe
        summary_stats = temp_by_translator_df.describe()

        # Print the descriptive statistics to the screen
        print(f"\n\n\nSummary statistics for translator: {translator}")
        print(summary_stats)

        # Write the descriptive statistics to an external CSV file
        summary_stats.to_csv(f"proust_swans-way_{translator}_sumstats_sentiment.csv")


In [ ]:
# Assuming combined_df has been generated using the previous function
combined_df = read_combine_csv_to_df(list_file_str)

# Call the function to generate and save the summary statistics
sentiment_summary_stats_by_author(combined_df)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import zscore

def plot_kde_by_translator(df):
    # Filter the dataframe to include only the sentiment columns and the translator column
    sentiment_columns = ['sentiment-bertmulti', 'sentiment-mistral7b', 'sentiment-textblob', 'sentiment-vader']
    filtered_df = df[['translator'] + sentiment_columns]

    # Get the list of unique translators
    translators = filtered_df['translator'].unique()

    # Set a seaborn style for the plots
    sns.set(style="whitegrid")

    # Loop over each unique translator to create individual KDE plots
    for translator in translators:
        plt.figure(figsize=(14, 7), dpi=300)
        temp_by_translator_df = filtered_df[filtered_df['translator'] == translator]
        for sentiment in sentiment_columns:
            sns.kdeplot(temp_by_translator_df[sentiment], label=sentiment)
        plt.title(f"KDE Plot of Sentiment Scores for {translator.capitalize()}", fontsize=18)
        plt.xlabel("Sentiment Score", fontsize=14)
        plt.ylabel("Density", fontsize=14)
        plt.legend(title="Sentiment Model", fontsize=12, bbox_to_anchor=(1.05, 1), loc='upper left')
        plt.grid(axis='y', linestyle='--', alpha=0.7)
        plt.tight_layout()
        plt.savefig(f"kde_sentiment_scores_{translator}.png", format='png', dpi=300, bbox_inches='tight')
        plt.show()

    # Combined KDE plot for all translators (raw sentiments)
    plt.figure(figsize=(14, 7), dpi=300)
    for sentiment in sentiment_columns:
        sns.kdeplot(filtered_df[sentiment], label=sentiment)
    plt.title("Combined KDE Plot of Sentiment Scores for All Translators", fontsize=18)
    plt.xlabel("Sentiment Score", fontsize=14)
    plt.ylabel("Density", fontsize=14)
    plt.legend(title="Sentiment Model", fontsize=12, bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    plt.tight_layout()
    plt.savefig("kde_sentiment_scores_all_translators.png", format='png', dpi=300, bbox_inches='tight')
    plt.show()

    # Normalize the sentiment columns using Z-score normalization
    normalized_df = filtered_df.copy()
    normalized_df[sentiment_columns] = normalized_df[sentiment_columns].apply(zscore)

    # Combined KDE plot for all translators (normalized sentiments)
    plt.figure(figsize=(14, 7), dpi=300)
    for sentiment in sentiment_columns:
        sns.kdeplot(normalized_df[sentiment], label=sentiment)
    plt.title("Combined KDE Plot of Normalized Sentiment Scores for All Translators", fontsize=18)
    plt.xlabel("Normalized Sentiment Score", fontsize=14)
    plt.ylabel("Density", fontsize=14)
    plt.legend(title="Sentiment Model", fontsize=12, bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    plt.tight_layout()
    plt.savefig("kde_normalized_sentiment_scores_all_translators.png", format='png', dpi=300, bbox_inches='tight')
    plt.show()

# Example usage:
list_file_str = [
    "book_proust_en_swans-way_davis_sentence_sentiment_combined.csv",
    "book_proust_en_swans-way_enright_sentence_sentiment_combined.csv",
    "book_proust_en_swans-way_moncrieff_sentence_sentiment_combined.csv",
    "book_proust_fr_swans-way_proust_sentence_sentiment_combined.csv"
]

# Assuming combined_df has been generated using the previous function
combined_df = read_combine_csv_to_df(list_file_str)

# Call the function to plot KDEs
plot_kde_by_translator(combined_df)


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

def plot_sumstats_by_translator(df):
    # Filter the dataframe to include only the sentiment columns and the translator column
    sentiment_columns = ['sentiment-bertmulti', 'sentiment-mistral7b', 'sentiment-textblob', 'sentiment-vader']
    filtered_df = df[['translator'] + sentiment_columns]

    # Get the list of unique translators
    translators = filtered_df['translator'].unique()

    # Initialize dictionaries to hold summary statistics
    mean_stats = {}
    std_stats = {}

    # Loop over each unique translator to compute summary statistics
    for translator in translators:
        # Create a temporary dataframe for the current translator
        temp_by_translator_df = filtered_df[filtered_df['translator'] == translator]

        # Generate descriptive statistics for the temporary dataframe
        summary_stats = temp_by_translator_df.describe()

        # Store the mean and std statistics
        mean_stats[translator] = summary_stats.loc['mean']
        std_stats[translator] = summary_stats.loc['std']

    # Convert the dictionaries to dataframes for easier plotting
    mean_df = pd.DataFrame(mean_stats).transpose()[sentiment_columns]
    std_df = pd.DataFrame(std_stats).transpose()[sentiment_columns]

    # Set a seaborn style for the plots
    sns.set(style="whitegrid")

    # Plot mean statistics
    plt.figure(figsize=(14, 7), dpi=300)
    ax = mean_df.plot(kind='bar', figsize=(14, 7), colormap='viridis', edgecolor='black', linewidth=1.2)
    ax.set_title("Mean Sentiment Scores by Translator", fontsize=18)
    ax.set_xlabel("Translator", fontsize=14)
    ax.set_ylabel("Mean Score", fontsize=14)
    ax.tick_params(axis='x', labelrotation=45, labelsize=12)
    ax.tick_params(axis='y', labelsize=12)
    # Move the legend outside the plot
    ax.legend(title="Sentiment Model", fontsize=12, bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    plt.tight_layout()

    # Save the plot as a high-resolution image
    plt.savefig("mean_sentiment_scores_by_translator.png", format='png', dpi=300, bbox_inches='tight')
    plt.show()

    # Plot std deviation statistics
    plt.figure(figsize=(14, 7), dpi=300)
    ax = std_df.plot(kind='bar', figsize=(14, 7), colormap='viridis', edgecolor='black', linewidth=1.2)
    ax.set_title("Standard Deviation of Sentiment Scores by Translator", fontsize=18)
    ax.set_xlabel("Translator", fontsize=14)
    ax.set_ylabel("Standard Deviation", fontsize=14)
    ax.tick_params(axis='x', labelrotation=45, labelsize=12)
    ax.tick_params(axis='y', labelsize=12)
    # Move the legend outside the plot
    ax.legend(title="Sentiment Model", fontsize=12, bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    plt.tight_layout()

    # Save the plot as a high-resolution image
    plt.savefig("std_sentiment_scores_by_translator.png", format='png', dpi=300, bbox_inches='tight')
    plt.show()

# Example usage:
list_file_str = [
    "book_proust_en_swans-way_davis_sentence_sentiment_combined.csv",
    "book_proust_en_swans-way_enright_sentence_sentiment_combined.csv",
    "book_proust_en_swans-way_moncrieff_sentence_sentiment_combined.csv",
    "book_proust_fr_swans-way_proust_sentence_sentiment_combined.csv"
]

# Assuming combined_df has been generated using the previous function
combined_df = read_combine_csv_to_df(list_file_str)

# Call the function to plot the summary statistics
plot_sumstats_by_translator(combined_df)


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import zscore

def plot_sumstats_by_translator(df, zscore_flag=False):
    # Filter the dataframe to include only the sentiment columns and the translator column
    sentiment_columns = ['sentiment-bertmulti', 'sentiment-mistral7b', 'sentiment-textblob', 'sentiment-vader']
    filtered_df = df[['translator'] + sentiment_columns]

    # Apply Z-score normalization if zscore_flag is True
    if zscore_flag:
        filtered_df[sentiment_columns] = filtered_df[sentiment_columns].apply(zscore)

    # Get the list of unique translators
    translators = filtered_df['translator'].unique()

    # Initialize dictionaries to hold summary statistics
    mean_stats = {}
    std_stats = {}

    # Loop over each unique translator to compute summary statistics
    for translator in translators:
        # Create a temporary dataframe for the current translator
        temp_by_translator_df = filtered_df[filtered_df['translator'] == translator]

        # Generate descriptive statistics for the temporary dataframe
        summary_stats = temp_by_translator_df.describe()

        # Store the mean and std statistics
        mean_stats[translator] = summary_stats.loc['mean']
        std_stats[translator] = summary_stats.loc['std']

    # Convert the dictionaries to dataframes for easier plotting
    mean_df = pd.DataFrame(mean_stats).transpose()[sentiment_columns]
    std_df = pd.DataFrame(std_stats).transpose()[sentiment_columns]

    # Set a seaborn style for the plots
    sns.set(style="whitegrid")

    # Plot mean statistics
    plt.figure(figsize=(14, 7), dpi=300)
    ax = mean_df.plot(kind='bar', figsize=(14, 7), colormap='viridis', edgecolor='black', linewidth=1.2)
    ax.set_title("Mean Sentiment Scores by Translator" + (" (Z-score Normalized)" if zscore_flag else ""), fontsize=18)
    ax.set_xlabel("Translator", fontsize=14)
    ax.set_ylabel("Mean Score", fontsize=14)
    ax.tick_params(axis='x', labelrotation=45, labelsize=12)
    ax.tick_params(axis='y', labelsize=12)
    # Move the legend outside the plot
    ax.legend(title="Sentiment Model", fontsize=12, bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    plt.tight_layout()

    # Save the plot as a high-resolution image
    plt.savefig("mean_sentiment_scores_by_translator" + ("_zscore" if zscore_flag else "") + ".png", format='png', dpi=300, bbox_inches='tight')
    plt.show()

    # Plot std deviation statistics
    plt.figure(figsize=(14, 7), dpi=300)
    ax = std_df.plot(kind='bar', figsize=(14, 7), colormap='viridis', edgecolor='black', linewidth=1.2)
    ax.set_title("Standard Deviation of Sentiment Scores by Translator" + (" (Z-score Normalized)" if zscore_flag else ""), fontsize=18)
    ax.set_xlabel("Translator", fontsize=14)
    ax.set_ylabel("Standard Deviation", fontsize=14)
    ax.tick_params(axis='x', labelrotation=45, labelsize=12)
    ax.tick_params(axis='y', labelsize=12)
    # Move the legend outside the plot
    ax.legend(title="Sentiment Model", fontsize=12, bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    plt.tight_layout()

    # Save the plot as a high-resolution image
    plt.savefig("std_sentiment_scores_by_translator" + ("_zscore" if zscore_flag else "") + ".png", format='png', dpi=300, bbox_inches='tight')
    plt.show()

# Example usage:
list_file_str = [
    "book_proust_en_swans-way_davis_sentence_sentiment_combined.csv",
    "book_proust_en_swans-way_enright_sentence_sentiment_combined.csv",
    "book_proust_en_swans-way_moncrieff_sentence_sentiment_combined.csv",
    "book_proust_fr_swans-way_proust_sentence_sentiment_combined.csv"
]

# Assuming combined_df has been generated using the previous function
combined_df = read_combine_csv_to_df(list_file_str)

# Call the function to plot the summary statistics without Z-score normalization
plot_sumstats_by_translator(combined_df, zscore_flag=False)

# Call the function to plot the summary statistics with Z-score normalization
plot_sumstats_by_translator(combined_df, zscore_flag=True)


**[SKIP] to Section III: Get Sentiments**

# III. Load Sentiments (sentiments_all_dict)

In [ ]:
# 20240525

# Upload Combined Files: (both text and all raw model sentiment values)
# "book_proust_en_swans-way_moncrieff_original_segments.csv"

# Saving book_proust_en_swans-way_davis_original_segments.csv to book_proust_en_swans-way_davis_original_segments.csv
# Saving book_proust_en_swans-way_enright_original_segments.csv to book_proust_en_swans-way_enright_original_segments.csv
# Saving book_proust_fr_swans-way_proust_original_segments.csv to book_proust_fr_swans-way_proust_original_segments.csv

uploaded = files.upload()

## From Cloud RunPod.io

In [ ]:
!ls *_segments.csv

In [ ]:
# GLOBAL VAR: filename sentiment_all_list
# list of all input filenames for computed sentiment values

filename_sentiment_all_list = [file for file in glob.glob("*_segments.csv")]
print(filename_sentiment_all_list)

In [ ]:
# GLOBAL VAR: sentiments_all_dict[filename_now]
# 1-level dictionary[filename] = text/sentiment values]

sentiments_all_dict = {}

for filename_now in filename_sentiment_all_list:
  print(f"PROCESSING: {filename_now}")
  df = pd.read_csv(filename_now)
  sentiments_all_dict[filename_now] = df
  print(df.info())

In [ ]:
# CHECK for dictionary keys/filenames

sentiments_all_dict.keys()

In [ ]:
# CHECK for complete and well-formed text, models, and values

sentiments_all_dict['book_proust_fr_swans-way_proust_original_segments.csv'][:10]
sentiments_all_dict['book_proust_fr_swans-way_proust_original_segments.csv'].info()

In [ ]:
# CHECK lengths of each text

for key, value in sentiments_all_dict.items():
  print(f"{key}\n\n")

for filename_now, sentiment_df in sentiments_all_dict.items():
    print(f"Processing: {filename_now} with len(sentiment_df) = {len(sentiment_df)}")

**[SKIP]**

## From Local VSCode

In [ ]:
sentiment_combined_files_list = [
    "book_proust_en_swans-way_davis_sentence_sentiment_combined.csv",
    "book_proust_en_swans-way_enright_sentence_sentiment_combined.csv",
    "book_proust_en_swans-way_moncrieff_sentence_sentiment_combined.csv",
    "book_proust_fr_swans-way_proust_sentence_sentiment_combined.csv"
]

sentiment_combined_file = sentiment_combined_files_list[0]
sentiment_df = pd.read_csv(sentiment_combined_file)

sentiment_df.head()
sentiment_df.tail()

In [ ]:
model_input_ls = sentiment_df.columns.tolist()
model_input_ls

In [ ]:

# "Swan's Way by Proust en_davis"
filename_subwords_list = sentiment_combined_file.split('_')
novel_title = '_'.join([filename_subwords_list[1],filename_subwords_list[3],filename_subwords_list[2],filename_subwords_list[4]])
print(novel_title)

# NOTES 2:

### START NEW CODE

In [ ]:
sentiments_all_equallen_dict.keys()

In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import zscore
import plotly.express as px
import seaborn as sns
import matplotlib.pyplot as plt
from statsmodels.nonparametric.smoothers_lowess import lowess

# Example dictionary of dataframes (using mock data for demonstration)
sentiments_all_dict = {
    "book_proust_fr_swans-way_proust_original_segments.csv": pd.DataFrame({
        'text': ["Parfois, a peine ma bougie eteinte, mes yeux se fermaient immédiatement " + str(i) for i in range(4514)],
        'vader': np.random.randn(4514),
        'textblob': np.random.randn(4514),
        'bertmulti': np.random.randn(4514),
        'mistral': np.random.randn(4514)
    }),
    "book_proust_en_swans-way_moncrieff_original_segments.csv": pd.DataFrame({
        'text': ["Sometimes, when I had put out my candle, my eyes would close so quickly that I had not even time to say 'I'm going to sleep.'" + str(i) for i in range(4287)],
        'vader': np.random.randn(4287),
        'textblob': np.random.randn(4287),
        'bertmulti': np.random.randn(4287),
        'mistral': np.random.randn(4287)
    }),
    "book_proust_en_swans-way_davis_original_segments.csv": pd.DataFrame({
        'text': ["The Way by Swann's, for a long time I used to go to bed early." + str(i) for i in range(4321)],
        'vader': np.random.randn(4321),
        'textblob': np.random.randn(4321),
        'bertmulti': np.random.randn(4321),
        'mistral': np.random.randn(4321)
    }),
    "book_proust_en_swans-way_enright_original_segments.csv": pd.DataFrame({
        'text': ["For a long time I would go to bed early, sometimes the candle barely out, my eyes would close so quickly that I had not even time to say I'm going to sleep." + str(i) for i in range(4449)],
        'vader': np.random.randn(4449),
        'textblob': np.random.randn(4449),
        'bertmulti': np.random.randn(4449),
        'mistral': np.random.randn(4449)
    }),
}

WIN_PER = 10  # Percentage for SMA smoothing

# 1. Dictionaries for clean output names
output_map_titles_dict = {
    "book_proust_fr_swans-way_proust_original_segments.csv": "Swan's Way (original Proust in French)",
    "book_proust_en_swans-way_moncrieff_original_segments.csv": "Swan's Way (trans. Moncrieff in English)",
    "book_proust_en_swans-way_davis_original_segments.csv": "Swan's Way (trans. Davis in English)",
    "book_proust_en_swans-way_enright_original_segments.csv": "Swan's Way (trans. Enright in English)"
}

output_map_legend_dict = {
    "book_proust_fr_swans-way_proust_original_segments.csv": "Proust Original, French",
    "book_proust_en_swans-way_moncrieff_original_segments.csv": "Moncrieff Translation, English",
    "book_proust_en_swans-way_davis_original_segments.csv": "Davis Translation, English",
    "book_proust_en_swans-way_enright_original_segments.csv": "Enright Translation, English"
}

# 2. Aggregate the longer time series
def aggregate_short_text(df, target_length):
    while len(df) > target_length:
        # Find the shortest text fragment
        df['text_length'] = df['text'].apply(len)
        shortest_index = df['text_length'].idxmin()

        # Identify the shortest neighbor (previous or next)
        if shortest_index == 0:
            neighbor_index = shortest_index + 1
        elif shortest_index == len(df) - 1:
            neighbor_index = shortest_index - 1
        else:
            neighbor_index = shortest_index + 1 if df['text_length'][shortest_index + 1] < df['text_length'][shortest_index - 1] else shortest_index - 1

        # Combine the shortest text fragment with its neighbor
        df.at[neighbor_index, 'text'] = df.at[neighbor_index, 'text'] + " " + df.at[shortest_index, 'text']

        # Average the sentiment values
        for sentiment in ['vader', 'textblob', 'bertmulti', 'mistral']:
            df.at[neighbor_index, sentiment] = (df.at[neighbor_index, sentiment] + df.at[shortest_index, sentiment]) / 2

        # Drop the shortest text fragment row
        df = df.drop(shortest_index).reset_index(drop=True)

    # Drop the temporary 'text_length' column
    df = df.drop(columns=['text_length'])

    return df

# Find the minimum length
min_length = min(len(df) for df in sentiments_all_dict.values())

# Create a new dictionary for equal length timeseries (using copies)
sentiments_all_equallen_dict = {}
for key, df in sentiments_all_dict.items():
    df_copy = df.copy()
    if len(df_copy) > min_length:
        df_copy = aggregate_short_text(df_copy, min_length)
    sentiments_all_equallen_dict[key] = df_copy

# Z-score normalize the sentiment values
for key, df in sentiments_all_equallen_dict.items():
    sentiments_all_equallen_dict[key][['vader', 'textblob', 'bertmulti', 'mistral']] = df[['vader', 'textblob', 'bertmulti', 'mistral']].apply(zscore)

# Apply SMA smoothing
def apply_smoothing(df, window_percentage):
    window_length = max(int(len(df) * window_percentage / 100), 1)
    for sentiment in ['vader', 'textblob', 'bertmulti', 'mistral']:
        df[sentiment] = df[sentiment].rolling(window=window_length, min_periods=1, center=True).mean()
    return df

if WIN_PER > 0:
    for key in sentiments_all_equallen_dict.keys():
        sentiments_all_equallen_dict[key] = apply_smoothing(sentiments_all_equallen_dict[key], WIN_PER)

# Combine dataframes and prepare for plotting
combined_df = pd.concat(sentiments_all_equallen_dict, axis=0)
combined_df.reset_index(level=0, inplace=True)
combined_df.rename(columns={'level_0': 'source'}, inplace=True)
combined_df['sentence_no'] = combined_df.groupby('source').cumcount() + 1

# Map the clean names to the combined dataframe
combined_df['source_clean'] = combined_df['source'].map(output_map_legend_dict)

# Create a column for shortened text for tooltips
combined_df['short_text'] = combined_df['text'].apply(lambda x: x[:25] + '...')

# Create columns for each text version
combined_df['text_proust'] = combined_df[combined_df['source'] == 'book_proust_fr_swans-way_proust_original_segments.csv']['text']
combined_df['text_moncrieff'] = combined_df[combined_df['source'] == 'book_proust_en_swans-way_moncrieff_original_segments.csv']['text']
combined_df['text_davis'] = combined_df[combined_df['source'] == 'book_proust_en_swans-way_davis_original_segments.csv']['text']
combined_df['text_enright'] = combined_df[combined_df['source'] == 'book_proust_en_swans-way_enright_original_segments.csv']['text']

# Merge the text columns for hover data
hover_data = combined_df[['source_clean', 'short_text', 'text_proust', 'text_moncrieff', 'text_davis', 'text_enright']].fillna('')
hover_data['hover_text'] = hover_data.apply(lambda row: f"Proust: {row['text_proust'][:25]}...<br>Moncrieff: {row['text_moncrieff'][:25]}...<br>Davis: {row['text_davis'][:25]}...<br>Enright: {row['text_enright'][:25]}...", axis=1)

# Create interactive Plotly plot focusing on the 'mistral' sentiment model
fig = px.line(
    combined_df,
    x='sentence_no',
    y='mistral',  # Ensure this is the Z-score normalized value
    color='source_clean',
    title="Swan's Way by Proust\nSentiment vs Sentence No\nMistral 7B LLM\nNormalized (Z-Score) and Smoothed (SMA 10%)",
    labels={'sentence_no': 'Sentence Number', 'mistral': 'Z-score Normalized Sentiment'}
)

fig.update_layout(
    xaxis_title='Sentence Number',
    yaxis_title='Z-score Normalized Sentiment',
    legend_title='Source',
    hovermode='x unified',
    legend=dict(x=0.01, y=0.01, xanchor='left', yanchor='bottom', bgcolor='rgba(255,255,255,0.5)')
)

# Add hover data for exact text from each translation
fig.update_traces(
    hovertemplate="<br>".join([
        "Source: %{customdata[0]}",
        "Sentence No: %{x}",
        "Sentiment: %{y}",
        "Text: %{customdata[1]}"
    ]),
    customdata=hover_data[['source_clean', 'hover_text']].values
)
fig.show()

# Create high-quality seaborn plots with normalized values
sns.set(style="whitegrid")
plt.figure(figsize=(14, 7), dpi=300)

# Plot mean sentiment scores with SMA smoothing and normalized values for the 'mistral' sentiment model
for source, group_df in combined_df.groupby('source_clean'):
    plt.plot(group_df['sentence_no'], group_df['mistral'], label=source)

plt.title("Swan's Way by Proust\nSentiment vs Sentence No\nMistral 7B LLM\nNormalized (Z-Score) and Smoothed (SMA 10%)", fontsize=18)
plt.xlabel("Sentence Number", fontsize=14)
plt.ylabel("Z-score Normalized Sentiment", fontsize=14)
plt.legend(title="Source", bbox_to_anchor=(0.01, 0.01), loc='lower left')
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()

plt.savefig("sentiment_timeseries_seaborn.png", format='png', dpi=300, bbox_inches='tight')
plt.show()

# Create high-resolution plot using LOWESS smoothing
combined_df['lowess_mistral'] = combined_df.groupby('source')['mistral'].transform(lambda x: lowess(x, np.arange(len(x)), frac=0.1)[:, 1])

plt.figure(figsize=(14, 7), dpi=300)

# Plot LOWESS smoothed sentiment scores for the 'mistral' sentiment model
for source, group_df in combined_df.groupby('source_clean'):
    plt.plot(group_df['sentence_no'], group_df['lowess_mistral'], label=source)

plt.title("Swan's Way by Proust\nSentiment vs Sentence No\nMistral 7B LLM\nNormalized (Z-Score) and Smoothed (LOWESS)", fontsize=18)
plt.xlabel("Sentence Number", fontsize=14)
plt.ylabel("Z-score Normalized Sentiment", fontsize=14)
plt.legend(title="Source", bbox_to_anchor=(0.01, 0.01), loc='lower left')
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()

plt.savefig("sentiment_timeseries_seaborn_lowess.png", format='png', dpi=300, bbox_inches='tight')
plt.show()



In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import zscore
import plotly.express as px
import seaborn as sns
import matplotlib.pyplot as plt
from statsmodels.nonparametric.smoothers_lowess import lowess

# Example dictionary of dataframes (using mock data for demonstration)
sentiments_all_dict = {
    "book_proust_fr_swans-way_proust_original_segments.csv": pd.DataFrame({
        'text': ["Parfois, a peine ma bougie eteinte, mes yeux se fermaient immédiatement " + str(i) for i in range(4514)],
        'vader': np.random.randn(4514),
        'textblob': np.random.randn(4514),
        'bertmulti': np.random.randn(4514),
        'mistral': np.random.randn(4514)
    }),
    "book_proust_en_swans-way_moncrieff_original_segments.csv": pd.DataFrame({
        'text': ["Sometimes, when I had put out my candle, my eyes would close so quickly that I had not even time to say 'I'm going to sleep.'" + str(i) for i in range(4287)],
        'vader': np.random.randn(4287),
        'textblob': np.random.randn(4287),
        'bertmulti': np.random.randn(4287),
        'mistral': np.random.randn(4287)
    }),
    "book_proust_en_swans-way_davis_original_segments.csv": pd.DataFrame({
        'text': ["The Way by Swann's, for a long time I used to go to bed early." + str(i) for i in range(4321)],
        'vader': np.random.randn(4321),
        'textblob': np.random.randn(4321),
        'bertmulti': np.random.randn(4321),
        'mistral': np.random.randn(4321)
    }),
    "book_proust_en_swans-way_enright_original_segments.csv": pd.DataFrame({
        'text': ["For a long time I would go to bed early, sometimes the candle barely out, my eyes would close so quickly that I had not even time to say I'm going to sleep." + str(i) for i in range(4449)],
        'vader': np.random.randn(4449),
        'textblob': np.random.randn(4449),
        'bertmulti': np.random.randn(4449),
        'mistral': np.random.randn(4449)
    }),
}

WIN_PER = 10  # Percentage for SMA smoothing

# 1. Dictionaries for clean output names
output_map_titles_dict = {
    "book_proust_fr_swans-way_proust_original_segments.csv": "Swan's Way (original Proust in French)",
    "book_proust_en_swans-way_moncrieff_original_segments.csv": "Swan's Way (trans. Moncrieff in English)",
    "book_proust_en_swans-way_davis_original_segments.csv": "Swan's Way (trans. Davis in English)",
    "book_proust_en_swans-way_enright_original_segments.csv": "Swan's Way (trans. Enright in English)"
}

output_map_legend_dict = {
    "book_proust_fr_swans-way_proust_original_segments.csv": "Proust Original, French",
    "book_proust_en_swans-way_moncrieff_original_segments.csv": "Moncrieff Translation, English",
    "book_proust_en_swans-way_davis_original_segments.csv": "Davis Translation, English",
    "book_proust_en_swans-way_enright_original_segments.csv": "Enright Translation, English"
}

# 2. Aggregate the longer time series
def aggregate_short_text(df, target_length):
    while len(df) > target_length:
        # Find the shortest text fragment
        df['text_length'] = df['text'].apply(len)
        shortest_index = df['text_length'].idxmin()

        # Identify the shortest neighbor (previous or next)
        if shortest_index == 0:
            neighbor_index = shortest_index + 1
        elif shortest_index == len(df) - 1:
            neighbor_index = shortest_index - 1
        else:
            neighbor_index = shortest_index + 1 if df['text_length'][shortest_index + 1] < df['text_length'][shortest_index - 1] else shortest_index - 1

        # Combine the shortest text fragment with its neighbor
        df.at[neighbor_index, 'text'] = df.at[neighbor_index, 'text'] + " " + df.at[shortest_index, 'text']

        # Average the sentiment values
        for sentiment in ['vader', 'textblob', 'bertmulti', 'mistral']:
            df.at[neighbor_index, sentiment] = (df.at[neighbor_index, sentiment] + df.at[shortest_index, sentiment]) / 2

        # Drop the shortest text fragment row
        df = df.drop(shortest_index).reset_index(drop=True)

    # Drop the temporary 'text_length' column
    df = df.drop(columns=['text_length'])

    return df

# Find the minimum length
min_length = min(len(df) for df in sentiments_all_dict.values())

# Create a new dictionary for equal length timeseries (using copies)
sentiments_all_equallen_dict = {}
for key, df in sentiments_all_dict.items():
    df_copy = df.copy()
    if len(df_copy) > min_length:
        df_copy = aggregate_short_text(df_copy, min_length)
    sentiments_all_equallen_dict[key] = df_copy

# Z-score normalize the sentiment values
for key, df in sentiments_all_equallen_dict.items():
    sentiments_all_equallen_dict[key][['vader', 'textblob', 'bertmulti', 'mistral']] = df[['vader', 'textblob', 'bertmulti', 'mistral']].apply(zscore)

# Apply SMA smoothing
def apply_smoothing(df, window_percentage):
    window_length = max(int(len(df) * window_percentage / 100), 1)
    for sentiment in ['vader', 'textblob', 'bertmulti', 'mistral']:
        df[sentiment] = df[sentiment].rolling(window=window_length, min_periods=1, center=True).mean()
    return df

if WIN_PER > 0:
    for key in sentiments_all_equallen_dict.keys():
        sentiments_all_equallen_dict[key] = apply_smoothing(sentiments_all_equallen_dict[key], WIN_PER)

# Combine dataframes and prepare for plotting
combined_df = pd.concat(sentiments_all_equallen_dict, axis=0)
combined_df.reset_index(level=0, inplace=True)
combined_df.rename(columns={'level_0': 'source'}, inplace=True)
combined_df['sentence_no'] = combined_df.groupby('source').cumcount() + 1

# Map the clean names to the combined dataframe
combined_df['source_clean'] = combined_df['source'].map(output_map_legend_dict)

# Create a column for shortened text for tooltips
combined_df['short_text'] = combined_df['text'].apply(lambda x: x[:25] + '...')

# Create columns for each text version
combined_df['text_proust'] = combined_df[combined_df['source'] == 'book_proust_fr_swans-way_proust_original_segments.csv']['text']
combined_df['text_moncrieff'] = combined_df[combined_df['source'] == 'book_proust_en_swans-way_moncrieff_original_segments.csv']['text']
combined_df['text_davis'] = combined_df[combined_df['source'] == 'book_proust_en_swans-way_davis_original_segments.csv']['text']
combined_df['text_enright'] = combined_df[combined_df['source'] == 'book_proust_en_swans-way_enright_original_segments.csv']['text']

# Merge the text columns for hover data
hover_data = combined_df[['source_clean', 'short_text', 'text_proust', 'text_moncrieff', 'text_davis', 'text_enright']].fillna('')
hover_data['hover_text'] = hover_data.apply(lambda row: f"Proust: {row['text_proust'][:25]}...<br>Moncrieff: {row['text_moncrieff'][:25]}...<br>Davis: {row['text_davis'][:25]}...<br>Enright: {row['text_enright'][:25]}...", axis=1)

# Create interactive Plotly plot focusing on the 'mistral' sentiment model
fig = px.line(
    combined_df,
    x='sentence_no',
    y='mistral',  # Ensure this is the Z-score normalized value
    color='source_clean',
    title="Swan's Way by Proust\nSentiment vs Sentence No\nMistral 7B LLM\nNormalized (Z-Score) and Smoothed (SMA 10%)",
    labels={'sentence_no': 'Sentence Number', 'mistral': 'Z-score Normalized Sentiment'}
)

fig.update_layout(
    xaxis_title='Sentence Number',
    yaxis_title='Z-score Normalized Sentiment',
    legend_title='Source',
    hovermode='x unified',
    legend=dict(x=0.01, y=0.01, xanchor='left', yanchor='bottom', bgcolor='rgba(255,255,255,0.5)')
)

# Add hover data for exact text from each translation
fig.update_traces(
    hovertemplate="<br>".join([
        "Source: %{customdata[0]}",
        "Sentence No: %{x}",
        "Sentiment: %{y}",
        "Text: %{customdata[1]}"
    ]),
    customdata=hover_data[['source_clean', 'hover_text']].values
)

fig.show()

# Create high-quality seaborn plots with normalized values
sns.set(style="whitegrid")
plt.figure(figsize=(14, 7), dpi=300)

# Plot mean sentiment scores with SMA smoothing and normalized values for the 'mistral' sentiment model
for source, group_df in combined_df.groupby('source_clean'):
    plt.plot(group_df['sentence_no'], group_df['mistral'], label=source)

plt.title("Swan's Way by Proust\nSentiment vs Sentence No\nMistral 7B LLM\nNormalized (Z-Score) and Smoothed (SMA 10%)", fontsize=18)
plt.xlabel("Sentence Number", fontsize=14)
plt.ylabel("Z-score Normalized Sentiment", fontsize=14)
plt.legend(title="Source", bbox_to_anchor=(0.01, 0.01), loc='lower left')
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()

plt.savefig("sentiment_timeseries_seaborn.png", format='png', dpi=300, bbox_inches='tight')
plt.show()

# Create high-resolution plot using LOWESS smoothing
combined_df['lowess_mistral'] = combined_df.groupby('source')['mistral'].transform(lambda x: lowess(x, np.arange(len(x)), frac=0.1)[:, 1])

plt.figure(figsize=(14, 7), dpi=300)

# Plot LOWESS smoothed sentiment scores for the 'mistral' sentiment model
for source, group_df in combined_df.groupby('source_clean'):
    plt.plot(group_df['sentence_no'], group_df['lowess_mistral'], label=source)

plt.title("Swan's Way by Proust\nSentiment vs Sentence No\nMistral 7B LLM\nNormalized (Z-Score) and Smoothed (LOWESS)", fontsize=18)
plt.xlabel("Sentence Number", fontsize=14)
plt.ylabel("Z-score Normalized Sentiment", fontsize=14)
plt.legend(title="Source", bbox_to_anchor=(0.01, 0.01), loc='lower left')
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()

plt.savefig("sentiment_timeseries_seaborn_lowess.png", format='png', dpi=300, bbox_inches='tight')
plt.show()



In [ ]:
sentiments_all_equallen_dict.keys()

In [ ]:
sentiments_all_dict['book_proust_fr_swans-way_proust_original_segments.csv'][100:103]

### START OLD CODE

In [ ]:
model_input_ls

In [ ]:
##@title Enter the Sliding Window width as Percent of Novel length (default 10%, larger=smoother)

Window_Percent = 10 #@param {type:"slider", min:1, max:20, step:1}

win_per = Window_Percent
win_size = int(win_per/100 * sentiment_df.shape[0])

In [ ]:
sentiment_df.head()
sentiment_df.info()
sentiment_df.describe()

## Merge and Plot Mutliple Translations

In [ ]:
sentiments_all_dict.keys()

In [ ]:


def plot_sma_zscores(dictionary_of_df, win_per=10):
    """
    Plots the sentiment time series with SMA and Z-scores for each DataFrame in the dictionary.

    Parameters:
    dictionary_of_df (dict): Dictionary where keys are filenames and values are pandas DataFrames.
                             Each DataFrame contains a 'text' column and other columns for sentiment scores.
    win_per (int): The window size percentage for the Simple Moving Average (SMA).
    """
    for key, df in dictionary_of_df.items():
        plt.figure(figsize=(10, 6))

        # Calculate the window size for SMA
        window_size = max(1, int(len(df) * win_per / 100))

        # Temporary DataFrame to store z-scores
        temp_df = pd.DataFrame()

        # Compute z-scores for each sentiment column
        for sentiment_model in df.columns[1:]:  # Skip the first column 'text'
            mean = df[sentiment_model].mean()
            std = df[sentiment_model].std()
            z_scores = (df[sentiment_model] - mean) / std
            temp_df[sentiment_model] = z_scores

        # Plot each sentiment column with SMA against the row number using z-scores
        for sentiment_model in temp_df.columns:
            smoothed_series = temp_df[sentiment_model].rolling(window=window_size, min_periods=1, center=True).mean()
            smoothed_series = smoothed_series.interpolate(method='linear')
            plt.plot(df.index, smoothed_series, label=sentiment_model)

        plt.title(f'Sentiment Analysis (Z-Scores) for {key}')
        plt.xlabel('Row Number')
        plt.ylabel('Z-Scores of Sentiment Scores')
        plt.legend()
        plt.xticks(rotation=90)
        plt.tight_layout()  # Adjust layout to prevent clipping of labels
        plt.show()

# Example usage
# Assuming 'dataframes_dict' is a dictionary of DataFrames
# plot_sma_zscores(dataframes_dict, win_per=10)


In [ ]:
plot_sma_zscores(sentiments_all_dict)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

def plot_sma_zscores(dictionary_of_df, win_per=10, plot_by_model=False):
    """
    Plots the sentiment time series with SMA and Z-scores for each DataFrame in the dictionary.

    Parameters:
    dictionary_of_df (dict): Dictionary where keys are filenames and values are pandas DataFrames.
                             Each DataFrame contains a 'text' column and other columns for sentiment scores.
    win_per (int): The window size percentage for the Simple Moving Average (SMA).
    plot_by_model (bool): If True, reorganize and plot by sentiment model instead of by filename.
    """
    if not plot_by_model:
        # Plot by filename (book)
        for key, df in dictionary_of_df.items():
            plt.figure(figsize=(10, 6))

            # Calculate the window size for SMA
            window_size = max(1, int(len(df) * win_per / 100))

            # Temporary DataFrame to store z-scores
            temp_df = pd.DataFrame()

            # Compute z-scores for each sentiment column
            for sentiment_model in df.columns[1:]:  # Skip the first column 'text'
                mean = df[sentiment_model].mean()
                std = df[sentiment_model].std()
                z_scores = (df[sentiment_model] - mean) / std
                temp_df[sentiment_model] = z_scores

            # Plot each sentiment column with SMA against the row number using z-scores
            for sentiment_model in temp_df.columns:
                smoothed_series = temp_df[sentiment_model].rolling(window=window_size, min_periods=1, center=True).mean()
                smoothed_series = smoothed_series.interpolate(method='linear')
                plt.plot(df.index, smoothed_series, label=sentiment_model)

            plt.title(f'Sentiment Analysis (Z-Scores) for {key}')
            plt.xlabel('Row Number')
            plt.ylabel('Z-Scores of Sentiment Scores')
            plt.legend()
            plt.xticks(rotation=90)
            plt.tight_layout()  # Adjust layout to prevent clipping of labels
            plt.show()
    else:
        # Plot by sentiment model
        models = dictionary_of_df[next(iter(dictionary_of_df))].columns[1:]  # Get list of sentiment models from the first DataFrame

        for model in models:
            plt.figure(figsize=(10, 6))

            # Temporary DataFrame to store concatenated z-scores for the model
            temp_df = pd.DataFrame()

            for key, df in dictionary_of_df.items():
                mean = df[model].mean()
                std = df[model].std()
                z_scores = (df[model] - mean) / std
                temp_df = pd.concat([temp_df, z_scores.reset_index(drop=True)], axis=0)

            # Reset index to make it continuous
            temp_df.reset_index(drop=True, inplace=True)

            # Calculate the window size for SMA
            window_size = max(1, int(len(temp_df) * win_per / 100))

            # Apply SMA and interpolation
            smoothed_series = temp_df.rolling(window=window_size, min_periods=1, center=True).mean()
            smoothed_series = smoothed_series.interpolate(method='linear')

            plt.plot(smoothed_series, label=model)

            plt.title(f'Sentiment Analysis (Z-Scores) for Model: {model}')
            plt.xlabel('Row Number')
            plt.ylabel('Z-Scores of Sentiment Scores')
            plt.legend()
            plt.xticks(rotation=90)
            plt.tight_layout()  # Adjust layout to prevent clipping of labels
            plt.show()

# Example usage
# Assuming 'dataframes_dict' is a dictionary of DataFrames
# plot_sma_zscores(dataframes_dict, win_per=10, plot_by_model=False)
# plot_sma_zscores(dataframes_dict, win_per=10, plot_by_model=True)


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

def plot_sma_zscores(dictionary_of_df, win_per=10, plot_by_model=False):
    """
    Plots the sentiment time series with SMA and Z-scores for each DataFrame in the dictionary.

    Parameters:
    dictionary_of_df (dict): Dictionary where keys are filenames and values are pandas DataFrames.
                             Each DataFrame contains a 'text' column and other columns for sentiment scores.
    win_per (int): The window size percentage for the Simple Moving Average (SMA).
    plot_by_model (bool): If True, reorganize and plot by sentiment model instead of by filename.
    """
    if not plot_by_model:
        # Plot by filename (book)
        for key, df in dictionary_of_df.items():
            plt.figure(figsize=(10, 6))

            # Calculate the window size for SMA
            window_size = max(1, int(len(df) * win_per / 100))

            # Temporary DataFrame to store z-scores
            temp_df = pd.DataFrame()

            # Compute z-scores for each sentiment column
            for sentiment_model in df.columns[1:]:  # Skip the first column 'text'
                mean = df[sentiment_model].mean()
                std = df[sentiment_model].std()
                z_scores = (df[sentiment_model] - mean) / std
                temp_df[sentiment_model] = z_scores

            # Plot each sentiment column with SMA against the row number using z-scores
            for sentiment_model in temp_df.columns:
                smoothed_series = temp_df[sentiment_model].rolling(window=window_size, min_periods=1, center=True).mean()
                smoothed_series = smoothed_series.interpolate(method='linear')
                plt.plot(df.index, smoothed_series, label=sentiment_model)

            plt.title(f'Sentiment Analysis (Z-Scores) for {key}')
            plt.xlabel('Row Number')
            plt.ylabel('Z-Scores of Sentiment Scores')
            plt.legend()
            plt.xticks(rotation=90)
            plt.tight_layout()  # Adjust layout to prevent clipping of labels
            plt.show()
    else:
        # Plot by sentiment model
        models = dictionary_of_df[next(iter(dictionary_of_df))].columns[1:]  # Get list of sentiment models from the first DataFrame

        for model in models:
            plt.figure(figsize=(10, 6))

            for key, df in dictionary_of_df.items():
                # Temporary DataFrame to store z-scores
                temp_df = pd.DataFrame()

                mean = df[model].mean()
                std = df[model].std()
                z_scores = (df[model] - mean) / std
                temp_df[model] = z_scores

                # Calculate the window size for SMA
                window_size = max(1, int(len(temp_df) * win_per / 100))

                # Apply SMA and interpolation
                smoothed_series = temp_df[model].rolling(window=window_size, min_periods=1, center=True).mean()
                smoothed_series = smoothed_series.interpolate(method='linear')
                plt.plot(df.index, smoothed_series, label=key)

            plt.title(f'Sentiment Analysis (Z-Scores) for Model: {model}')
            plt.xlabel('Row Number')
            plt.ylabel('Z-Scores of Sentiment Scores')
            plt.legend()
            plt.xticks(rotation=90)
            plt.tight_layout()  # Adjust layout to prevent clipping of labels
            plt.show()

# Example usage
# Assuming 'dataframes_dict' is a dictionary of DataFrames
# plot_sma_zscores(dataframes_dict, win_per=10, plot_by_model=False)
# plot_sma_zscores(dataframes_dict, win_per=10, plot_by_model=True)


In [ ]:
plot_sma_zscores(sentiments_all_dict, win_per=10, plot_by_model=True)

## Plots

### Plot Raw Timeseries

### Ensure sentiment_df has no NaNs

In [ ]:
sentiment_df.info()

In [ ]:
# fillna(0)

model_ls = sentiment_df.columns.to_list()

for col_name in model_input_ls:
  sentiment_df[col_name] = sentiment_df[col_name].fillna(0)

sentiment_df.info()

### Define model columns in sentiment_df

##### Raw Sentiment Plots

In [ ]:
# Plot Raw Timeseries

win_per = 10
win_size = int((win_per)/100 * sentiment_df.shape[0])
ax = sentiment_df[model_ls].rolling(win_size, center=True).mean().plot(grid=True, lw=3, colormap='Dark2')
ax.title.set_text(f'Sentiment Analysis \n {novel_title} \n Raw Sentiment Timeseries')

plt.show()

##### Seaborn Plots

#### Plotly Interactive Plotly

In [ ]:
%whos list

In [ ]:
novel_lines_ls[]

In [ ]:
model_ls


In [ ]:
# Let crux point sentences

win_lines = 10
win_half = int(win_lines / 2)
print(f"win_half: {win_half}")

crux_sentence_no = 1795
crux_range = f"[{crux_sentence_no - win_half}:{crux_sentence_no + win_half}]"
print(f"crux_range: {crux_range}")

index_start = crux_sentence_no - win_half
index_end = crux_sentence_no + win_half

# Assuming lines_list is defined somewhere in your code
# Example: lines_list = ["sentence 1", "sentence 2", ..., "sentence n"]

print(f"Crux around Sentence #{crux_sentence_no}: \n")
for line_now in lines_list[index_start:index_end]:
  print(f"  {line_now}")


In [ ]:
%whos dict

**[END]**

In [ ]:
import pandas as pd
import plotly.graph_objects as go

# Example sentiment_df and model_input_ls for testing
sentiment_df = pd.DataFrame({
    'sentence_num': [1, 2, 3, 4, 5],
    'sentence_text': ["The world is changed.", "I feel it in the water.", "I feel it in the earth.", "I smell it in the air.", "Much that once was is lost."],
    'model_col_1': [0.5, 0.6, 0.7, 0.8, 0.9],
    'model_col_2': [0.1, 0.2, 0.3, 0.4, 0.5],
})
model_input_ls = ['model_col_1', 'model_col_2']

# Define the transform_zscore_norm function
def transform_zscore_norm(original_df, model_list):
    transformed_df = original_df.copy()
    for col in model_list:
        mean_val = transformed_df[col].mean()
        std_val = transformed_df[col].std()
        transformed_df[col] = (transformed_df[col] - mean_val) / std_val
    return transformed_df

# Transform the dataframe
sentiment_zscore_df = transform_zscore_norm(sentiment_df, model_input_ls)

# Create Plotly plot with rollover points showing sentence number and text
def plot_with_rollover(dataframe, model_list, novel_title):
    fig = go.Figure()

    for column in model_list:
        fig.add_trace(go.Scatter(
            x=dataframe['sentence_num'],
            y=dataframe[column],
            mode='lines+markers',
            name=column,
            hovertext=dataframe['sentence_num'].astype(str) + ": " + dataframe['sentence_text'],
            hoverinfo='text'
        ))

    # Update layout
    fig.update_layout(
        title_text=f'Sentiment Analysis - {novel_title} - Raw Sentiment Timeseries',
        xaxis_title="Sentence Number",
        yaxis_title="Standardized Value",
        autosize=False,
        width=800,
        height=500,
        margin=dict(
            l=50,
            r=50,
            b=100,
            t=100,
            pad=4
        ),
        paper_bgcolor="LightSteelBlue",
    )

    fig.show()

# Example usage
novel_title = "Example Novel"
plot_with_rollover(sentiment_zscore_df, model_input_ls, novel_title)


#### Get Most Incoherent Sentences in Novel

In [ ]:
sentiment_df.head()
sentiment_df.info()
sentiment_df.describe()

In [ ]:
model_input_ls

In [ ]:
model_input_ls = [
 'sentiment-bertmulti',
 'sentiment-mistral7b',
 'sentiment-textblob',
 'sentiment-vader',
]

In [ ]:
# Compute the coherence column
sentiment_df['incoherence'] = sentiment_df[model_input_ls].apply(lambda x: x.max() - x.min(), axis=1)

# Display the updated DataFrame
sentiment_df.head()

In [ ]:
# Calculate the maximum difference between 'sentiment-mistral7b' and all other model columns
sentiment_df['max_mistral_diff'] = sentiment_df.apply(
    lambda row: max(abs(row['sentiment-mistral7b'] - row[col]) for col in model_input_ls if col != 'sentiment-mistral7b'), axis=1
)

# Display the updated DataFrame
sentiment_df.head()
sentiment_df.info()
sentiment_df.describe()

In [ ]:
model_input_ls

In [ ]:
def transform_scale_center(original_df, model_list):
    # Step 0: Create a new DataFrame to contain the transformed time series
    transformed_df = original_df.copy()

    # Step 1: Rescale all values between -1.0 and +1.0 for specified columns
    for col in model_list:
        min_val = transformed_df[col].min()
        max_val = transformed_df[col].max()

        # Rescale the values between -1.0 and +1.0
        transformed_df[col] = (2 * (transformed_df[col] - min_val) / (max_val - min_val)) - 1

    # Step 2: Adjust all means to 0 for specified columns
    for col in model_list:
        transformed_df[col] = transformed_df[col] - transformed_df[col].mean()

    return transformed_df

In [ ]:
model_input_ls

In [ ]:
def plot_only_models(dataframe_in, model_cols, win_per=10):
    # Calculate the window size for the moving average (win_per% of the data length)
    win_size = max(1, int((win_per / 100) * len(dataframe_in)))
    if win_size % 2 == 0:
        win_size += 1  # Ensure window size is odd for centering

    plt.figure(figsize=(15, 8))

    for model in model_cols:
        # Calculate the moving average with the specified window size
        sma = dataframe_in[model].rolling(window=win_size, min_periods=1, center=True).mean()
        plt.plot(dataframe_in.index, sma, label=f'{model} SMA {win_per}%')

    plt.title(f'Smoothed Time Series Plot with SMA {win_per}%\n{novel_title}\nZ-Score Normed')
    plt.xlabel('Index')
    plt.ylabel('Value')
    plt.legend(model_ls, fontsize=10, loc='upper right')
    plt.grid(True)

    plt.tight_layout()
    plt.show();

plot_only_models(sentiment_zscore_df, model_input_ls, win_per=10)


In [ ]:
def transform_zscore_norm(original_df, model_list):
    # Step 0: Create a new DataFrame to contain the transformed time series
    transformed_df = original_df.copy()

    # Step 1: Standardize (z-score normalization) for specified columns
    for col in model_list:
        mean_val = transformed_df[col].mean()
        std_val = transformed_df[col].std()

        # Standardize the values to have mean 0 and standard deviation 1
        transformed_df[col] = (transformed_df[col] - mean_val) / std_val

    return transformed_df

sentiment_zscore_df = transform_zscore_norm(sentiment_df, model_input_ls)

# Display the transformed DataFrame
# print(sentiment_zscore_df)
plot_only_models(sentiment_zscore_df, model_input_ls)

In [ ]:
model_input_ls

In [ ]:
# sentiment_transformed_zscore_df = sentiment_zscore_df(sentiment_df, model_input_ls)
sentiment_zscore_df.head()
sentiment_zscore_df.info()
sentiment_zscore_df.describe()

In [ ]:
def plot_models_plus(dataframe_in, model_cols, win_per=10):
    # Calculate the window size for the moving average (win_per% of the data length)
    win_size = max(1, int((win_per / 100) * len(dataframe_in)))
    if win_size % 2 == 0:
        win_size += 1  # Ensure window size is odd for centering

    # Identify columns not in model_cols
    additional_cols = [col for col in dataframe_in.columns if col not in model_cols]

    # Create a figure with subplots
    fig, axs = plt.subplots(1 + len(additional_cols), 1, figsize=(15, 8 + 3 * len(additional_cols)), gridspec_kw={'height_ratios': [3] + [1] * len(additional_cols)})

    # Plot the specified time series and their SMA in the main plot
    for model in model_cols:
        # Calculate the moving average with the specified window size
        sma = dataframe_in[model].rolling(window=win_size, min_periods=1, center=True).mean()
        axs[0].plot(dataframe_in.index, sma, label=f'{model} SMA {win_per}%')

    axs[0].set_title(f'Smoothed Time Series Plot with SMA\n{novel_title}\nZ-Score Normed')
    axs[0].set_xlabel('Index')
    axs[0].set_ylabel('Value')
    axs[0].legend(model_ls, fontsize=10, loc='upper right')
    axs[0].grid(True)

    # Plot each additional column in its own subplot with SMA smoothing
    for i, col in enumerate(additional_cols):
        # Calculate the moving average for the additional columns
        sma = dataframe_in[col].rolling(window=win_size, min_periods=1, center=True).mean()
        axs[i + 1].plot(dataframe_in.index, sma, label=f'{col} SMA {win_per}%', color='orange')
        axs[i + 1].set_title(col)
        axs[i + 1].set_xlabel('Index')
        axs[i + 1].set_ylabel(col)
        axs[i + 1].legend(model_ls, fontsize=10, loc='upper right')
        axs[i + 1].grid(True)

    # Adjust the layout
    plt.tight_layout()
    plt.show();

plot_models_plus(sentiment_zscore_df, model_input_ls, win_per=10)

In [ ]:
sentiment_df['sentiment-bertmulti'].unique()

In [ ]:
pd.Series(sentiment_df['incoherence'] == sentiment_df['max_mistral_diff']).value_counts()

In [ ]:
pd.Series(sentiment_zscore_df['incoherence'] == sentiment_zscore_df['max_mistral_diff']).value_counts()

#### Sentiment Distribution by Model

In [ ]:
model_ls
len(model_ls)

In [ ]:
# Ensure color_ls is long enough by cycling through the colors if needed
base_colors = ['red', 'blue', 'green', 'purple', 'orange', 'gray', 'cyan', 'magenta', 'olive', 'black', 'pink', 'brown']
color_ls = [color for _, color in zip(range(len(model_ls)), cycle(base_colors))]
len(color_ls)

##### KDE ALL

In [ ]:


# Define your model list
# model_ls = ['sentiment-bertmulti', 'sentiment-mistral7b', 'sentiment-textblob', 'sentiment-vader']  # example

# Ensure color_ls is long enough by cycling through the colors if needed
base_colors = ['red', 'blue', 'green', 'purple', 'orange', 'gray', 'cyan', 'magenta', 'olive', 'black', 'pink', 'brown']
color_ls = [color for _, color in zip(range(len(model_ls)), cycle(base_colors))]

# Increase font sizes
plt.rcParams.update({
    'axes.titlesize': 30,
    'axes.labelsize': 27,
    'xtick.labelsize': 24,
    'ytick.labelsize': 24,
    'legend.fontsize': 27,
    'figure.titlesize': 33
})

for i, col in enumerate(model_ls):
    print(f"Processing {col}")
    sns.kdeplot(data=sentiment_zscore_df[col], color=color_ls[i], alpha=0.2, linewidth=2, fill=True)

# Add vertical dashed red lines with labels
plt.axvline(x=-1.0, color='red', linestyle='--', linewidth=4, alpha=0.3)
plt.axvline(x=1.0, color='red', linestyle='--', linewidth=4, alpha=0.3)
plt.text(-1.0, plt.ylim()[1] * 0.5, '-1.0 Min Sentiment Value', color='red', fontsize=18, rotation=90, ha='left')
plt.text(1.0, plt.ylim()[1] * 0.5, '+1.0 Max Sentiment Value', color='red', fontsize=18, rotation=90, ha='left')

# Add title and subtitle to the plot
plt.title(f"KDE Sentiment Value Distributions by Model\nfor All Sentiment Sentence Values\n{novel_title}\nZ-Score Normed", fontsize=26)

# Add key to the plot
plt.legend(model_ls, fontsize=10, loc='upper right')

# Show the plot
plt.show();


**[SKIP]**

##### KDE Top 10% Frequency

In [ ]:
# create KDE smooth distributions

ten_percent = int(0.1*sentiment_df.shape[0])

"""
if NOVEL_CUR == 'b_tm':
  # upto 13 models (eg Beloved by Toni Morrison)
  color_ls = ['red', 'orange', 'yellow', 'green', 'blue', 'purple', 'pink', 'brown', 'gray', 'black', 'cyan', 'magenta', 'olive']
elif NOVEL_CUR == 'ttl_vf':
  # upto 6 models (eg To The Lighthouse by Virgina Woolf)
  color_ls = ['red', 'blue', 'green', 'purple', 'orange', 'gray']
else:
  print(f"ERROR: NOVEL_CUR={NOVEL_CUR} does not have a color_ls assigned.")

""";

color_ls = ['red', 'blue', 'green']

for i, col in enumerate(model_ls):
    sns.kdeplot(data=sentiment_df.iloc[:ten_percent][col], color=color_ls[i], alpha=0.2, linewidth=2, fill=True)

# add vertical dashed red lines with labels
plt.axvline(x=-1.0, color='red', linestyle='--', linewidth=4, label='-1.0 Min Sentiment Value')
plt.axvline(x=1.0, color='red', linestyle='--', linewidth=4, label='+1.0 Max Sentiment Value')
# plt.text(-1.0, 0.1, '-1.0 Min Sentiment Value', color='red', fontsize=12, rotation=90)
# plt.text(1.0, 0.1, '+1.0 Max Sentiment Value', color='red', fontsize=12, rotation=90)
plt.text(-0.9, 2.5, '-1.0 Min Sentiment Value', color='red', fontsize=12, rotation=90, ha='center')
plt.text(1.1, 2.5, '+1.0 Max Sentiment Value', color='red', fontsize=12, rotation=90, ha='center')


# add title and subtitle to the plot
# plt.suptitle('KDE Sentiment Value Distributions by Model', fontsize=16)
plt.title('KDE Sentiment Value Distributions by Model\nfor Top 10% Incoherent Sentiment Sentence Values\nTo The Lighthouse by Virginia Woolf', fontsize=16)

# add key to the plot
plt.legend(model_ls)

# show the plot
plt.show();

In [ ]:
model_ls

##### KDE Top 10% Incoherent

In [ ]:
# create KDE smooth distributions


if NOVEL_CUR == 'b_tm':
  # upto 13 models (eg Beloved by Toni Morrison)
  color_ls = ['red', 'orange', 'yellow', 'green', 'blue', 'purple', 'pink', 'brown', 'gray', 'black', 'cyan', 'magenta', 'olive']
elif NOVEL_CUR == 'ttl_vf':
  # upto 6 models (eg To The Lighthouse by Virgina Woolf)
  color_ls = ['red', 'blue', 'green', 'purple', 'orange', 'gray']
else:
  print(f"ERROR: NOVEL_CUR={NOVEL_CUR} does not have a color_ls assigned.")

for i, col in enumerate(model_ls):
    sns.kdeplot(data=thresh_minmax_diff_df[col], color=color_ls[i], alpha=0.2, linewidth=2, fill=True)

# add vertical dashed red lines with labels
plt.axvline(x=-1.0, color='red', linestyle='--', linewidth=4, label='-1.0 Min Sentiment Value')
plt.axvline(x=1.0, color='red', linestyle='--', linewidth=4, label='+1.0 Max Sentiment Value')
# plt.text(-1.0, 0.1, '-1.0 Min Sentiment Value', color='red', fontsize=12, rotation=90)
# plt.text(1.0, 0.1, '+1.0 Max Sentiment Value', color='red', fontsize=12, rotation=90)
plt.text(-0.9, 1.7, '-1.0 Min Sentiment Value', color='red', fontsize=12, rotation=90, ha='center')
plt.text(1.1, 1.7, '+1.0 Max Sentiment Value', color='red', fontsize=12, rotation=90, ha='center')


# add title and subtitle to the plot
# plt.suptitle('KDE Sentiment Value Distributions by Model', fontsize=16)
plt.title(f"KDE Sentiment Value Distributions by Model\nfor Maximally Incoherent Sentiment Sentence Values\n{Novel_Title}", fontsize=16)

# add key to the plot
plt.legend(model_ls)

# show the plot
plt.show();

In [ ]:
sentiment_df.info()

In [ ]:
# Convert model sentiment columns to numeric and handle missing values (if any)
for model in model_ls:
    sentiment_df[model] = pd.to_numeric(sentiment_df[model], errors='coerce')
    sentiment_df[model].fillna(0, inplace=True)  # replacing missing values with 0

In [ ]:
# Convert model sentiment columns to float and handle missing values (if any)
for model in model_ls:
    sentiment_df[model] = sentiment_df[model].astype(float)

In [ ]:
model_ls

In [ ]:
# Sentiment Value Distributions by Model

MODEL_CORE_FL = True

if MODEL_CORE_FL:
  if NOVEL_CUR == 'b_tm':
    model_subset_ls = ['nlptown', 'gpt35', 'gpt4', 'roberta15lg', 'textblob', 'vader']
  elif NOVEL_CUR == 'ttl_vf':
    model_subset_ls = ['distilbert', 'gpt35', 'gpt4', 'roberta15lg', 'textblob', 'vader']
  else:
    print(f"ERROR: NOVEL_CUR={NOVEL_CUR} does not have a color_ls assigned.")
else:
  if NOVEL_CUR == 'b_tm':
    model_subset_ls = ['ada_v1p_score','ada_v2p_score','ada_v3p_score','ada_v4p_score','ada_v5p_score','ada_v6p_score','gpt35','gpt4','nlptown','roberta15lg','textblob','vader']
  elif NOVEL_CUR == 'ttl_vf':
    model_subset_ls = ['nlptown','distilbert', 'gpt35', 'gpt4', 'roberta15lg', 'textblob', 'vader']
  else:
    print(f"ERROR: NOVEL_CUR={NOVEL_CUR} does not have a color_ls assigned.")

# Melt the DataFrame to have models and their sentiment scores in two columns
# melted_df = pd.melt(sentiment_df, id_vars=['line_no'], value_vars=model_ls,
melted_df = pd.melt(sentiment_df, id_vars=['line_no'], value_vars=model_subset_ls,
                    var_name='model', value_name='sentiment_score')

# Create a grid of histograms
g = sns.FacetGrid(melted_df, col="model", col_wrap=3, sharex=True, sharey=True)
g.map(plt.hist, 'sentiment_score', bins=10)

plt.subplots_adjust(top=0.85)  # Adjust the top to provide space for the title
g.fig.suptitle(f'Sentiment Value Distributions by Model\n{Novel_Title}', fontsize=14)


plt.show();


#### Culumative

In [ ]:
# DATA show a Pandas DataFrame where each row is a line_no
#   where sentiment values between -1.0-1.0 are recorded for 6 models in ['distilbert', 'gpt35', 'gpt4', 'roberta15lg', 'textblob', 'vader']
#   and difference is the max difference between any two of the 6 models.

# Create a beautiful seaborn line chart with 6 lines for each model with x-axis running from line_no in range(0,3700)
# and the y-axis showing a cumulative amount of times each model was either the min or max sentiment value in it's row.
# Add a key, gridlines alpha=0.3, and a title="Cumulative count of extreme sentiment values\nTo the Lighthouse by Virginia Woolf"

# Initialize DataFrame to hold cumulative counts
cumulative_df = pd.DataFrame(columns=model_ls)

# Initialize counters for each model
counters = dict.fromkeys(model_ls, 0)

# Iterate over DataFrame rows
# for i, row in sentiment_df.iterrows():
for i, row in tqdm(sentiment_df.iterrows(), total=len(sentiment_df)):
    # TypeError: reduction operation 'argmin' not allowed for this dtype
    # min_model = row[model_ls].idxmin()
    # max_model = row[model_ls].idxmax()

    min_model = model_ls[np.argmin(row[model_ls].values)]
    max_model = model_ls[np.argmax(row[model_ls].values)]

    counters[min_model] += 1
    counters[max_model] += 1

    # Add current counters to cumulative DataFrame
    cumulative_df = cumulative_df.append(counters, ignore_index=True)

# Set line_no as index for cumulative_df
cumulative_df.index = sentiment_df['line_no']

# Plot cumulative counts
plt.figure(figsize=(10, 8))
sns.lineplot(data=cumulative_df)
plt.grid(True, alpha=0.3)
plt.title("Cumulative count of extreme sentiment values\nTo the Lighthouse by Virginia Woolf")
plt.show();


In [ ]:
# Models for which sentiment values are recorded
model_subset_ls = model_ls # ['distilbert', 'gpt35', 'gpt4', 'roberta15lg', 'textblob', 'vader']

# Create an empty DataFrame with columns for each model. This DataFrame will be used to hold the cumulative counts.
cumulative_df = pd.DataFrame(columns=model_subset_ls)

# Initialize a counter for each model to zero. This counter will be used to track the number of times
# each model's sentiment value is the minimum or maximum in a row.
counters = dict.fromkeys(model_subset_ls, 0)

# Iterate over the rows of the sentiment DataFrame
for i, row in tqdm(sentiment_df.iterrows(), total=len(sentiment_df)):
    # Identify the model with the minimum and maximum sentiment value for the current row
    min_model = model_subset_ls[np.argmin(row[model_subset_ls].values)]
    max_model = model_subset_ls[np.argmax(row[model_subset_ls].values)]

    # Increment the counter for the models with the minimum and maximum sentiment values
    counters[min_model] += 1
    counters[max_model] += 1

    # Add the current values of the counters to the cumulative DataFrame
    cumulative_df = cumulative_df.append(counters, ignore_index=True)

# Set the line_no from the sentiment DataFrame as the index for the cumulative DataFrame
cumulative_df.index = sentiment_df['line_no']

# Plot the cumulative counts of the extreme sentiment values as a line chart.
# Each line in the chart represents one of the models.
plt.figure(figsize=(10, 8))
sns.lineplot(data=cumulative_df)
plt.grid(True, alpha=0.3)
plt.title("Cumulative count of extreme sentiment values\nTo the Lighthouse by Virginia Woolf")
plt.show()


#### Heatmap

In [ ]:
model_a = 'gpt35'
model_b = 'gpt4'
diff_threshold_min = 1.99

gpt_ct = 0
for idx, row in thresh_minmax_diff_df.iterrows():
    if row['difference'] >= diff_threshold_min:  # Adjusted condition
        print(f"Example #{gpt_ct}: {row['text_raw']}")
        print(f"     {model_a}: {row[model_a]}")
        print(f"     {model_b}:  {row[model_b]}")
        gpt_ct += 1

print(f"TOTAL {gpt_ct} examples where the difference column is greater or equal to {diff_threshold_min} out of {len(thresh_minmax_diff_df)}")


In [ ]:
model_a = 'gpt35'
model_b = 'gpt4'
diff_threshold_min = 1.99

gpt_ct = 0
for idx, row in thresh_minmax_diff_df.iterrows():
    print(f"row: {row}")
    if np.abs(row[model_a] - row[model_b]) >= diff_threshold_min:
        print(f"Example #{gpt_ct}: {row['text_raw']}")
        print(f"     {model_a}: {row[model_a]}")
        print(f"     {model_b}:  {row[model_b]}\n")
        gpt_ct += 1

print(f"TOTAL {gpt_ct} examples where {model_a} and {model_b} disagreed out of {len(thresh_minmax_diff_df)}")


In [ ]:
%whos DataFrame

In [ ]:
# TODO: Bug in Beloved (many cols) vs TTL

model_minmax_diff_df    .head()
model_minmax_diff_df    .info()

In [ ]:
np.abs(model_minmax_diff_df.iloc[10]['gpt35'] - model_minmax_diff_df.iloc[10]['vader'])

In [ ]:
model_minmax_diff_df

In [ ]:
thresh_minmax_diff_df.head()

In [ ]:
text_minmax_diff_df.head()

In [ ]:
# Find the sentences where model_a and model_b differ >= diff_threshold_min

model_a = 'gpt35'
model_b = 'gpt4'
diff_threshold_min = 1.7

gpt_ct = 0
# for idx, aline_no in enumerate(thresh_minmax_diff_df):
for idx, aline_no in enumerate(text_minmax_diff_df['difference']):
  print(type(aline_no))
  # print(f"{sentiment_df.iloc[aline_no]['text_raw']}\n")
  if np.abs(model_minmax_diff_df.iloc[aline_no][model_a] - model_minmax_diff_df.iloc[aline_no][model_b]) >= diff_threshold_min:
    print(f"Example #{gpt_ct}: {model_minmax_diff_df.iloc[aline_no]['text_raw']}")
    print(f"     {model_a}: {model_minmax_diff_df.iloc[aline_no][model_a]}")
    print(f"     {model_b}:  {model_minmax_diff_df.iloc[aline_no][model_b]}")
    gpt_ct += 1

print(f"TOTAL {gpt_ct} examples where {model_a} and {model_b} disagreed beyond threshold={diff_threshold} out of {len(model_minmax_diff_df)}")


In [ ]:
%whos DataFrame

In [ ]:
from itertools import combinations

def compute_model_diffs(df, model_ls):
    # Create all combinations of models
    comb = combinations(model_ls, 2)

    # For each combination of models, compute the absolute difference
    for pair in list(comb):
        model_a, model_b = pair
        df[f'diff_{model_a}_{model_b}'] = np.abs(df[model_a] - df[model_b])

    return df

# model_ls = ['nlptown', 'gpt35', 'gpt4', 'roberta15lg', 'textblob', 'vader']
model_diff_df = compute_model_diffs(model_minmax_diff_df, model_ls)

model_diff_df.head()


In [ ]:
%whos DataFrame

In [ ]:
model_minmax_diff_df.info()

In [ ]:
from itertools import combinations
import random

In [ ]:
# UPDATE 20240526 Heatmap

# Calculate the correlation matrix
correlation_matrix = sentiment_df[model_ls].corr()

# Set font size for all text elements
font_size = 12

# Plot the heatmap of the correlation matrix
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, fmt=".2f", cmap='coolwarm', xticklabels=model_ls, yticklabels=model_ls,
            annot_kws={"size": font_size}, cbar_kws={"ticks": np.linspace(-1, 1, 11), "label": "Correlation"})
plt.title(f"Correlation Between Sentiment Scores by Model\n{novel_title}", fontsize=font_size)
plt.xlabel("Models", fontsize=font_size)
plt.ylabel("Models", fontsize=font_size)
plt.xticks(fontsize=font_size)
plt.yticks(fontsize=font_size)
plt.show()


In [ ]:
model_subset_ls

In [ ]:
# Heatmap of min/max extreme model combination

MODEL_CORE_FL = True

if MODEL_CORE_FL:
  if NOVEL_CUR == 'b_tm':
    model_subset_ls = ['nlptown', 'gpt35', 'gpt4', 'roberta15lg', 'textblob', 'vader']
  elif NOVEL_CUR == 'ttl_vwoolf ':
    model_subset_ls = ['distilbert', 'gpt35', 'gpt4', 'roberta15lg', 'textblob', 'vader']
  else:
    print(f"ERROR: NOVEL_CUR={NOVEL_CUR} does not have a color_ls assigned.")
else:
  if NOVEL_CUR == 'b_tm':
    model_subset_ls = ['ada_v1p_score','ada_v2p_score','ada_v3p_score','ada_v4p_score','ada_v5p_score','ada_v6p_score','gpt35','gpt4','nlptown','roberta15lg','textblob','vader']
  elif NOVEL_CUR == 'ttl_vwoolf ':
    model_subset_ls = ['nlptown','distilbert', 'gpt35', 'gpt4', 'roberta15lg', 'textblob', 'vader']
  else:
    print(f"ERROR: NOVEL_CUR={NOVEL_CUR} does not have a color_ls assigned.")



# Initialize a square DataFrame with 0s to store the counts for each pair of models
# heatmap_df = pd.DataFrame(np.zeros((6, 6)), index=model_ls, columns=model_ls)
heatmap_df = pd.DataFrame(np.zeros((6, 6)), index=model_subset_ls, columns=model_subset_ls)

# Iterate through each row in the DataFrame
for i, row in sentiment_df.iterrows():
    # Get the model with the minimum and maximum sentiment score
    # min_values = row[model_ls][row[model_ls] == row[model_ls].min()]
    # max_values = row[model_ls][row[model_ls] == row[model_ls].max()]
    min_values = row[model_subset_ls][row[model_subset_ls] == row[model_subset_ls].min()]
    max_values = row[model_subset_ls][row[model_subset_ls] == row[model_subset_ls].max()]

    # If there are ties, randomly choose one model
    min_model = random.choice(min_values.index)
    max_model = random.choice(max_values.index)

    # Update the count for the pair of models
    heatmap_df.loc[min_model, max_model] += 1
    heatmap_df.loc[max_model, min_model] += 1

# Create a heatmap
plt.figure(figsize=(10, 10))
sns.heatmap(heatmap_df, annot=True, fmt=".0f", cmap='YlGnBu')
plt.title(f"Model Pairs with Extreme Sentiment Scores\n{novel_title}")
plt.show();


In [ ]:
sentiment_df.head()

In [ ]:

# Heatmap of min/max extreme model combination

MODEL_CORE_FL = True

if MODEL_CORE_FL:
  if NOVEL_CUR == 'b_tm':
    model_subset_ls = ['nlptown', 'gpt35', 'gpt4', 'roberta15lg', 'textblob', 'vader']
  elif NOVEL_CUR == 'ttl_vwoolf ':
    model_subset_ls = ['distilbert', 'gpt35', 'gpt4', 'roberta15lg', 'textblob', 'vader']
  else:
    print(f"ERROR: NOVEL_CUR={NOVEL_CUR} does not have a color_ls assigned.")
else:
  if NOVEL_CUR == 'b_tm':
    model_subset_ls = ['ada_v1p_score','ada_v2p_score','ada_v3p_score','ada_v4p_score','ada_v5p_score','ada_v6p_score','gpt35','gpt4','nlptown','roberta15lg','textblob','vader']
  elif NOVEL_CUR == 'ttl_vwoolf ':
    model_subset_ls = ['nlptown','distilbert', 'gpt35', 'gpt4', 'roberta15lg', 'textblob', 'vader']
  else:
    print(f"ERROR: NOVEL_CUR={NOVEL_CUR} does not have a color_ls assigned.")

# Initialize a square DataFrame with 0s to store the counts for each pair of models
heatmap_df = pd.DataFrame(np.zeros((6, 6)), index=model_subset_ls, columns=model_subset_ls)
# heatmap_df = pd.DataFrame(np.zeros((6, 6)), index=model_subset_ls, columns=model_subset_ls)

# Iterate through each row in the DataFrame
for i, row in sentiment_df.iterrows():
    # Get the model with the minimum and maximum sentiment score
    # min_values = row[model_ls][row[model_ls] == row[model_ls].min()]
    # max_values = row[model_ls][row[model_ls] == row[model_ls].max()]
    min_values = row[model_subset_ls][row[model_subset_ls] == row[model_subset_ls].min()]
    max_values = row[model_subset_ls][row[model_subset_ls] == row[model_subset_ls].max()]

    # If there are ties, randomly choose one model
    min_model = random.choice(min_values.index)
    max_model = random.choice(max_values.index)

    # Update the count for the pair of models
    heatmap_df.loc[min_model, max_model] += 1
    heatmap_df.loc[max_model, min_model] += 1

# Create a heatmap
plt.figure(figsize=(10, 10))
sns.heatmap(heatmap_df, annot=True, fmt=".0f", cmap='YlGnBu')
plt.title(f"Model Pairs with Extreme Sentiment Scores\n{Novel_Title}")
plt.show();


In [ ]:
# create six KDE smooth distributions

if NOVEL_CUR == 'b_tm':
  # upto 13 models (eg Beloved by Toni Morrison)
  color_ls = ['red', 'orange', 'yellow', 'green', 'blue', 'purple', 'pink', 'brown', 'gray', 'black', 'cyan', 'magenta', 'olive']
elif NOVEL_CUR == 'ttl_vf':
  # upto 6 models (eg To The Lighthouse by Virgina Woolf)
  color_ls = ['red', 'blue', 'green', 'purple', 'orange', 'gray']
else:
  print(f"ERROR: NOVEL_CUR={NOVEL_CUR} does not have a color_ls assigned.")

MODEL_CORE_FL = True

if MODEL_CORE_FL:
  if NOVEL_CUR == 'b_tm':
    model_subset_ls = ['nlptown', 'gpt35', 'gpt4', 'roberta15lg', 'textblob', 'vader']
  elif NOVEL_CUR == 'ttl_vf':
    model_subset_ls = ['distilbert', 'gpt35', 'gpt4', 'roberta15lg', 'textblob', 'vader']
  else:
    print(f"ERROR: NOVEL_CUR={NOVEL_CUR} does not have a color_ls assigned.")
else:
  if NOVEL_CUR == 'b_tm':
    model_subset_ls = ['ada_v1p_score','ada_v2p_score','ada_v3p_score','ada_v4p_score','ada_v5p_score','ada_v6p_score','gpt35','gpt4','nlptown','roberta15lg','textblob','vader']
  elif NOVEL_CUR == 'ttl_vf':
    model_subset_ls = ['nlptown','distilbert', 'gpt35', 'gpt4', 'roberta15lg', 'textblob', 'vader']
  else:
    print(f"ERROR: NOVEL_CUR={NOVEL_CUR} does not have a color_ls assigned.")



ten_percent = int(0.1*model_minmax_diff_df.shape[0])

for i, col in enumerate(model_subset_ls):
    sns.kdeplot(data=model_minmax_diff_df.iloc[:ten_percent][col], color=color_ls[i], alpha=0.2, linewidth=2, fill=True)

# add vertical dashed red lines with labels
plt.axvline(x=-1.0, color='red', linestyle='--', linewidth=4, label='-1.0 Min Sentiment Value')
plt.axvline(x=1.0, color='red', linestyle='--', linewidth=4, label='+1.0 Max Sentiment Value')
# plt.text(-1.0, 0.1, '-1.0 Min Sentiment Value', color='red', fontsize=12, rotation=90)
# plt.text(1.0, 0.1, '+1.0 Max Sentiment Value', color='red', fontsize=12, rotation=90)
plt.text(-0.95, 1.5, '-1.0 Min Sentiment Value', color='red', fontsize=12, rotation=90, ha='center')
plt.text(1.05, 1.5, '+1.0 Max Sentiment Value', color='red', fontsize=12, rotation=90, ha='center')


# add title and subtitle to the plot
# plt.suptitle('KDE Sentiment Value Distributions by Model', fontsize=16)
plt.title('KDE Sentiment Value Distributions by Model\nfor Top 10% Incoherent Sentiment Sentence Values\nTo The Lighthouse by Virginia Woolf', fontsize=16)

# add key to the plot
plt.legend(model_subset_ls)

# show the plot
plt.show();

In [ ]:
# Heatmap of min/max extreme model combination

MODEL_CORE_FL = True

if MODEL_CORE_FL:
  if NOVEL_CUR == 'b_tm':
    model_subset_ls = ['nlptown', 'gpt35', 'gpt4', 'roberta15lg', 'textblob', 'vader']
  elif NOVEL_CUR == 'ttl_vf':
    model_subset_ls = ['distilbert', 'gpt35', 'gpt4', 'roberta15lg', 'textblob', 'vader']
  else:
    print(f"ERROR: NOVEL_CUR={NOVEL_CUR} does not have a color_ls assigned.")
else:
  if NOVEL_CUR == 'b_tm':
    model_subset_ls = ['ada_v1p_score','ada_v2p_score','ada_v3p_score','ada_v4p_score','ada_v5p_score','ada_v6p_score','gpt35','gpt4','nlptown','roberta15lg','textblob','vader']
  elif NOVEL_CUR == 'ttl_vf':
    model_subset_ls = ['nlptown','distilbert', 'gpt35', 'gpt4', 'roberta15lg', 'textblob', 'vader']
  else:
    print(f"ERROR: NOVEL_CUR={NOVEL_CUR} does not have a color_ls assigned.")

cutoff_per = 10
cutoff_idx = int((cutoff_per/100)*model_minmax_diff_df.shape[0])
cutoff_df = model_minmax_diff_df.iloc[:cutoff_idx]

# Initialize a square DataFrame with 0s to store the counts for each pair of models
# heatmap_df = pd.DataFrame(np.zeros((6, 6)), index=model_ls, columns=model_ls)
heatmap_df = pd.DataFrame(np.zeros((6, 6)), index=model_subset_ls, columns=model_subset_ls)

# Iterate through each row in the DataFrame
for i, row in cutoff_df.iterrows():
    # Get the model with the minimum and maximum sentiment score
    # min_values = row[model_ls][row[model_ls] == row[model_ls].min()]
    # max_values = row[model_ls][row[model_ls] == row[model_ls].max()]
    min_values = row[model_subset_ls][row[model_subset_ls] == row[model_subset_ls].min()]
    max_values = row[model_subset_ls][row[model_subset_ls] == row[model_subset_ls].max()]

    # If there are ties, randomly choose one model
    min_model = random.choice(min_values.index)
    max_model = random.choice(max_values.index)

    # Update the count for the pair of models
    heatmap_df.loc[min_model, max_model] += 1
    heatmap_df.loc[max_model, min_model] += 1

# Create a heatmap
plt.figure(figsize=(10, 10))
sns.heatmap(heatmap_df, annot=True, fmt=".0f", cmap='YlGnBu')
plt.title(f"Model Pairs with Extreme {cutoff_per}% Divergent Sentiment Scores\n{Novel_Title}")
plt.show();


### Plot StandardScaler Normalized

In [ ]:
# Compute the mean of each raw Sentiment Timeseries and adjust to [-1.0, 1.0] Range

model_samelen_adj_mean_dt = {}

for amodel in model_ls:
  amodel_min = sentiment_df[amodel].min()
  amodel_max = sentiment_df[amodel].max()
  amodel_range = amodel_max - amodel_min
  amodel_raw_mean = sentiment_df[amodel].mean()

  if amodel_range > 2.0:
    model_samelen_adj_mean_dt[amodel] = (amodel_raw_mean + amodel_min)/(amodel_max - amodel_min)*2 + -1.0
  elif amodel_range < 1.1:
    model_samelen_adj_mean_dt[amodel] = (amodel_raw_mean + amodel_min)/(amodel_max - amodel_min)*2 + -1.0
  else:
    model_samelen_adj_mean_dt[amodel] = amodel_raw_mean

  print(f'Model: {amodel}\n  Raw Mean: {amodel_raw_mean}\n  Adj Mean: {model_samelen_adj_mean_dt[amodel]}\n  Min: {amodel_min}\n  Max: {amodel_max}\n  Range: {amodel_range}\n')

In [ ]:
sentiment_all_norm_df.head()
sentiment_all_norm_df.info()

In [ ]:
# Normalize Timeseries with StandardScaler (u=0, sd=+/- 1)
model_all_ls = model_ls + ['distilbert']

# sentiment_all_norm_df = pd.DataFrame()
sentiment_all_norm_df = sentiment_df[['line_no','text_raw','text_clean']].copy(deep=True)
sentiment_all_norm_df[model_all_ls] = StandardScaler().fit_transform(sentiment_df[model_all_ls])
sentiment_all_norm_df.head()


In [ ]:
# UPDATE 20240526
# _ = sentiment_df[model_ls].rolling(win_size, min_periods=1, center=True).mean().plot(grid=True)

# Parameters
win_per = 10  # Window size percent
win_size = int((win_per / 100) * sentiment_df.shape[0])  # Calculate window size
if win_size % 2 == 0:
    win_size += 1  # Ensure win_size is odd as required by S-G Algo

# Compute the rolling mean and center around mean=0
rolling_mean_centered = sentiment_df[model_ls].rolling(win_size, min_periods=1, center=True).mean()
rolling_mean_centered = rolling_mean_centered - rolling_mean_centered.mean()

# Plot the centered rolling means
font_size = 12  # Uniform font size for all elements

plt.figure(figsize=(12, 8))
rolling_mean_centered.plot(grid=True, ax=plt.gca())
plt.axhline(0, color='black', linestyle='--', linewidth=0.5)
plt.title(f"Centered Rolling Mean of Sentiment Scores\n{novel_title}", fontsize=font_size)
plt.xlabel("Index", fontsize=font_size)
plt.ylabel("Sentiment", fontsize=font_size)
plt.xticks(fontsize=font_size)
plt.yticks(fontsize=font_size)
plt.legend(fontsize=font_size)
plt.show()

In [ ]:
# Plot Normalized Time Series to same mean

# ax = sentiment_all_norm_df[model_all_ls].rolling(win_size, center=True).mean().plot(grid=True, colormap='Dark2', lw=2)
ax = sentiment_all_norm_df[model_ls].rolling(win_size, center=True).mean().plot(grid=True, colormap='Dark2', lw=2)

ax.title.set_text(f'Sentiment Analysis \n {novel_title} \n Normalization: Standard Scaler')

plt.show();


In [ ]:
NOVEL_LS

In [ ]:
if NOVEL_CUR == 'b_tm':
  model_gpt_ls = ['gpt35','gpt4','ada_v1p_score','ada_v2p_score','ada_v3p_score','ada_v4p_score','ada_v5p_score','ada_v6p_score']

  # Plot subset of Normalized Timeseries to same mean

  ax = sentiment_all_norm_df[model_gpt_ls].rolling(win_size, center=True).mean().plot(grid=True, colormap='Dark2', lw=3)
  ax.title.set_text(f'Sentiment Analysis \n {Novel_Title} \n Normalization: Standard Scaler')

  plt.show();


In [ ]:
model_main_ls = ['vader', 'textblob', 'nlptown', 'roberta15lg', 'gpt35', 'gpt4'] # , 'distilbert']
model_main_ls = model_subset_ls

# Plot subset of Normalized Timeseries to same mean

ax = sentiment_all_norm_df[model_main_ls].rolling(win_size, center=True).mean().plot(grid=True, colormap='Dark2', lw=2)
ax.title.set_text(f'Sentiment Analysis \n {novel_title} \n Normalization: Standard Scaler')

plt.show();

### Secondary SG-Smoothing

Savitzky-Golay filtering. Savitzky-Golay filtering is a smoothing method that can effectively preserve important features, such as peaks and valleys, while reducing noise.

Savitzky-Golay filtering fits a polynomial to small subsets of data points within a sliding window and uses the polynomial coefficients to estimate the smoothed values. This technique can provide better preservation of local features compared to simple moving average smoothing.

In [ ]:
from scipy.signal import savgol_filter

In [ ]:
%whos DataFrame

In [ ]:
sentiment_zscore_df.head()
sentiment_zscore_df.info()
sentiment_zscore_df.describe()

In [ ]:
def plot_sma_sv(dataframe_in, model_cols, win_per=10, polynomial_order=3):
    # Create an empty DataFrame to store the smoothed values
    sentiment_sg_df = pd.DataFrame()

    # Calculate the window size for the moving average (win_per% of the data length)
    win_size = max(1, int((win_per / 100) * dataframe_in.shape[0]))
    if win_size % 2 == 0:
        win_size += 1  # Ensure window size is odd for centering

    # Apply SMA and Savitzky-Golay filtering to each model in model_cols
    for amodel in model_cols:
        print(f"Processing Model: {amodel}")

        # Apply savgol_filter to the rolling mean of the normalized sentiment scores
        rolling_mean = dataframe_in[amodel].rolling(window=win_size, min_periods=1, center=True).mean()
        smoothed_values = savgol_filter(rolling_mean, win_size, polynomial_order)
        sentiment_sg_df[amodel] = smoothed_values

        # Plot the smoothed sentiment scores with a key
        sentiment_sg_df[amodel].plot(label=amodel)

    # Set the title of the plot
    plt.title(f"Sentiment Analysis\n{novel_title}\nZ-Score > 10% SMA > SG 10%")
    plt.xlabel("Index")
    plt.ylabel("Smoothed Values")
    plt.legend(model_ls, fontsize=10, loc='upper right')  # Add a legend to the plot
    plt.grid(True)
    plt.tight_layout()
    plt.show();

plot_sma_sv(sentiment_zscore_df, model_input_ls, win_per=10, polynomial_order=3)


In [ ]:
#Apply sequentially SMA 10% then Savitzky-Golay filtering

sentiment_sg_df = pd.DataFrame()

# Assuming your time series is stored in the variable 'data'
win_per = 10 # Window size percent
win_size = int((win_per/100)*sentiment_zscore_df.shape[0])  # Adjust the window size as needed
# Ensure win_size is odd as required by S-G Algo
if win_size % 2 == 0:
  # If not, add 1
  win_size += 1

polynomial_order = 3  # Adjust the polynomial order as needed

for amodel in model_subset_ls:
  print(f"Processing Model: {amodel}")

  # Apply savgol_filter to the rolling mean of the normalized sentiment scores
  sentiment_sg_df[amodel] = savgol_filter(sentiment_zscore_df[amodel].rolling(win_size, min_periods=1, center=True).mean(), win_size, polynomial_order)

  # Set the title of the plot
  title = f"Sentiment Analysis: Normed & Double Smoothed\n (Standard Scaler + SMA 10% + SG 10%)\n{novel_title}"

  # Plot the smoothed sentiment scores with a key
  _ = sentiment_sg_df[amodel].plot(title=title, label=amodel)
  plt.legend(); # Add a legend to the plot

### Pearson Correlation Heatmap

In [ ]:
def make_pearson_heat(dataframe_in, model_cols, novel_title):
    # Calculate the Pearson correlation matrix
    correlation_matrix = dataframe_in[model_cols].corr(method='pearson')

    # Plot the heatmap
    plt.figure(figsize=(10, 8))
    sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0, vmin=-1, vmax=1)
    plt.title(f"Pearson Correlation Heatmap\n{novel_title}\nZ-Score Norm > 10% SMA > 10% S-G Smoothed")
    plt.show()

make_pearson_heat(sentiment_zscore_df, model_input_ls, novel_title)

**[SKIP]**

In [ ]:
sentiment_sg_df = pd.DataFrame()

win_per = 10  # Window size percent
win_size = int((win_per / 100) * sentiment_all_norm_df.shape[0])  # Adjust the window size as needed
# Ensure win_size is odd as required by S-G Algo
if win_size % 2 == 0:
    # If not, add 1
    win_size += 1

polynomial_order = 3  # Adjust the polynomial order as needed

for amodel in model_subset_ls:
    print(f"Processing Model: {amodel}")

    # Apply savgol_filter to the rolling mean of the normalized sentiment scores
    sentiment_sg_df[amodel] = savgol_filter(
        sentiment_all_norm_df[amodel].rolling(win_size, min_periods=1, center=True).mean(),
        win_size,
        polynomial_order
    )

# Set the title of the plot
title = f"Euclidean Distance Between Sentiment Time Series: Normed & Double Smoothed\n (Standard Scaler + SMA 10% + SG 10%)\n{Novel_Title}"

# Heatmap of Euclidean Distance of Norm/SMA/SG smoothed model time series

# Calculate the pairwise Euclidean distance matrix using the smoothed data
euclidean_distance_matrix = np.linalg.norm(
    sentiment_sg_df.values[:, :, np.newaxis] - sentiment_sg_df.values[:, np.newaxis, :],
    axis=0
)

# Plot the heatmap of the distance matrix
sns.heatmap(euclidean_distance_matrix, cmap='YlGnBu', xticklabels=sentiment_sg_df.columns,
            yticklabels=sentiment_sg_df.columns)
plt.title(title)
plt.show()


**[SKIP] To next Section

In [ ]:


sentiment_sg_df = pd.DataFrame()

win_per = 10  # Window size percent
win_size = int((win_per / 100) * sentiment_all_norm_df.shape[0])  # Adjust the window size as needed
# Ensure win_size is odd as required by S-G Algo
if win_size % 2 == 0:
    # If not, add 1
    win_size += 1

polynomial_order = 3  # Adjust the polynomial order as needed

for amodel in model_subset_ls:
    print(f"Processing Model: {amodel}")

    # Apply savgol_filter to the rolling mean of the normalized sentiment scores
    sentiment_sg_df[amodel] = savgol_filter(
        sentiment_all_norm_df[amodel].rolling(win_size, min_periods=1, center=True).mean(),
        win_size,
        polynomial_order
    )

    # Set the title of the plot
    title = f"Sentiment Analysis: Normed & Double Smoothed\n (Standard Scaler + SMA 10% + SG 10%)\n{Novel_Title}"

    # Plot the smoothed sentiment scores with a key
    sentiment_sg_df[amodel].plot(title=title, label=amodel)

# Add a legend to the plot
plt.legend()

# Heatmap of Euclidean Distance of Norm/SMA/SG smoothed model time series

# Calculate the pairwise Euclidean distance matrix using the smoothed data
euclidean_distance_matrix = np.linalg.norm(
    sentiment_sg_df.values[:, :, np.newaxis] - sentiment_sg_df.values[:, np.newaxis, :],
    axis=0
)

# Plot the heatmap of the distance matrix
sns.heatmap(euclidean_distance_matrix, xticklabels=sentiment_sg_df.columns, yticklabels=sentiment_sg_df.columns)
plt.title("Euclidean distance between all time series in sentiment_sg_df")
plt.show()


In [ ]:
# Heatmap of Euclidean Distance of Norm/SMA/SG smoothed model time series

# Calculate the pairwise Euclidean distance matrix using the smoothed data
euclidean_distance_matrix = np.linalg.norm(sentiment_sg_df[:, np.newaxis] - smoothed_data, axis=2)

# Plot the heatmap of the distance matrix
sns.heatmap(euclidean_distance_matrix, xticklabels=sentiment_dg_df.columns, yticklabels=sentiment_dg_df.columns)
plt.title("Euclidean distance between all time series in sentiment_dg_df")

In [ ]:
# UPDATE 20240526 Efficient S-G

# Parameters
win_per = 10  # Window size percent
win_size = int((win_per / 100) * sentiment_df.shape[0])  # Calculate window size
if win_size % 2 == 0:
    win_size += 1  # Ensure win_size is odd as required by S-G Algo
polynomial_order = 3  # Polynomial order for S-G filter

# Initialize DataFrame for smoothed data
sentiment_sg_df = pd.DataFrame()

# Apply filters
for amodel in model_ls:
    print(f"Processing Model: {amodel}")

    # Apply rolling mean
    rolling_mean = sentiment_df[amodel].rolling(win_size, min_periods=1, center=True).mean()

    # Apply Savitzky-Golay filter
    smoothed = savgol_filter(rolling_mean, win_size, polynomial_order)

    # Center the smoothed data around mean = 0
    smoothed_centered = smoothed - np.mean(smoothed)

    sentiment_sg_df[amodel] = smoothed_centered

# Check the DataFrame after processing
print("Smoothed and Centered DataFrame Head:")
print(sentiment_sg_df.head())

# Plot the smoothed and centered sentiment scores
plt.figure(figsize=(12, 8))
title = f"Sentiment Analysis: Normed & Double Smoothed\n (Standard Scaler + SMA 10% + SG 10%)\n{novel_title}"
plt.title(title, fontsize=12)
for amodel in model_ls:
    plt.plot(sentiment_sg_df[amodel], label=amodel)
plt.legend(fontsize=8)
plt.xlabel("Index", fontsize=7)
plt.ylabel("Sentiment", fontsize=7)
plt.axhline(0, color='black', linewidth=0.5, linestyle='--')
plt.show();

"""
# Calculate the pairwise Euclidean distance matrix
# Optimize by using efficient NumPy operations
sentiment_sg_values = sentiment_sg_df.values
euclidean_distance_matrix = np.sqrt(((sentiment_sg_values[:, np.newaxis, :] - sentiment_sg_values[np.newaxis, :, :]) ** 2).sum(axis=2))

# Check the distance matrix dimensions and a sample
print("Euclidean Distance Matrix Shape:", euclidean_distance_matrix.shape)
print("Euclidean Distance Matrix Sample:")
print(euclidean_distance_matrix[:5, :5])


# Plot the heatmap of the distance matrix
plt.figure(figsize=(10, 8))
sns.heatmap(euclidean_distance_matrix, xticklabels=model_ls, yticklabels=model_ls, cmap='viridis', annot=True, fmt=".2f", annot_kws={"size": 8})
plt.title("Euclidean Distance Between All Time Series in Sentiment SG DF", fontsize=12)
plt.xlabel("Models", fontsize=10)
plt.ylabel("Models", fontsize=10)
plt.xticks(fontsize=8)
plt.yticks(fontsize=8)
plt.show()
""";

In [ ]:

# Parameters
win_per = 10  # Window size percent
win_size = int((win_per / 100) * sentiment_df.shape[0])  # Calculate window size
if win_size % 2 == 0:
    win_size += 1  # Ensure win_size is odd as required by S-G Algo
polynomial_order = 3  # Polynomial order for S-G filter

# Initialize DataFrame for smoothed data
sentiment_sg_df = pd.DataFrame()

# Apply filters
for amodel in model_ls:
    print(f"Processing Model: {amodel}")

    # Apply rolling mean
    rolling_mean = sentiment_df[amodel].rolling(win_size, min_periods=1, center=True).mean()

    # Apply Savitzky-Golay filter
    smoothed = savgol_filter(rolling_mean, win_size, polynomial_order)

    # Center the smoothed data around mean = 0
    smoothed_centered = smoothed - np.mean(smoothed)

    sentiment_sg_df[amodel] = smoothed_centered

# Plot the smoothed and centered sentiment scores
plt.figure(figsize=(12, 8))
title = f"Sentiment Analysis: Normed & Double Smoothed\n (Standard Scaler + SMA 10% + SG 10%)\n{novel_title}"
plt.title(title, fontsize=12)
for amodel in model_ls:
    plt.plot(sentiment_sg_df[amodel], label=amodel)
plt.legend(fontsize=8)
plt.xlabel("Index", fontsize=7)
plt.ylabel("Sentiment", fontsize=7)
plt.axhline(0, color='black', linewidth=0.5, linestyle='--')
plt.show();

"""
# Calculate the pairwise Euclidean distance matrix
euclidean_distance_matrix = np.linalg.norm(sentiment_sg_df.values[:, np.newaxis] - sentiment_sg_df.values, axis=2)

# Plot the heatmap of the distance matrix
plt.figure(figsize=(10, 8))
sns.heatmap(euclidean_distance_matrix, xticklabels=model_ls, yticklabels=model_ls, cmap='viridis', annot=True, fmt=".2f", annot_kws={"size": 8})
plt.title("Euclidean Distance Between All Time Series in Sentiment SG DF", fontsize=12)
plt.xlabel("Models", fontsize=10)
plt.ylabel("Models", fontsize=10)
plt.xticks(fontsize=8)
plt.yticks(fontsize=8)
plt.show();
"""

In [ ]:


#Apply sequentially SMA 10% then Savitzky-Golay filtering

sentiment_sg_df = pd.DataFrame()

# Assuming your time series is stored in the variable 'data'
win_per = 10 # Window size percent
win_size = int((win_per/100)*sentiment_all_norm_df.shape[0])  # Adjust the window size as needed
# Ensure win_size is odd as required by S-G Algo
if win_size % 2 == 0:
  # If not, add 1
  win_size += 1

polynomial_order = 3  # Adjust the polynomial order as needed

for amodel in model_subset_ls:
  print(f"Processing Model: {amodel}")

  # Apply savgol_filter to the rolling mean of the normalized sentiment scores
  sentiment_sg_df[amodel] = savgol_filter(sentiment_all_norm_df[amodel].rolling(win_size, min_periods=1, center=True).mean(), win_size, polynomial_order)

  # Set the title of the plot
  title = f"Sentiment Analysis: Normed & Double Smoothed\n (Standard Scaler + SMA 10% + SG 10%)\n{Novel_Title}"

  # Plot the smoothed sentiment scores with a key
  _ = sentiment_sg_df[amodel].plot(title=title, label=amodel)
  plt.legend(); # Add a legend to the plot

# Heatmap of Euclidean Distance of Norm/SMA/SG smoothed model time series

# Calculate the pairwise Euclidean distance matrix using the smoothed data
euclidean_distance_matrix = np.linalg.norm(sentiment_sg_df[:, np.newaxis] - smoothed_data, axis=2)

# Plot the heatmap of the distance matrix
sns.heatmap(euclidean_distance_matrix, xticklabels=sentiment_dg_df.columns, yticklabels=sentiment_dg_df.columns)
plt.title("Euclidean distance between all time series in sentiment_dg_df")

In [ ]:
for amodel in model_subset_ls:
  print(f"Processing Model: {amodel}")

  # Apply savgol_filter to the rolling mean of the normalized sentiment scores
  sentiment_sg_df[amodel] = savgol_filter(sentiment_all_norm_df[amodel].rolling(win_size, min_periods=1, center=True).mean(), win_size, polynomial_order)


In [ ]:
# Heatmap of Euclidean Distance of Norm/SMA/SG smoothed model time series

# Calculate the pairwise Euclidean distance matrix using the smoothed data
euclidean_distance_matrix = np.linalg.norm(sentiment_sg_df[:, np.newaxis] - smoothed_data, axis=2)

# Plot the heatmap of the distance matrix
sns.heatmap(euclidean_distance_matrix, xticklabels=sentiment_dg_df.columns, yticklabels=sentiment_dg_df.columns)
plt.title("Euclidean distance between all time series in sentiment_dg_df")

In [ ]:
#Apply sequentially SMA 10% then Savitzky-Golay filtering

sentiment_sg_df = pd.DataFrame()

# Assuming your time series is stored in the variable 'data'
win_per = 10 # Window size percent
win_size = int((win_per/100)*sentiment_all_norm_df.shape[0])  # Adjust the window size as needed
# Ensure win_size is odd as required by S-G Algo
if win_size % 2 == 0:
  # If not, add 1
  win_size += 1

polynomial_order = 3  # Adjust the polynomial order as needed

for amodel in model_subset_ls:
  print(f"Processing Model: {amodel}")
  # Apply Savitzky-Golay filtering
  # ax = sentiment_all_norm_df[model_gpt_ls].rolling(win_size, center=True).mean().plot(grid=True, colormap='Dark2', lw=3)
  # ax.title.set_text(f'Sentiment Analysis \n {Novel_Title} \n Normalization: Standard Scaler')
  ax = sentiment_sg_df[amodel] = savgol_filter(sentiment_all_norm_df[amodel].rolling(win_size, min_periods=1, center=True).mean(), win_size, polynomial_order)
  ax.title.set_text(f"Sentiment Analysis: Normed & Double Smoothed\n (Standard Scaler + SMA 10% + SG 10%)\n{novel_title}")
  sentiment_sg_df[amodel].plot()
  # Calculate Euclidean distance using the smoothed data
  # euclidean_distance = np.linalg.norm(smoothed_data - other_time_series)


### TS Euclidian Distance Metrics

In [ ]:
# Normalize Timeseries with StandardScaler (u=0, sd=+/- 1)
model_all_ls = model_ls + ['distilbert']

# sentiment_all_norm_df = pd.DataFrame()
sentiment_all_norm_df = sentiment_df[['line_no','text_raw','text_clean']].copy(deep=True)
sentiment_all_norm_df[model_all_ls] = StandardScaler().fit_transform(sentiment_df[model_all_ls])
sentiment_all_norm_df.head()

### In/coherence Plots

In [ ]:
# ORIGINAL FULL Ensemble

# TODO: Here and everywhere, replace model_mail_ls with model_subset_ls
# model_main_ls = model_ls
# model_main_ls = model_subset_ls
model_main_ls = model_ls

# Compute rolling mean dataframe
rolling_mean_df = sentiment_df.rolling(win_size, min_periods=1, center=True).mean()

# Compute range dataframe
# incoherence_df = rolling_mean_df[model_ls].apply(lambda x: np.abs(x.max() - x.min()), axis=1)

# Create subplot with 2 rows, 1 column
fig, ax = plt.subplots(2, 1, sharex=True, gridspec_kw={'height_ratios': [4, 1]})

# Plot main sentiment series
rolling_mean_df.plot(ax=ax[0], grid=True, colormap='Dark2', lw=2)
ax[0].title.set_text(f'Sentiment Analysis \n {novel_title} \n Normalization: Standard Scaler')

# Plot range series
# incoherence_df.plot(ax=ax[1], grid=True, color='red')

# Invert Y-axis and add labels
# ax[1].invert_yaxis()
ax[1].set_title('Ensemble Incoherence')
# ax[1].set_ylabel('incoherence')

# Show the plot
plt.show();


In [ ]:
# REMOVE: outlier model(s)

model_remove_ls = ['distilbert'] # ['nlptown']
model_main_ls = model_ls

# Remove the strings from the large list
for amodel in model_remove_ls:
    try:
        model_main_ls.remove(amodel)
    except ValueError:
        print("The string {} does not exist in the large list.".format(string))

model_main_ls

In [ ]:
sentiment_all_norm_df.info()

In [ ]:
incoherence_df.info()

In [ ]:
# Plot Emsemble centered around 0 mean

# Increase font sizes
plt.rcParams.update({
    'axes.titlesize': 30,
    'axes.labelsize': 27,
    'xtick.labelsize': 24,
    'ytick.labelsize': 24,
    'legend.fontsize': 27,
    'figure.titlesize': 33
})

# Compute the rolling mean dataframe
rolling_mean_df = sentiment_df[model_ls].rolling(win_size, min_periods=1, center=True).mean()

# Center each rolling mean time series around 0 baseline
for col in model_ls:
    rolling_mean_df[col] = rolling_mean_df[col] - rolling_mean_df[col].mean()

#
# Compute range dataframe
incoherence_df = rolling_mean_df.apply(lambda x: np.abs(x.max() - x.min()), axis=1)
coherence_df = incoherence_df * -1.0

# Create subplot with 2 rows, 1 column
fig, ax = plt.subplots(2, 1, sharex=True, gridspec_kw={'height_ratios': [4, 1]})

# Plot main sentiment series
rolling_mean_df.plot(ax=ax[0], grid=True, colormap='Dark2', lw=2)
ax[0].title.set_text(f'Sentiment Analysis \n {novel_title} \n Normalization: Standard Scaler')

# Plot incoherence_df series
coherence_df.plot(ax=ax[1], grid=True, color='red')

# Invert Y-axis and add labels
# ax[1].invert_yaxis()
ax[1].set_title('Ensemble Incoherence')
ax[1].set_ylabel('coherence')

# Show the plot
plt.show();


# **END OF NOTEBOOK**